In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 오하나 도곡점
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%98%A4%ED%95%98%EB%82%98%20%EB%8F%84%EA%B3%A1%EC%A0%90/place/1931511433?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 오하나 도곡점
업종 카테고리: 카페,디저트
별점: 방문자 리뷰 30
방문자 리뷰: 3
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 10


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 우연히 들려본 곳인데 가격이 합리적이고 분위기가 좋아용
샐러드랑 샌드위치 종류가 다양해서 하나씩 맛보고 싶어용 (2024년 8월 9일 금요일) - 작성자: 팅커별 (사진 17)
리뷰 2: 강남에서 샌드위치가 6500원이라
괜찮네요 ㅎㅎ 한끼먹기 너무좋습니다 (2024년 2월 26일 월요일) - 작성자: kim**** (사진 678)
리뷰 3: 샐러드의 경우 다양한 옵션이 가능했고, 샌드위치의 종류도 다양했습니다. 가게는 크기가 크지 않지만 사람이 많지는 않아서 충분히 앉을 수 있었어요. 근방에 들를 샐러드집이 애매했는데 점심에 자주 올 수 있을 것 같은 곳을 찾아서 좋네요! 건강하게 점심 먹고 싶을 때 딱인듯합니다. (2024년 2월 16일 금요일) - 작성자: 배가 꾸룩거리네 (사진 4)
리뷰 4: 굿 (2024년 8월 1일 목요일) - 작성자: 루비코비1024 (사진 617)
리뷰 5: 친절해요.
샌드위치 종류 다양해요.
신선해요. (2024년 1월 2일 화요일) - 작성자: hellomysun (사진 123)
리뷰 6: 점심 시간에 간편하고 맛있게 먹기 좋아서 종종 방문합니다~ 에그단호박, 바질크림치킨 샌드위치 너무 맛있어요! (2024년 1월 31일 수요일) - 작성자: 은디 (사진 12)
리뷰 7: 사장님도 친절하시고 양도 많고 맛도 좋아요! (2024년 1월 3일 수요일) - 작성자: iys**** (사진 92)
리뷰 8: 사장님 친절하시고 맛있어요. 음식 나오는건 시간 좀 걸려요. (2023년 11월 1일 수요일) - 작성자: iys**** (사진 92)
리뷰 9: 가게는 좁지만 맛있고 사장님 직원분 다 친절하세요. (2023년 10월 20일 금요일) - 작성자: iys**** (사진 92)
리뷰 10: 맛있어용 (2023년 11

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 두메 도곡점
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%91%90%EB%A9%94%20%EB%8F%84%EA%B3%A1%EC%A0%90/place/36932173?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 두메 도곡점
업종 카테고리: 한식
별점: 4.32
방문자 리뷰: 334
블로그 리뷰: 67
리뷰 탭 클릭 완료
총 리뷰 수: 131


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 회사 주변 1등 맛집 두메!!!!!✨️✨️✨️
우리팀 모두 제일 좋아하는 대존맛 맛집이에요
일단 낙지볶음 최강. 낙지가 엄청 토실하고 부드럽고요
곤드레나물밥 진짜 찰기 윤기 엄청나고 그냥 밥만 먹어도 맛있어요 글고 반찬들이 어쩜이리 잘나오는지ㅠ 반찬들도 하나같이 다 맛있고, 건강한 손맛 그 자체에요. 그중 저는 취나물 무친게 제일 맛있어요ㅎ 사장님도 넘 친절하셔서 반찬 먼저 더 주신다고 하고 양껏 주시고요
우리팀 주 1회 꼭 먹으러 가는 점심 맛집입니다 ㅎ 사장님 오래오래 해주세요~ ><
지난번 보쌈 정식 먹어봤는데 그것도 맛있음! (2024년 7월 31일 수요일) - 작성자: 밍쩐찐 (사진 94)
리뷰 2: 낙지정식먹었습니다
반찬처럼 나오고요 그냥 다 맛있어요 그냥 백반집이라고 생각함 될거같아요 곤드레밥 진짜 맛있네요 (2024년 7월 31일 수요일) - 작성자: cartunist (사진 600)
리뷰 3: 가끔가는 집인데요..
여기 음식이 너무 맛있어서..
힝상 사진찍는걸 까먹고 만다능....
오늘도 다 먹고 나서..
계산할때야 생각이 나서..
뒤늦게 가게 사진이라도 찍었네요..

이 집은요..
음식이 좀 건강한 집입니다.
반찬들도 하나같이 건강한 반찬들..
나물, 두부, 묵,.. 요런 스타일..
반찬 리필되구용~
채식주의자들이 좋아하는 식당일꺼에용~~ ㅎ (2024년 5월 13일 월요일) - 작성자: idisjien94 (사진 1,088)
리뷰 4: 들깨수제비 먹었어요.
반찬이 6가지 깍두기, 오이지, 두부조림, 취나물, 묵들깨무침, 부침개
맛없는 반찬 하나도 없고 사장님 친철하시고 만족하며 식사했어요.
만 원이 아깝지 않은.. (2024년 8월 7일 수요일) - 작성자: narara13 (사진 71)
리뷰 5: 여긴 가도가도 맛있어요. 자주 생각나

로드된 리뷰 개수: 100, 수집된 리뷰 개수: 90
리뷰 91: 맛있어요 (2022년 11월 1일 화요일) - 작성자: kky**** (Unknown)
리뷰 92: 물은 주지도 않고
젓가락 없다고 하니 뭉탱이로 집어던지듯 상에 놓고가버리고 친절은 찿아볼 수가없네요.
음식맛이 있건없건 기본이 안되어있어서 내돈 내고 먹는데도 다시는 가고싶지않은 그런 가게 였습니다. (2022년 3월 11일 금요일) - 작성자: 토토허니 (사진 46)
리뷰 93: 굿 (2022년 6월 23일 목요일) - 작성자: hae**** (사진 9)
리뷰 94: 맛있어요 (2022년 4월 21일 목요일) - 작성자: shally9113 (사진 135)
리뷰 95: 맛있어요 (2022년 3월 8일 화요일) - 작성자: iys**** (사진 92)
리뷰 96: 맛있어요❤️ (2021년 12월 2일 목요일) - 작성자: 고독한쭌식가 (사진 1,248)
리뷰 97: ㅏ (2022년 2월 18일 금요일) - 작성자: h45**** (Unknown)
리뷰 98: ㅏ (2022년 2월 28일 월요일) - 작성자: h45**** (Unknown)
리뷰 99: ㅏ (2022년 2월 3일 목요일) - 작성자: h45**** (Unknown)
리뷰 100: ㅏ (2022년 1월 12일 수요일) - 작성자: h45**** (Unknown)
로드된 리뷰 개수: 110, 수집된 리뷰 개수: 100
리뷰 101: 너무너무 맛있어요 (2021년 9월 16일 목요일) - 작성자: 고독한쭌식가 (사진 1,248)
리뷰 102: 굿 (2022년 2월 15일 화요일) - 작성자: BE47 (사진 18)
리뷰 103: 1번 (2021년 12월 7일 화요일) - 작성자: cutyguy1 (사진 36)
리뷰 104: 매장이 깔끔해요 (2021년 11월 5일 금요일) - 작성자: 푸른날개21 (사진 183)
리뷰 105: 질기네요 (2021년 11월 18일 목요일) - 작성자: h45**** (Unknown)
리뷰 106: 좋아요 

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 고래똥
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EA%B3%A0%EB%9E%98%EB%98%A5/place/1035640949?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 고래똥
업종 카테고리: 포장마차
별점: 4.7
방문자 리뷰: 106
블로그 리뷰: 49
리뷰 탭 클릭 완료
총 리뷰 수: 54


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 맛있는 안주와 함께 2차한다면 무조건 추천합니다!
시그니처인 꼬막은 철이 아니라 못먹어서 아쉬웠네요. (2024년 7월 16일 화요일) - 작성자: 아름다운날에는 (사진 83)
리뷰 2: 쭈꾸미가 맛나는집이에요..그치만 배불러서 제대로 못먹었어요
배불러도 파인애플샤베트는 잘들어갑니당 (2024년 3월 12일 화요일) - 작성자: 하이빈나 (사진 727)
리뷰 3: 친구 만나러 갔다 우연히 방문했는데 너무 만족했던 술집, 고래똥...!!
❤️석화, 소라숙회, 연어머리구이, 감자전, 짜파게티 뭐 하나 맛있지 않은 메뉴가 없었어요. 지금 당장이라도 또 갈 수 있을 것 같아요.
🩷생맥주도 있고 기본으로 나오는 콩나물국 정말 맛있어서 2번 리필해 먹었습니다.근데 무서운 점은, 여기는 보통
💜쭈꾸미를 많이 먹으러 오는 곳이라는 거에요. 저흰 아직 메인을 안 먹어본 상태라는 것... 다음엔 쭈꾸미부터 조지러 오겠습니다. (2023년 12월 19일 화요일) - 작성자: 홍미식가 (사진 1,251)
리뷰 4: 네명이서 안주 네개 뿌시고옴

고래똥 최고 (2024년 1월 26일 금요일) - 작성자: 2H Traveler (사진 1,876)
리뷰 5: 친절맛도리그잡채 (2024년 3월 20일 수요일) - 작성자: 젤리하뚜 (사진 229)
리뷰 6: 맛있어요!! (2024년 4월 11일 목요일) - 작성자: dndnmmm (사진 122)
리뷰 7: 지나다 들렀는데 분위기 시끌벅적하니 좋고 쭈꾸미 맵지않고 맛있습니다~ (2023년 6월 23일 금요일) - 작성자: 만렙너구리 (사진 4,723)
리뷰 8: 회사앞에 이런 곳이???
3년을 지나다녔는데 이런 곳이🙊🙊

쭈삼 오오오오오
묵은지광어회 오오오오오~~
오이 짜파게티 오오오오오
가리비치즈구이 오오오~
도시락 오호라~~
고동?소라?

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%B9%84%EC%B1%84/place/1729260818?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 비채
업종 카테고리: 카페,디저트
별점: 방문자 리뷰 11
방문자 리뷰: 없음
블로그 리뷰: 없음
오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0089EBD3+24307]
	(No symbol) [0x00828D74]
	(No symbol) [0x0070C323]
	(No symbol) [0x0074DC86]
	(No symbol) [0x0074DECB]
	(No symbol) [0x0078B9F2]
	(No symbol) [0x0076FED4]
	(No symbol) [0x00789579]
	(No symbol) [0x0076FC26]
	(No symbol) [0x0074219C]
	(No symbol) [0x0074311D]
	GetHandleVerifier [0x00B48D93+2818227]
	GetHandleVerifier [0x00BA542E+3196750]
	GetHandleVerifier [0x00B9D9D2+3165426]
	GetHandleVerifier [0x0093DA70+675216]
	(No symbol) [0x00831B3D]
	(No symbol) [0x0082EA18]
	(No symbol) [0x0082EBB5]
	(No symbol) [0x00821640]
	BaseThreadInitThunk [0x7701FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x771E809E+286]
	RtlGetAppContainerNamedObjectPath [0x771E806E+238]

리뷰 데이터가 '비채_reviews.csv' 파일로 저장되었습니다.


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%98%81%EC%9D%B4%EB%84%A4/place/1972614021?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 영이네
업종 카테고리: 종합분식
별점: 방문자 리뷰 360
방문자 리뷰: 119
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 316


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 사실 다른 곳 가려다가 웨이팅이 너무 심해서 다른곳 찾아보려고 걸어가던 중에 우연히 간판이 눈에띄고 인테리어도 좋아서 한번 먹어볼까 방문했는데

원래 가려던곳보다 더 맛있어서 깜짝놀랬어요

기대이상으로 맛있어서 재방문 의사 1000%입니다. (2024년 10월 31일 목요일) - 작성자: ify**** (사진 11)
리뷰 2: 영이네 회사 점심시간에 떡볶이 땡길때마다 와요~ 음식도 빨리 빨리 나오고 무엇보다 맛있고 양많아용~ 사장님도 친절하시구.. 마무리로 볶음밥 꼭 볶아드세요!! (2024년 10월 18일 금요일) - 작성자: 배부른 벨라 (사진 58)
리뷰 3: 푸짐하고 맛있어요😋❤️❤️ 언제와도 존맛! 감사합니다 응원할게요👍👍👍 (2024년 10월 7일 월요일) - 작성자: 대왕연어 (사진 78)
리뷰 4: 다시 방문한 영이네!!
음식 하나하나가 다 맛있습니다~~
맛있었던 김밥이 없어진건 좀 아쉽네요~~
다시 또 올께요~ (2024년 11월 12일 화요일) - 작성자: cielyuri (사진 31)
리뷰 5: 냉삼 맛집이예요 짱짱짱 맛있어요 (2024년 10월 12일 토요일) - 작성자: 애그파이 (사진 513)
리뷰 6: 맛있어요..

또 올 거에요... (2024년 10월 5일 토요일) - 작성자: hongjae12345 (사진 35)
리뷰 7: 예전에 작게 가게 운영 할때부터 자주 이용해서 먹었는데 매장 크게 이전해서 더 좋아요!!! 떡볶이 러버로써 넘 맛있어요!! (2024년 10월 12일 토요일) - 작성자: yj**** (사진 19)
리뷰 8: 너무 맛있어요! 특히 순대가 진짜 맛있어요~!! (2024년 9월 26일 목요일) - 작성자: dlt**** (사진 15)
리뷰 9: 떡볶이 썩 좋아하지 않는데도 이 근방 떡볶이 중에 제일

리뷰 74: 완전 맛있어요-!-!-! 추천합니다🩷🩷🩷🩷🩷🩷🩷🩷 (2024년 5월 23일 목요일) - 작성자: nowd (사진 67)
리뷰 75: 너무 맛있어용 (2024년 6월 19일 수요일) - 작성자: kda**** (사진 30)
리뷰 76: 맛있어요! 매장이 시원해서 좋아요 (2024년 6월 19일 수요일) - 작성자: 야호86 (사진 9)
리뷰 77: 회사 점심시간에 밥먹으러왔는데 맛있게 잘 먹구 갑니당! (2024년 6월 27일 목요일) - 작성자: rbg**** (사진 11)
리뷰 78: 맛있러요!! (2024년 9월 26일 목요일) - 작성자: dda**** (사진 9)
리뷰 79: 맛나요 좋아요!! ㅎㅎ (2024년 7월 27일 토요일) - 작성자: 아트스쿨그리고 (사진 29)
리뷰 80: 오랜만에 옛날 즉석 떡볶이 먹으니까 감회가 새롭네요! 학교 다닐 때 즉떡 많이 먹었는뎁.. 추억 돋는다아 ㅠㅠ 영이네 즉떡도 김말이 튀김 추가해서 맛있게 먹고 왔어요 :) 꽤 늦은 시간이었는데 배달 손님, 그냥 방문 손님이 꾸준히 오시더라고요 🫢 (2024년 3월 25일 월요일) - 작성자: 민지16 (사진 268)
로드된 리뷰 개수: 90, 수집된 리뷰 개수: 80
리뷰 81: 떡볶이 맛있어요 :) (2024년 6월 14일 금요일) - 작성자: 정직한 금융 (사진 3)
리뷰 82: 애들이 분식집 오고싶다고 졸라서 왔는데 맛있어요!
사장님도 친절하시고
다음에 또 올께요!! (2024년 5월 25일 토요일) - 작성자: hek**** (사진 2)
리뷰 83: 자주 오는 뱅뱅사거리 떡볶이 맛집!
가성비도 짱짱 맛도 짱짱💘👏👏👏
존맛이에요ㅠㅠ 죽떡도 그냥떡볶이도 왕맛
둘다 꼭 드셔보시기 ㅎㅎㅎㅎ (2024년 2월 15일 목요일) - 작성자: D3056 (사진 504)
리뷰 84: 너무너무너무너무맛있어요…💕💕 (2024년 5월 14일 화요일) - 작성자: 신예린767 (사진 4)
리뷰 85: 여기 즉석떡볶에 진짜 맛있어요 ! 도곡동맛집 인정! 여러분 꼭 드세요 아니 

리뷰 148: 맛있고 친절해요 (2024년 2월 24일 토요일) - 작성자: ekd**** (사진 34)
리뷰 149: 너무 맛있어요 우연히들린 식당인데 다음에 또 올것같아오~~♡ (2024년 2월 7일 수요일) - 작성자: csd8844 (사진 26)
리뷰 150: 죤맛탱 꼭 와보세용 ㅎㅎㅎ (2024년 2월 23일 금요일) - 작성자: 안녕4779 (사진 24)
로드된 리뷰 개수: 160, 수집된 리뷰 개수: 150
리뷰 151: 도곡동맛집을 찾아 헤메다 만난 영이네
감동적인 떡볶이를 만났다
달달하면서도 매콤한
쫄깃한 밀떡은 감동이었다
볶음밥으로 마무리까지 완벽한
마음의 평화를 가져다준
즉석떡볶이 영이네 (2023년 11월 1일 수요일) - 작성자: Nobrandchicken (사진 259)
리뷰 152: 양이 넉넉하니 많고 딱 즉석떡볶이 같아요~~
즉떡 먹고 싶을 때 방문하면 좋을 듯 해요^^ (2024년 1월 27일 토요일) - 작성자: dongjunam (사진 10)
리뷰 153: 가끔 김밥 떡볶이 오뎅 맛있어서 포장해서 먹습니다.
참치김밥 종종 포장하는데 시간이 조금걸려서
급하시면 미리 전화하고 가야할듯합니다.
다음에는 즉석떡볶이를 먹어보려합니다. (2024년 1월 26일 금요일) - 작성자: hos**** (사진 439)
리뷰 154: 혼밥하기에도 쾌저하고 특히 김치볶음밥 맛있어요!
오뎅국물은 덤으로 주시네요 (2023년 12월 19일 화요일) - 작성자: 코헨80 (사진 7)
리뷰 155: 항상 맛있어서 생각나는 맛집이에요~!!
떡볶이가 아이도 먹을 수 있는 적당한 매운정도~ 김밥도 맛있고 튀김은 바삭해서 좋아요^^
특히 계란튀김 추천합니다!! (2023년 12월 30일 토요일) - 작성자: 라떼9511 (사진 8)
리뷰 156: 너무 맛있어요~!!! (2024년 1월 30일 화요일) - 작성자: 재롱이27 (사진 68)
리뷰 157: 맛이 너무나 맜있습니다. 부모님도 가족들도 너무나도 맜있다고 하네요. 다시 오고 싶습니다!!!!!! (2024년

로드된 리뷰 개수: 240, 수집된 리뷰 개수: 230
리뷰 231: 영이네에 즉석떡볶이가 생기다니!! 너무 맛있어요!!! 적당히 매콤하고 배불러도 계속 들어가는 맛 ㅠㅠ 배불러도 볶음밥 꼭 먹어야 합니다...👍 가게도 넓어지구 메뉴도 전보다 더 많아져서 메뉴고르는데 고민했어요 헤헤 매장도 컨셉이 특이해서 보는재미가 쏠쏠하네욬ㅋㅋㅋ 담에는 로제 떡볶이 먹으러 갈께요~~ (2023년 6월 13일 화요일) - 작성자: 푸스터 (사진 140)
리뷰 232: 여러명이 점심으로 회식대신 왔어요.
완전 양푸짐하고 매콤하니 맛있어요.
날씨 쌀쌀해지는데 따뜻한 떡볶이 강츄합니다. (2023년 11월 8일 수요일) - 작성자: mimeur (사진 78)
리뷰 233: 너무 맛닜어요 가선비도 훌륭하고 분위기도 좋아요 (2023년 11월 10일 금요일) - 작성자: voi**** (사진 6)
리뷰 234: 양도 많고 직원분들도 친절하시고 가격도 합리적이어서 또 올것같애요~ 맛도 👍 영이네 떡볶이 강추합니당~ (2023년 10월 15일 일요일) - 작성자: Gloria30 (사진 2)
리뷰 235: 인테리어가 정말 멋져요. 떡볶이도 맛있어요. (2024년 1월 17일 수요일) - 작성자: Jinny Kim (사진 178)
리뷰 236: 맛있는 김밥과 라면!! 영이네 좋으요 (2023년 11월 1일 수요일) - 작성자: 똥깡츄 (사진 53)
리뷰 237:  (2023년 12월 29일 금요일) - 작성자: 콩치빠 (사진 13)
리뷰 238: 맛있어요!! (2023년 12월 30일 토요일) - 작성자: 수민520 (사진 2)
리뷰 239: 자주 찾는 영이네 역시 최고랍니다.
친절하시고, 모든 메뉴가 최고!!! (2023년 10월 21일 토요일) - 작성자: 200016108 (사진 10)
리뷰 240: 즉석떡볶이 양념이 잘베어있구
순대도 넘맛있어용 (2023년 11월 26일 일요일) - 작성자: Wed24 (사진 4)
로드된 리뷰 개수: 250, 수집된 리뷰 개수: 240
리뷰 241:

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%9B%90%ED%95%A0%EB%A7%A4%EC%9D%B4%EB%AA%A8%EB%84%A4%EB%8B%AD%ED%95%9C%EB%A7%88%EB%A6%AC/place/38325046?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 원할매이모네닭한마리
업종 카테고리: 닭요리
별점: 4.32
방문자 리뷰: 431
블로그 리뷰: 82
리뷰 탭 클릭 완료
총 리뷰 수: 197


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 오늘은 않먹어 보던 닭칼국수 먹었는데
그것도 맛있음^^;;
이 집은 점심먹으로 자주감^^
가성비 좋음 (2024년 10월 31일 목요일) - 작성자: 베야로드 (사진 333)
리뷰 2: 칼국수 국물이 너무 맛있움ㅎㅎ가격도 만원안넘거 굿 (2024년 11월 4일 월요일) - 작성자: wns59164 (사진 51)
리뷰 3: 누가 사진을 이렇게 찍었는지 참..
진짜 매운 닭볶음탕
엽떡급 매움인데 맛인게 매워요
다른 메뉴들도 다 맛있데요
점심에 직장인 많은 곳 (2024년 9월 9일 월요일) - 작성자: 팩폭2 (사진 896)
리뷰 4: 여름이라 기운 딸려 닭 먹으러 갔는데
여기 가격도 좋고 닭곰탕도 맛있음
닭고기 살도 많이 들어 있구 (2024년 8월 8일 목요일) - 작성자: 베야로드 (사진 333)
리뷰 5: 2번째 방문
이번엔 초계국수 도전!
맛과 가격 모두 잡았음 ㅇㅈ
면발 괜찮고, 닭 고명 맛있고 국물맛 다른 거 추가할 것 없이 적절함
이런집이 냉면도 잘하네 ㅋㅋ여름에 자주 봐요 (2024년 6월 14일 금요일) - 작성자: 팩폭2 (사진 896)
리뷰 6: 1. 닭냄새 비린내 최악
2. 불친절 서비스상태최악 (2024년 10월 18일 금요일) - 작성자: psh8330 (사진 10)
리뷰 7: 솔직히.. 닭한마리 맛도 그냥 그랬는데 그건 그러려니 합니다. 근데 뭐 좀 더 달라고 말하려는데 워낙 바빠보여서 쟁반 들고 갔더니 들고다니지말라고 넘어지면 냄새난다고 짜증내시고 계속 말투 툴툴대시고 이래라 저래라 명령하시고 참 . 내 돈내고 밥 먹으러왔다가 짜증이 나네요. 쟁반을 던저버리러다가 진짜 주인아주머니는 친절해서 참았습니다. 그 따님인지 종업원인지 헤비급 여자분. 그따위 태도로 장사하실거면 제발 그냥 가게에 나오질 마시는게 도움되지 싶습니다

리뷰 94: 닭곰탕 최애 (2022년 11월 30일 수요일) - 작성자: 리아케이 (사진 14)
리뷰 95: 굿굿 (2022년 11월 16일 수요일) - 작성자: 랄라룰라랄라라 (사진 60)
리뷰 96: 음식 양도 많고 가성비 좋아요 (2022년 9월 22일 목요일) - 작성자: 진진9682 (Unknown)
리뷰 97: 맛있어요 (2022년 12월 14일 수요일) - 작성자: gur**** (사진 2)
리뷰 98: 직원분이 친절하고 닭칼국수도 초계국수도 맛있어서 좋습니다 ㅎㅎ (2022년 8월 16일 화요일) - 작성자: Jung100099 (사진 46)
리뷰 99: 맛이 진하고 좋음. 죽사리가 찹쌀이라 쫄깃하고 찰져서 맛있음. 죽이 별미. (2022년 6월 7일 화요일) - 작성자: 뜨끈뜨끈호떡 (사진 387)
리뷰 100: 가성비가 좋습니다. (2022년 9월 6일 화요일) - 작성자: 은박4 (사진 157)
로드된 리뷰 개수: 110, 수집된 리뷰 개수: 100
리뷰 101: 종로 본점에 이어 강남점이고 맛있고 가성비 좋아요 (2022년 8월 24일 수요일) - 작성자: 진진9682 (Unknown)
리뷰 102: 요즘 드문 7000원의 가격으로 맛있게 먹을수 있어요 (2022년 8월 24일 수요일) - 작성자: 진진9682 (Unknown)
리뷰 103: 굿굿 (2022년 10월 13일 목요일) - 작성자: 랄라룰라랄라라 (사진 60)
리뷰 104: 긋굿 (2022년 10월 11일 화요일) - 작성자: 랄라룰라랄라라 (사진 60)
리뷰 105: 맛잇어요 (2022년 11월 10일 목요일) - 작성자: 리아케이 (사진 14)
리뷰 106: 굿굿 (2022년 9월 15일 목요일) - 작성자: 랄라룰라랄라라 (사진 60)
리뷰 107: 굿굿 (2022년 9월 1일 목요일) - 작성자: 랄라룰라랄라라 (사진 60)
리뷰 108: 굿굿 (2022년 9월 23일 금요일) - 작성자: 랄라룰라랄라라 (사진 60)
리뷰 109: 맛잇어요 (2022년 10월 24일 

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/S%EC%B9%B4%ED%8E%98%20%EB%8F%84%EA%B3%A1%EB%8F%99/place/35781714?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: S카페
업종 카테고리: 바(BAR)
별점: 방문자 리뷰 34
방문자 리뷰: 없음
블로그 리뷰: 없음
오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0089EBD3+24307]
	(No symbol) [0x00828D74]
	(No symbol) [0x0070C323]
	(No symbol) [0x0074DC86]
	(No symbol) [0x0074DECB]
	(No symbol) [0x0078B9F2]
	(No symbol) [0x0076FED4]
	(No symbol) [0x00789579]
	(No symbol) [0x0076FC26]
	(No symbol) [0x0074219C]
	(No symbol) [0x0074311D]
	GetHandleVerifier [0x00B48D93+2818227]
	GetHandleVerifier [0x00BA542E+3196750]
	GetHandleVerifier [0x00B9D9D2+3165426]
	GetHandleVerifier [0x0093DA70+675216]
	(No symbol) [0x00831B3D]
	(No symbol) [0x0082EA18]
	(No symbol) [0x0082EBB5]
	(No symbol) [0x00821640]
	BaseThreadInitThunk [0x7701FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x771E809E+286]
	RtlGetAppContainerNamedObjectPath [0x771E806E+238]

리뷰 데이터가 'S카페_reviews.csv' 파일로 저장되었습니다.


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%97%90%EC%9D%B4%EB%A6%BC%EC%BB%A4%ED%94%BC/place/13349601?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 에이림커피
업종 카테고리: 카페
별점: 4.65
방문자 리뷰: 576
블로그 리뷰: 28
리뷰 탭 클릭 완료
총 리뷰 수: 144


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 뱅뱅사거리 골목쪽에 위티해있어요.
가격도 높지 않고 메뉴가 금방 나오는데도 맛나요~
바에서 3분이서 뚝딱!
조용한 분위기라서 대화하기에도 좋습니당 (2024년 7월 31일 수요일) - 작성자: 다슬파더 (사진 1,865)
리뷰 2: 커피맛으로는 이곳 넘버원입니다, (2024년 9월 19일 목요일) - 작성자: 파워배추 (사진 1,515)
리뷰 3: 좋은 찻집 커피집 (2024년 10월 14일 월요일) - 작성자: YJ081 (사진 399)
리뷰 4: 친절하고 커피맛은 이 쪽 지역에서 원탑입니다 ~~~ (2024년 7월 24일 수요일) - 작성자: 배산임수85 (사진 485)
리뷰 5: 커피맛이 좋아요 (2024년 8월 22일 목요일) - 작성자: 코끼리다리78 (사진 12)
리뷰 6: 굿초이스 (2024년 7월 10일 수요일) - 작성자: 청아48 (사진 31)
리뷰 7: 드립 카페인데, 저도 카페 시장에 뛰어 들 수 있을정도의 자신감을 북돋아주는 그런 카페였어요 최고에요 (2023년 12월 20일 수요일) - 작성자: 하울7 (사진 902)
리뷰 8: Take Out 20%할인이라니~~
Good!! (2024년 6월 6일 목요일) - 작성자: 김귀임 교원맘 (사진 35)
리뷰 9: 여기 커피맛은 인근 까페에서 맛보지 못한 스폐셜한 맛입니다. 원두도 직접 볶고 로스팅도 하시는 거 같아서 더 맛있어요- (2024년 2월 13일 화요일) - 작성자: 파워배추 (사진 1,515)
리뷰 10: 가까워서 좋아요 (2024년 6월 7일 금요일) - 작성자: 김귀임 교원맘 (사진 35)
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 커피 맛집인거 같아요. 매장이 조금 어수선했어요 (2024년 2월 2일 금요일) - 작성자: lTree (사진 113)


로드된 리뷰 개수: 120, 수집된 리뷰 개수: 110
리뷰 111: 서비스로 콜드브루 미니 주셨는데 커피본제품도 맛있어요 향긋 (2021년 7월 2일 금요일) - 작성자: 뽀링82 (사진 21)
리뷰 112: 완전 진짜 (2021년 2월 25일 목요일) - 작성자: Dreamcurl (사진 18)
리뷰 113: 커피 좋아요~~~! (2021년 2월 15일 월요일) - 작성자: kinbbh (사진 29)
리뷰 114: 강남구 도곡동,에이림커피.
감사해요. (2020년 11월 12일 목요일) - 작성자: rud**** (사진 137)
리뷰 115: 좋아요 (2021년 1월 25일 월요일) - 작성자: 최재영9774 (사진 3)
리뷰 116: 아인슈페너 너무 맛있었어요 맛집입다!! (2021년 5월 29일 토요일) - 작성자: ksj7566s (사진 3)
리뷰 117: 맛있어요 ^^ (2020년 10월 24일 토요일) - 작성자: vjdzlzjffl (사진 3,387)
리뷰 118: 매우 만족스러운 장소입니다. (2020년 9월 18일 금요일) - 작성자: eoe**** (Unknown)
리뷰 119: 좋은 장소입니다. (2020년 9월 18일 금요일) - 작성자: cha**** (사진 17)
리뷰 120: 매우 만족스러운 공간입니다. (2020년 9월 18일 금요일) - 작성자: eoe**** (Unknown)
로드된 리뷰 개수: 130, 수집된 리뷰 개수: 120
리뷰 121: 좋아요 (2021년 3월 2일 화요일) - 작성자: pet**** (사진 2)
리뷰 122: 고래똥 항상 맛있어요. (2020년 10월 6일 화요일) - 작성자: gpd**** (사진 17)
리뷰 123: 매우 만족 합니다. (2020년 9월 18일 금요일) - 작성자: He3 (Unknown)
리뷰 124: 좋아요 (2020년 11월 12일 목요일) - 작성자: spr**** (Unknown)
리뷰 125: 커피는 역시 에이림이 맛있어요
라떼가 특히 고소해요 (2020년 8월 7일 금

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%9A%9C%EB%A0%88%EC%A5%AC%EB%A5%B4%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/18548937?c=15.45,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 뚜레쥬르 뱅뱅사거리점
업종 카테고리: 베이커리
별점: 4.32
방문자 리뷰: 1,860
블로그 리뷰: 30
리뷰 탭 클릭 완료
총 리뷰 수: 747


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 뚜레쥬르 케잌 진짜 오랫만!

보통은 생일때도 아이스크림케이크나 파리바게뜨 케이크를 주로 사먹었더랬는데 케티 생일쿠폰 써보까 하는맘에 간만에 사본 뚜쥬 케잌~ㅎ 근데 웬걸 크림이 빵보다 두껍게 발려있고 안에 시트빵도 아주~ 촉촉하니 딸기쨈?스런 아이들이 발려져있어 상큼달달 겉부속촉한 간만에 혜자스런 생크림 케이크를 발견 해 버렸네용.ㅎㅎ 아이스크림 케잌만 잘 먹는 아이들도 간만에 일인 일조각씩!! 해치웠다는~~~맛있어성 내돈내산 리뷰씁니당♡ (2024년 9월 18일 수요일) - 작성자: 러블리뷰러 (사진 241)
리뷰 2: 인사이드아웃 빨리 봐야하는데 ㅠ
컵홀더가 귀여워서 기분좋게마셨어요
콜드부르 맛있네요오

야외테이블도 있고 내부에도 테이블 넉넉해서
샐러드나 샌드위치로 식사하기도 좋아보여요
그리고 제가 결제가 좀 시간이 걸렸는데 사장님이 친절하게 기다려주셔서 감사했어요! (2024년 6월 27일 목요일) - 작성자: 먹먹먹먹고 (사진 679)
리뷰 3: 빵 종류가 다양해요 (2024년 9월 30일 월요일) - 작성자: aquaoflife (사진 81)
리뷰 4: 에이닷 특판 50% ❤️🧡💛 (2024년 9월 19일 목요일) - 작성자: 바부야15 (사진 1,000)
리뷰 5: 브라우니 맛이 조아요 (2024년 10월 15일 화요일) - 작성자: 마중물747 (사진 1)
리뷰 6: 매장도 깔끔하고 좌석도 많아서 배고플때 이용하기 좋았어요. (2024년 7월 28일 일요일) - 작성자: marui27 (사진 690)
리뷰 7: 직원분이 친절해요 (2024년 9월 21일 토요일) - 작성자: bluepoison1953 (사진 198)
리뷰 8: 그런데 T map 모시 반인한테 살면 정말 해자인 것 (2024년 7월 17일 수요일) - 작성자:

로드된 리뷰 개수: 100, 수집된 리뷰 개수: 90
리뷰 91: 결혼기념일에 아이가 뚜레쥬르 케잌 디자인을 좋아해서 멀지만 다녀왔어요. ^^
예쁜 디자인 항상 부탁합니다. ㅎㅎ (2023년 11월 21일 화요일) - 작성자: 또또리13 (사진 9)
리뷰 92: 직원분이 친절하세요 (2023년 12월 4일 월요일) - 작성자: kjk**** (사진 17)
리뷰 93: 굿뜨. . (2024년 5월 13일 월요일) - 작성자: 먹보호랑 (사진 186)
리뷰 94: 유효기간 짧은 빵만 진열해놓고, 신선한 빵은 숨겨놓는다. 매번 느끼지만 거짓말로 응대하라고 교육받는듯하다 (2023년 12월 13일 수요일) - 작성자: plusma (사진 15)
리뷰 95: 굿 (2024년 6월 12일 수요일) - 작성자: 김귀임 교원맘 (사진 35)
리뷰 96: 매장이 넓고 의자가 많아서 차 마시고 얘기하기 좋아요~ 빵 종류가 많습니다 (2023년 10월 11일 수요일) - 작성자: 명랑집사 (사진 3,460)
리뷰 97: 굿 (2024년 2월 7일 수요일) - 작성자: 코코네 언니 (사진 205)
리뷰 98: 샌드위치 종류가많고 매장넓고 깨끗해서 좋아요 (2024년 1월 14일 일요일) - 작성자: 쩡쩡이73 (사진 86)
리뷰 99: 참지 못하고 빵을 사버린 나.
맛있긴 하다. (2023년 11월 9일 목요일) - 작성자: 맛도리보안관 (사진 697)
리뷰 100: 굿 (2024년 6월 7일 금요일) - 작성자: 김귀임 교원맘 (사진 35)
로드된 리뷰 개수: 110, 수집된 리뷰 개수: 100
리뷰 101: 새로 나온 라라스윗생우유아이슈크림
그냥슈라 눅눅하고 크림은 그럭저럭한 맛
가격에 비해 맛있는편은 아니라고 생각들어
재구매의사는 없습니다 ㅠ(또르륵)
녹차롤케익은 처음 사먹어보았습니다
크림이 녹차크림이 아닌 생크림에 색소나 가루를 넣은것 같아 실망했지만 그래도 팥이 있어서 엄청 느끼하지않게 맛있게 먹었어요!! (2023년 7월 19일 수요일) - 작성자: 엔젤스져니 (사진 8

로드된 리뷰 개수: 210, 수집된 리뷰 개수: 200
리뷰 201: 사장님 친절하시고 매장 깨끗해요

샐러드 9800원 와우 강남 물가 놀라고 가요 (2023년 1월 17일 화요일) - 작성자: 냥냥파월 (사진 518)
리뷰 202: 사장님께서 상당히 친절하셨어요 (2023년 3월 6일 월요일) - 작성자: 오공이99 (사진 78)
리뷰 203: 할인,적립잘 챙겨줘서 좋아요 (2023년 1월 7일 토요일) - 작성자: naye517 (사진 360)
리뷰 204: 굿 (2023년 6월 25일 일요일) - 작성자: 박주하61 (사진 8)
리뷰 205:  (2023년 3월 30일 목요일) - 작성자: alstn0311 (사진 483)
리뷰 206: 아침마다 빵사러 빵순이 출근 (2023년 2월 7일 화요일) - 작성자: Hanamanaham (사진 299)
리뷰 207: 직원분이 친절해요^^ (2023년 7월 21일 금요일) - 작성자: wartos (사진 149)
리뷰 208: 샌드위치 맛있어요. (2023년 3월 24일 금요일) - 작성자: 피요나슈렉 (사진 975)
리뷰 209: 대로변에있어서 매장 찾기 좋아요. (2023년 3월 23일 목요일) - 작성자: sujie23 (사진 1)
리뷰 210: 굳 (2023년 4월 21일 금요일) - 작성자: 달똥구리 (사진 980)
로드된 리뷰 개수: 220, 수집된 리뷰 개수: 210
리뷰 211: 친절해요 치즈타르트 꿀맛 (2023년 2월 15일 수요일) - 작성자: LiLangi (사진 246)
리뷰 212: 뚜쥬중에 땡겨요 행사제외매장 (2023년 5월 11일 목요일) - 작성자: 담이욤 (사진 120)
리뷰 213: 매장이 넓어요 (2023년 4월 17일 월요일) - 작성자: ins**** (사진 2)
리뷰 214: 기프티콘이 있어서 갔는데 가격이 올랐다고..
같은 빵도 추가비용 내야한대서
그냥 다른 빵으로 가져왔어요..
커피숍들은 차액할인되던데 아쉽네요..
그리고 꽈베기가 좀 기름지고 퍽퍽하네요ㅠ (2023

리뷰 318: . (2022년 11월 15일 화요일) - 작성자: 아마존조로존존 (사진 77)
리뷰 319: 샐러드맛있똬 (2022년 11월 11일 금요일) - 작성자: ace7445 (사진 6)
리뷰 320: 케익 종류가 다양해요 (2022년 10월 8일 토요일) - 작성자: marchrose (사진 104)
로드된 리뷰 개수: 330, 수집된 리뷰 개수: 320
리뷰 321: 맛있어요 (2023년 1월 5일 목요일) - 작성자: 아아아350 (사진 2)
리뷰 322: 맛있어요~ (2023년 2월 1일 수요일) - 작성자: 아기볶음밥0115 (사진 400)
리뷰 323: 좋아요 (2022년 12월 30일 금요일) - 작성자: luft08 (사진 8)
리뷰 324: 좋아요 (2022년 12월 30일 금요일) - 작성자: 뚠뚜니98 (사진 28)
리뷰 325: 할인받아 저렴하게 구매했어요 (2022년 8월 31일 수요일) - 작성자: 엘리네 (사진 21)
리뷰 326: 친철하진 않네요 (2022년 10월 23일 일요일) - 작성자: 봄이조타97 (사진 28)
리뷰 327: 자주들립니ㅏㅇ (2022년 10월 31일 월요일) - 작성자: 아치미유 (사진 163)
리뷰 328: 굿 (2022년 11월 16일 수요일) - 작성자: 라스칼라 (사진 90)
리뷰 329: 좋아요 (2022년 12월 18일 일요일) - 작성자: 이종미79 (Unknown)
리뷰 330: 베지랩이 나왔어요!!
나쵸딥도 매콤해서 좋아요~
맛나네요ㅠㅠ
낼은 텐더도 먹어볼거에요! (2022년 6월 15일 수요일) - 작성자: 과자처돌이 (사진 330)
로드된 리뷰 개수: 340, 수집된 리뷰 개수: 330
리뷰 331: 굿 (2022년 12월 16일 금요일) - 작성자: 티티새7788 (사진 21)
리뷰 332: 굿 (2023년 3월 12일 일요일) - 작성자: 불타는쓰레빠30 (사진 121)
리뷰 333: 이건 아닌것같아서 사진찍었어요... 아무리 크리스마스시즌이라 빵시트를 예전에 만든거라도 너무

리뷰 444: 빵 너무 맛있고, 매장도 깨끗 (2022년 2월 12일 토요일) - 작성자: djd**** (사진 1)
리뷰 445: 굿 (2022년 5월 29일 일요일) - 작성자: 나도아라99 (Unknown)
리뷰 446: 굿 (2022년 5월 16일 월요일) - 작성자: lee**** (Unknown)
리뷰 447: 소금빵 에프에 돌려먹으면 맛나요 (2022년 3월 9일 수요일) - 작성자: 최다혜7211 (Unknown)
리뷰 448: 맛있어요 (2022년 4월 28일 목요일) - 작성자: iys**** (사진 92)
리뷰 449: 굿 (2022년 5월 28일 토요일) - 작성자: 나도아라99 (Unknown)
로드된 리뷰 개수: 460, 수집된 리뷰 개수: 449
리뷰 450: 빵이맛있어요.종종갑니다~ (2022년 1월 26일 수요일) - 작성자: 그는귀인이다 (사진 217)
리뷰 451: 좋아요 (2022년 4월 18일 월요일) - 작성자: Vvv9557 (사진 124)
리뷰 452: 뚜레쥬르는 참 샐러드가 식사대용으로 좋아요 푸짐하고 배불러요...그리고 베이직한 맛이에요! (2021년 11월 24일 수요일) - 작성자: cartunist (사진 600)
리뷰 453: 굿 (2022년 5월 10일 화요일) - 작성자: 최다혜7211 (Unknown)
리뷰 454: 매장 깨끗하고, 친절해요. 빵은 비싸요 (2022년 1월 14일 금요일) - 작성자: FCMOM (사진 537)
리뷰 455: 굿 (2022년 4월 10일 일요일) - 작성자: 왕밤빵5475 (사진 20)
리뷰 456:  (2022년 3월 2일 수요일) - 작성자: Vvv9557 (사진 124)
리뷰 457: 종종 방문드리는곳^^ (2022년 1월 26일 수요일) - 작성자: 이은희127 (사진 29)
리뷰 458: 굿 (2022년 3월 30일 수요일) - 작성자: Superj (사진 31)
리뷰 459: 굳 (2022년 4월 27일 수요일) - 작성자: suj**** (Unknown)
로

리뷰 576: 친절하지 않음 (2021년 6월 6일 일요일) - 작성자: vickie99 (사진 30)
리뷰 577: 빵 맛 괜찮은 제과점 (2021년 5월 29일 토요일) - 작성자: 피노누아76 (사진 20)
리뷰 578: 종류도 많고 넓어서 좋아요 (2021년 5월 14일 금요일) - 작성자: Hangangs (사진 27)
리뷰 579: 맛있어요 (2021년 7월 3일 토요일) - 작성자: 메텔832 (사진 60)
로드된 리뷰 개수: 590, 수집된 리뷰 개수: 579
리뷰 580: 좋아요 (2021년 8월 4일 수요일) - 작성자: JHCD (사진 3)
리뷰 581: 뚜레쥬르 뱅뱅사거리 (2021년 5월 25일 화요일) - 작성자: 예이니22 (Unknown)
리뷰 582: 좋아요 분위기도 맛도 ~~ (2021년 3월 29일 월요일) - 작성자: moo**** (사진 111)
리뷰 583: 좋아요 (2021년 6월 19일 토요일) - 작성자: 딸기우유19 (사진 34)
리뷰 584: 오늘 t멤버쉬 매직바코드 행사해서 갑자기 방문!
여기 캐릭터케익 특히 아이들이 좋아해요~^^ (2021년 3월 10일 수요일) - 작성자: ej62 (사진 50)
리뷰 585: 좋아요 (2021년 6월 15일 화요일) - 작성자: tjfrltititi (사진 109)
리뷰 586: 맛있어용 (2021년 7월 16일 금요일) - 작성자: mice516 (사진 14)
리뷰 587: 좋은데 빵종류가 다양하지, (2021년 2월 23일 화요일) - 작성자: 오리진826 (사진 51)
리뷰 588: 괜찮음 의외로 커피 맛있음 (2021년 4월 3일 토요일) - 작성자: woohe5800 (사진 8)
리뷰 589: 좋아요 음료는 그다지~~ (2021년 4월 27일 화요일) - 작성자: phr007 (Unknown)
로드된 리뷰 개수: 600, 수집된 리뷰 개수: 589
리뷰 590: 조아요 (2021년 9월 19일 일요일) - 작성자: 브리스92 (사진 13)
리뷰 591: 커피 맛있

리뷰 707: 믿고먹는 뚜레 (2020년 5월 15일 금요일) - 작성자: 분홍당근 (사진 51)
리뷰 708: 조아요 (2020년 8월 7일 금요일) - 작성자: 쏩16 (사진 5)
리뷰 709: 그냥 가까운곳에 있어서 가는곳 맛도 다른 매장비해 평범, 서비스도 평범.. (2020년 4월 17일 금요일) - 작성자: 스웨덴에서카푸치노를 (사진 21)
로드된 리뷰 개수: 720, 수집된 리뷰 개수: 709
리뷰 710:  (2020년 7월 7일 화요일) - 작성자: 오유정1826 (사진 7)
리뷰 711: 편리하고 깨끗한 곳이예요 (2020년 12월 7일 월요일) - 작성자: 이웃사촌2084 (사진 8)
리뷰 712: 치즈방앗간 맛있네요.
달달겉 속엔 크림치즈가 쏙 (2020년 4월 6일 월요일) - 작성자: 복분자43 (사진 12)
리뷰 713: 좋아요 ㅎㅎ (2020년 7월 23일 목요일) - 작성자: 달달하니 (사진 21)
리뷰 714: 케익 샀는데 엄청 오래된 걸 파는지 위에 과일 시럽이 되게 끈적하고 크림이 메말라있음 (2020년 4월 8일 수요일) - 작성자: 육학년이웃겨 (사진 29)
리뷰 715: . (2020년 9월 26일 토요일) - 작성자: poo**** (사진 10)
리뷰 716: 맛있는데 빵종류가 많지 않은듯해요 (2021년 2월 16일 화요일) - 작성자: 짱아736 (Unknown)
리뷰 717: 맛나요 (2020년 9월 4일 금요일) - 작성자: Abby An (사진 162)
리뷰 718: 맛있어유 (2020년 9월 2일 수요일) - 작성자: 밍밍9544 (사진 33)
리뷰 719: 맛있고 다양하다 (2020년 3월 19일 목요일) - 작성자: 맛지비지비빕 (사진 32)
로드된 리뷰 개수: 730, 수집된 리뷰 개수: 719
리뷰 720: 좋아요^^ (2020년 6월 16일 화요일) - 작성자: scarlet star (사진 1)
리뷰 721: 존맛쿠 (2020년 6월 16일 화요일) - 작성자: 꾸릉93 (사진 10)
리뷰 722

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%BB%A4%ED%94%BC%EB%B0%95%EC%8A%A4/place/1982336630?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 커피박스
업종 카테고리: 카페
별점: 방문자 리뷰 114
방문자 리뷰: 1
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 29


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 자주 들러요, 커피 끝맛까지 좋아요 (2024년 6월 18일 화요일) - 작성자: pastamonster (사진 11)
리뷰 2: 좋아요 (2024년 6월 27일 목요일) - 작성자: 실천하는 양프로 (사진 776)
리뷰 3: 친절하시고 커피 음료 가격대비 나쁘지 않은 맛이네용👏🏻ㅎㅎ (2024년 2월 27일 화요일) - 작성자: 현구4414 (사진 59)
리뷰 4: 사장님 매번 너무 친절하시네요 (2024년 2월 15일 목요일) - 작성자: pastamonster (사진 11)
리뷰 5: 출근길에 커피박스 오늘의 메뉴 꼭 확인하고 가는편 ㅎ 꿀생강차 맛있음 (2022년 12월 7일 수요일) - 작성자: 7409 (사진 518)
리뷰 6: 자주방문하고있어요 (2022년 12월 9일 금요일) - 작성자: 듕아39 (사진 177)
리뷰 7: 굳 (2023년 2월 1일 수요일) - 작성자: 뭐임뫄어쩔 (사진 284)
리뷰 8: 맛있어여 (2022년 11월 29일 화요일) - 작성자: 리넛60 (사진 3)
리뷰 9: 자몽에이드 맛있어요 (2022년 9월 26일 월요일) - 작성자: tnstks6893 (사진 2)
리뷰 10: 저렴하지만 맛있습니다
친절하시구요~ 많이 파시고 부자되세요 (2022년 7월 6일 수요일) - 작성자: 뭐임뫄어쩔 (사진 284)
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 굿 (2022년 7월 6일 수요일) - 작성자: 아름다운당신을사랑합니다 (사진 32)
리뷰 12: 굿 (2022년 2월 11일 금요일) - 작성자: 샴푸의요정49 (사진 3)
리뷰 13: 굿 (2021년 11월 17일 수요일) - 작성자: susu9612 (사진 4)
리뷰 14: 맛있어요 (2021년 10월 28일 목요일) - 작성자: 이아름186

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%9E%A5%EC%88%98%EC%B4%8C%ED%92%8D%EC%B2%9C%EC%9E%A5%EC%96%B4/place/1997046045?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 장수촌풍천장어
업종 카테고리: 장어,먹장어요리
별점: 4.26
방문자 리뷰: 333
블로그 리뷰: 44
리뷰 탭 클릭 완료
총 리뷰 수: 159


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 넘 더워 몸 보신 맛나게 잘 먹음
버섯 추가로 시켰는데 좀 외소 ㅎ
3명이 2키로 여유있게 먹음 (2024년 7월 29일 월요일) - 작성자: 강 ja (사진 781)
리뷰 2: 위치와 가성비는 좋아요. 서비스(친절)는 기대하면 안됨. (2024년 10월 24일 목요일) - 작성자: Mezemura (사진 538)
리뷰 3: 첨으로 짜글이 먹어봄 ㅎ
장어 세명이 1키로는 조금 모자른듯
1키로 추가 주문
너무 배부르고 맛나게 먹고옴 (2024년 7월 13일 토요일) - 작성자: 강 ja (사진 781)
리뷰 4: 장어 1키로 59000원
상차림 1인당 3000원
장어 완전 싱싱하고 맛있어요.
집과 가까운곳에 있어 자주 가야되겠어요. (2024년 7월 16일 화요일) - 작성자: 보름이27 (사진 758)
리뷰 5: 싱싱하고 장어 큼직해 먹을만해요 (2024년 8월 2일 금요일) - 작성자: 보름이27 (사진 758)
리뷰 6: 두 곳 가봤는데 두 곳다 너무 만족스럽게 먹고 왔습니다. 장어 먹고 싶을때 가는 곳입니다. (2024년 5월 21일 화요일) - 작성자: lan**** (사진 938)
리뷰 7: 한까 괜찬아요 (2024년 9월 13일 금요일) - 작성자: zhenzhu76 (사진 69)
리뷰 8: 역시 맛있다 대파김치가 별미~
사장님도 친절하시다 (2024년 5월 25일 토요일) - 작성자: 강 ja (사진 781)
리뷰 9: 점심특선 가성비, 맛 최고!! (2024년 10월 11일 금요일) - 작성자: yoa**** (사진 320)
리뷰 10: 이게 장어덮밥 정식이래요
어딜봐서 장어덮밥인지, 전에 있던 장어정식 이름만 바꾼거같은데 어이가 없어요 (2024년 6월 17일 월요일) - 작성자: 더나39 (사진 3)
로드된 리뷰 개수: 20

리뷰 108: 쩝쩝박사인 지인 소개로 갔는데
말 그대로 너무 맛있고 신선하고
안느끼하네욤 ㅋㅋ 맛있었어요 😋

뭔가 계속 먹을수 있는 맛
서비스로 나온 묵사발이 두번째로 맛있음

또 가고싶어요 >< (2021년 10월 3일 일요일) - 작성자: Ariana Grande size (사진 3,341)
로드된 리뷰 개수: 120, 수집된 리뷰 개수: 108
리뷰 109: 장어구이 맛있어요. (2021년 12월 31일 금요일) - 작성자: 나나무라 (사진 692)
리뷰 110: 점심 (2022년 1월 27일 목요일) - 작성자: SirMD (사진 41)
리뷰 111: 점심 (2022년 1월 25일 화요일) - 작성자: SirMD (사진 41)
리뷰 112: 좋아요 (2022년 6월 10일 금요일) - 작성자: won**** (Unknown)
리뷰 113: 좋아요 (2022년 7월 4일 월요일) - 작성자: 아힌6795 (사진 2)
리뷰 114: 좋아요 (2022년 7월 4일 월요일) - 작성자: youji841 (Unknown)
리뷰 115: 좋아요. (2022년 3월 12일 토요일) - 작성자: luft08 (사진 8)
리뷰 116: 굿굿 (2022년 10월 7일 금요일) - 작성자: Laksmi (사진 9)
리뷰 117: 조ㅅ아요 (2022년 6월 13일 월요일) - 작성자: kjo**** (팔로워 2)
리뷰 118: ㅑ (2022년 2월 7일 월요일) - 작성자: h45**** (Unknown)
로드된 리뷰 개수: 130, 수집된 리뷰 개수: 118
리뷰 119: 음식은 맛있는데 양이 적어요 (2021년 11월 29일 월요일) - 작성자: 천하의김선달 (사진 14)
리뷰 120: 예술의 전당 피카소전 보고 일행 2명과 나까지 3명이 여름 폭염대비 몸보신하러 가서 먹은 장어구이. 맛은 두툼하고 맛이 너무 좋아 일부러 장어구이 먹으러 찾아다니는건 아닌데 엄지척! 밑반찬도 하나같이 맛있어서 정신 못차리고 흡입. 난 다엿중이었고 ㅠㅠ. 다엿은 낼부터. 조만간 한번 더 

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%82%AC%EC%B2%9C%EB%A3%A8/place/1917137340?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 사천루
업종 카테고리: 중식당
별점: 방문자 리뷰 37
방문자 리뷰: 8
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 21


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 처음 먹어보는 냉짬뽕 새콤달콤 맛있었고
탕수육은 바로 튀긴 거 같은데 너무 단단해서 씹기가..ㅜㅜ
다른 날은 오늘 같지 않고 맛있었다고 함 (2024년 8월 23일 금요일) - 작성자: 버들강아지는 AB형 (사진 421)
리뷰 2: 음식이 맛있고 빠르게 나옵니다. 혼밥도 좋고 빠르게 식사할 때도 좋아요~ (2024년 10월 8일 화요일) - 작성자: 파워배추 (사진 1,515)
리뷰 3: 일반 짜장이 맛있는 집이지요.
간짜장도 먹어봤는데 맛있었어요.
가게가 크지 않아서..
점심시간에는 웨이팅이 기본인 집입니다.
그치만 가격 착하고, 맛도 조코!
좋아용~ (2024년 7월 8일 월요일) - 작성자: idisjien94 (사진 1,088)
리뷰 4: 친절하고 매장 깨끗하고 음식맛이 깔끔해요ㅎㅎ 바삭한 탕수육 제가 좋아하는 스타일이에요 >.< 번창하세요 (2024년 5월 21일 화요일) - 작성자: 뱅뱅17 (사진 123)
리뷰 5: 맛있고 배달도 빨라요 (2024년 9월 20일 금요일) - 작성자: iys**** (사진 92)
리뷰 6: 빠르고 친절한 배달와주셨지만 해물이 더 있어야 한다고 생각을 했어요! ! 담에 또 먹을게요! (2024년 6월 16일 일요일) - 작성자: ta**** (사진 1,284)
리뷰 7: 좀 이른 점심을 먹기위해 11시반쯤 방문 !!
음식도 빨리나오고 직원분들도 친절해 기분좋게
먹고있는데 사장인지 매니저인지 출근해선
아니 손님도 없는데 에어컨을 켰냐며 직원들 타박시작
혼자지만 나도 손님이긴 한데...
제가 켜달라한건 아니었어요에효
그래도 짬뽕은 맛있었어요 (2024년 5월 30일 목요일) - 작성자: 윤 뚱구 (사진 9)
리뷰 8: 맛있고 양 많고 친절합니다. (2024년 6월 13일 목요일) - 작성자: 하하하9768 (사진 2

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%91%90%EB%91%90%EB%8F%BC%EC%A7%80%EB%B6%88%EB%B0%B1/place/36690429?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 두두
업종 카테고리: 한식
별점: 4.36
방문자 리뷰: 421
블로그 리뷰: 224
리뷰 탭 클릭 완료
총 리뷰 수: 183


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 두두 여긴 찐 숨은 양재역맛집 입니다
그것도 네이티브들만 아는...
등갈비김치찜은 완존 극락맛
김치가 최극강에 등갈비는 끝도 없이
나와요 (2024년 10월 31일 목요일) - 작성자: joy**** (사진 111)
리뷰 2: 가족끼리 집근처라 들렸는데, 진짜 너무 맛있어서 재방문 할 예정입니다!! 등갈비 김치찜 너무 맛있어요!! 신김치 한포기 들어가있고 고기도 부드러워서 잘 발려 편하게 먹을수있었습니다!! 흰쌀밥에 같이 먹으니 최고입니다....❤️ 반주하기도 좋아용 (2024년 7월 12일 금요일) - 작성자: 최와와와 (사진 656)
리뷰 3: 양이 푸짐하고요
김치가 맛있어요 고기도 연하고 많이들어있어서 ㄴ좋았어요
1인 1만4천원이라
가성비 있다고생각해요 김치찜먹어보니 불백은 보나마나 맛있을거같다 생각했는데
불백 맛집으로 이미 유명하더라구요!

국물에 김치 넣고 쓱싹 비벼서 다 먹었어요 (2024년 7월 16일 화요일) - 작성자: 간장통닭 (사진 368)
리뷰 4: 김치찜 너무 푸짐하게 잘먹었어요!!!!!☺️👍🏻
가성비도 너무 좋아요.. 저녁에 한식에 술한잔 하고싶으시다면 강추입니닷>.<
글고 뼈랑 살이랑 엄청 잘 발려서 먹기도 짱 편해요! (2024년 10월 31일 목요일) - 작성자: hanb32 (사진 31)
리뷰 5: 등갈비 김치찜 2인분 주문했는데 양이 진짜 넉넉하고 등갈비도 많이 들어있어서 가성비 진짜 좋아요 👍 사장님 친절하시고 음식도 반찬도 다 맛있어서 자주 들를것 같아요 (2024년 8월 26일 월요일) - 작성자: 리뷰에진심이거든요 (사진 570)
리뷰 6: 진짜 너무 맛있습니다.. 등갈비김치찜 미역국 오뎅 반찬들 다 맛있네요 멀리사는데 다음에 또 오고 싶을 정도입니다 잘먹고갑니다 사장님! (2024년 11월 7일 목요일) - 작성자: 

리뷰 59: 좋아요 (2024년 5월 20일 월요일) - 작성자: 바다코끼리61 (사진 1)
리뷰 60: 김치찜 맛나게 먹었어요~^^ (2024년 5월 10일 금요일) - 작성자: hjlim001 (사진 34)
로드된 리뷰 개수: 70, 수집된 리뷰 개수: 60
리뷰 61: 맛있는 점심 한끼 먹었습니다. (2024년 1월 11일 목요일) - 작성자: 다나유 (사진 104)
리뷰 62: 밑반찬도 많고 돼지불백 맛이 깔끔해서 좋습니다! (2023년 12월 20일 수요일) - 작성자: Jung100099 (사진 46)
리뷰 63: 정말 좋았어요 (2024년 1월 17일 수요일) - 작성자: Baldeagle (사진 9)
리뷰 64: 굿 (2024년 3월 5일 화요일) - 작성자: rub**** (사진 416)
리뷰 65: 미미!!!!

넘 맛있어요!🩷
특히 김치가 도른자!!

담에 불백먹으러 갈게용🤭
번창하세여!!! (2023년 11월 21일 화요일) - 작성자: BKBKBK (사진 929)
리뷰 66: 굿 (2024년 1월 24일 수요일) - 작성자: 오솔길풍경 (사진 1)
리뷰 67: 굿 (2024년 1월 23일 화요일) - 작성자: 오솔길풍경 (사진 1)
리뷰 68: 공기밥은 따로추가라 아쉬웠음 (2023년 9월 4일 월요일) - 작성자: 분노의질주77 (사진 30)
리뷰 69: 두두돼지불백은 언제 먹어도 정말 맛있습니다👍 오래오래 장사해주세요💓 그리고 2번이상 방문하신다면 김치찜 도전 추천드립니다. 존맛이에요 (2023년 7월 6일 목요일) - 작성자: 도로그 (사진 120)
리뷰 70: 맛나다 (2024년 5월 20일 월요일) - 작성자: hh**** (사진 19)
로드된 리뷰 개수: 80, 수집된 리뷰 개수: 70
리뷰 71: 인태리어도 멋지고
친절하고
음식은 메뉴별로
다 맛있어요 (2023년 6월 20일 화요일) - 작성자: Awildpink (사진 130)
리뷰 72: 괜춘괜춘한 점심 집!! (2023년 8월 9일 수요일) - 작성자: 뜨끈뜨끈호떡 (

'더보기' 버튼이 더 이상 없습니다.
총 183개의 리뷰를 수집했습니다. (예상 총 리뷰 수: 183)
리뷰 데이터가 '두두_reviews.csv' 파일로 저장되었습니다.


In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EA%B3%A0%ED%92%88%EA%B2%A9%EC%BB%A4%ED%94%BC%EA%B3%B5%EC%9E%A5%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1710007700?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 고품격커피공장 뱅뱅사거리점
업종 카테고리: 카페
별점: 방문자 리뷰 270
방문자 리뷰: 6
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 103


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 여유로운 오전 크로플과 커피 한 잔이면 하루 에너지 충전 완료 입니다!
사장님 크로플 잘 구우세요~
꼭 드셔보세요^^ (2024년 11월 6일 수요일) - 작성자: 삼복더위15 (사진 7)
리뷰 2: 직원분들이 매우 친절하세요~ ㅎ (2024년 9월 27일 금요일) - 작성자: 배산임수85 (사진 485)
리뷰 3: 근처에 볼 일이 있어서 들렸어요. 음료 시키면 크로플 500원 할인됩니다. 설탕을 녹여서 코팅이 살짝 되있어서 그런지 시간이 지나도 바삭하고 커피랑 궁합도 너무 좋네요. 매장은 작지만 자리마다 콘센트가 있어서 충전하기 편했어요. 응대해주신 여자 사장님이 엄청 친절하세요! 덕분에 혼자왔지만 잘 이용하고 갑니당~♡ (2024년 3월 26일 화요일) - 작성자: YekiM (사진 232)
리뷰 4: 더운 날씨엔 오미자 뱅쇼가 최고👍
올때마다 기분이 좋아지는 나만의 동네 맛집~♡
사장님 예쁘고 친절하신곳~ (2024년 6월 18일 화요일) - 작성자: 삼복더위15 (사진 7)
리뷰 5: 이 동네에서 가장 많이 이용하는 까페에요~ 친절하게 빠르게 음료를 준비해줍니다~ ㅎㅎ (2024년 4월 22일 월요일) - 작성자: 파워배추 (사진 1,515)
리뷰 6: 회사앞에 있어서 자주 이용하는 커피집이에요~~~ (2024년 3월 29일 금요일) - 작성자: 파워배추 (사진 1,515)
리뷰 7: 항상 친절하게 음료준비해주십니다~ 도곡동 뱅뱅사거리의 까페3대장!!!!!! (2024년 4월 15일 월요일) - 작성자: 파워배추 (사진 1,515)
리뷰 8: 가성비 가심비 둘 다 좋은곳^^
오늘도 감사합니다🙇 (2024년 2월 2일 금요일) - 작성자: 반야수현 (사진 1,045)
리뷰 9: 친절합니다 (2024년 5월 20일 월요일) - 작

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%B8%8C%EB%9D%BC%EC%9A%B4%EB%8F%88%EA%B9%8C%EC%8A%A4%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1776021604?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 브라운돈까스 뱅뱅사거리점
업종 카테고리: 돈가스
별점: 방문자 리뷰 194
방문자 리뷰: 33
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 92


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 돈까스
• 바삭하고 두툼한 돈가스의 겉바깥이 황금빛 브라운 색으로 잘 튀겨져 나왔습니다
• 고기는 부드럽고 육즙이 풍부했으며, 특제 소스와 잘 어울렸습니다
• 곁들임 반찬 나온 피클과 깍두기도 신선했습니다

김치 우동
• 쫄깃한 면발과 얼큰한 국물이 조화롭게 어우러졌습니다
• 김치의 시원하고 매콤한 맛이 우동의 담백함을 살려주었습니다 (2024년 11월 4일 월요일) - 작성자: 상품 스케치 (사진 161)
리뷰 2: 돈까스는 바삭한 튀김옷과 부드러운 고기의 조화가 일품이었습니다. 고기는 두툼하면서도 육즙이 풍부했고, 튀김옷은 기름기가 적당해 느끼하지 않았습니다.

모밀은 쫄깃한 면발과 깔끔한 국물이 돈까스의 느끼함을 중화시켜주어 균형 잡힌 한 끼가 되었습니다. 시원한 모밀 국물과 돈까스는 매우 좋은 선택이었네요.
양도 넉넉해서 든든한 식사가 되었습니다. (2024년 10월 24일 목요일) - 작성자: 상품 스케치 (사진 161)
리뷰 3: 돈까스는 외관부터 바삭하고 황금빛으로 잘 튀겨진 것이 눈에 띄고 첫 입을 먹자마자 느껴지는 바삭한 식감과 부드러운 고기의 조화는 최고에요. 소스도 적당히 달콤하면서도 짭조름해서 돈까스와 적절한 조화입니다

우동은 부드럽고 쫄깃한 면발에 국물 맛이 조금 짭조름 하지만 큰부담 없이 즐길 수 있었어요.

돈까스의 바삭함과 우동의 쫄깃함이 잘 어우러져 맛있는 한 끼였어요. (2024년 10월 14일 월요일) - 작성자: 상품 스케치 (사진 161)
리뷰 4: 돈까스는 바삭바삭하고 겉은 황금빛, 속은 촉촉한 돼지고기가 입안에서 사르르 녹아요. 고소한 소스가 잘 어우러져서 정말 맛있었어요.

냉모밀은 시원한 국물에 쫄깃한 면발이 한 입 가득 들어오면서, 더운 날씨에 딱이에요. 특히, 국물의 감칠맛이 더해져서

'더보기' 버튼이 더 이상 없습니다.
총 92개의 리뷰를 수집했습니다. (예상 총 리뷰 수: 92)
리뷰 데이터가 '브라운돈까스 뱅뱅사거리점_reviews.csv' 파일로 저장되었습니다.


In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 비채
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%84%9C%EC%B4%88%EB%82%A8%EC%88%9C%EB%82%A8%EC%88%9C%EB%8C%80%EA%B5%AD%20%EB%B3%B8%EC%A0%90/place/11692888?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 남순남순대국 본점
업종 카테고리: 순대,순댓국
별점: 4.11
방문자 리뷰: 818
블로그 리뷰: 259
리뷰 탭 클릭 완료
총 리뷰 수: 313


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 비가 오고 날씨가 흐린 날엔 무조건 순대국인데, 양재역 주변에 이미 직장인 맛집들은 가봤지만, 뱅뱅사거리쪽에 위치한 집으로 가봄

저녁 시간대에 가니깐 은근 주민들이 순대국에 막걸리 마시러 오심 매장은 꽤나 커서 낮에는 직장인으로 붐빌것 같음

토종순대국에는 찹쌀순대랑 곱창이 무지막지하게 들어있어서 받았을때 비주얼보고 잠시 흠짓했지만 먹어보니깐 잡내 1도 없어서 물에 빠진 곱창은 다먹은적 처음이었음

순대도 아주 큰 크기로 네개 정도 들어있어서 꽤나 푸짐했음 간은 이미 맞춰서 나와있어서 들깨랑 다대기만 추가 해서 먹었는데 적당했음 그리고 안에 부속고기는 야들야들하니 여기서 파는 족발도 맛있겠는걸 하는 생각이 들었음

그리고 가게가 뭔가 힙하고 밀키트 팔고있는 자체가 신기했음 (2024년 9월 6일 금요일) - 작성자: 웱오옹 (사진 1,286)
리뷰 2: 맛있네요 순대국. 일반 순대국과 토종 순대국이 있는데, 토종순대국 막었습니다. 일반 순대국은 그냥 당면 순대가 들어있다고 합니다. 일단 국물의 감칠맛이 어마합니다. 새우젓 조금 넣고 먹으니 너무 맛있어요. 그리고 건더기가 엄청 많이 들어있어요. 특히 순대가 참 통통하니 맛있는데 야채가 많이 들어가있어요. 특히 김치 들어가 있는 순대가 있었는데 이건 진짜 맛있어요. 뭔사 김치의 시원한 맛과 특유의 감칠맛이 입에 착 달라붙네요. 고기 양도 꽤 많고 고기 질도 좋은 것 같아요. 두툼하고 큼직하게 썰려있어서 입에 가득 채우고 와구와구 먹었습니다. 진짜 맛있게 잘 먹었습니다. (2024년 9월 4일 수요일) - 작성자: 키킴킴 (사진 1,141)
리뷰 3: 밥때를 놓쳐 혼밥하게 된 오늘. 남순남순대국 왔어요. 혼밥도 가능하고, 무엇보다 순대국 맛이 일품. 옆테이블 손님들이 토종순대국 맛있다며 계속 

리뷰 88: 맛있어요. (2023년 5월 30일 화요일) - 작성자: syu**** (사진 18)
리뷰 89: 전보다 맛이 살짝 덜해진 것 같아요ㅎ 깍두기 맛있음 (2023년 1월 30일 월요일) - 작성자: 뱅뱅17 (사진 123)
리뷰 90: 삼색순대도 맛나고 순대국도 맛납니다. (2022년 10월 28일 금요일) - 작성자: 켈리0315 (사진 682)
로드된 리뷰 개수: 100, 수집된 리뷰 개수: 90
리뷰 91:  (2023년 3월 9일 목요일) - 작성자: 하와이키 (사진 138)
리뷰 92: 오래된 가게로 맛있게 먹었어요 (2023년 5월 9일 화요일) - 작성자: 자구대 (사진 1)
리뷰 93: 아버지가 여기순대국 진짜 좋아하시구 3색순대 맛있어요~ (2023년 1월 11일 수요일) - 작성자: petitgosol (사진 77)
리뷰 94: 토종순대국 먹었어요
벽에 붙어 있는 슌대국 맛있게 먹는법 을 따라서 하니까 더 풍성한 맛이 되었네요~
완뚝!

👍🏻맛있게 먹는법👍🏻
다대기 한스푼
새우젓 2/3스푼
들깨가루 한스푼
후주 약간 (2022년 11월 12일 토요일) - 작성자: 여린고기 (사진 1,102)
리뷰 95: 맛있어요~ 깍두기도 맛있네요 (2022년 11월 25일 금요일) - 작성자: 뱅뱅17 (사진 123)
리뷰 96: 일반 순대국 주문했고요
국에 들어있는 고기도 실하고 국물도 맛있네요
다음에는 다른 메뉴도 먹으러 갈게요 (2022년 11월 12일 토요일) - 작성자: 우앙7020 (사진 542)
리뷰 97: 순대국을 별로 좋아하지 않는데 손님이 먹고 싶다 해서 ㅠ.ㅠ
비린내 나지 않고 아빠가 좋아하시던 맛 같아서 급... 아빠 보고 싶은 음식!!(^_^) (2022년 12월 8일 목요일) - 작성자: 원더우먼1004 (사진 484)
리뷰 98: 순대가 너무 맛있어요 (2023년 1월 19일 목요일) - 작성자: 리뷰왕 으니 (사진 173)
리뷰 99: 순대국 알아주는 맛집이죠 (2022년 11월 16일 수요일) - 작성자: 정때문에 

리뷰 207: 맛나요 (2022년 1월 25일 화요일) - 작성자: 안식 연구소 (사진 2)
리뷰 208: 족발은 드시지 마세요 (2021년 11월 23일 화요일) - 작성자: Sang Honey (사진 21)
리뷰 209: 맛남 (2022년 1월 20일 목요일) - 작성자: soo**** (사진 1)
리뷰 210: 굿 (2022년 7월 12일 화요일) - 작성자: 행복만들기79 (사진 876)
로드된 리뷰 개수: 220, 수집된 리뷰 개수: 210
리뷰 211: 조아용 (2022년 8월 11일 목요일) - 작성자: 인증달인 (사진 5)
리뷰 212: 좋아요 (2022년 1월 8일 토요일) - 작성자: 안녕하세용773 (사진 13)
리뷰 213: 굿 (2022년 5월 11일 수요일) - 작성자: kjezzang11 (사진 57)
리뷰 214: 음식이 맛있어요 (2021년 10월 28일 목요일) - 작성자: 천하의김선달 (사진 14)
리뷰 215: 진짜 맛있네요 양도 엄청많아요 만원의행복 ㅋㅋ (2021년 7월 23일 금요일) - 작성자: 오거으리 (사진 630)
리뷰 216: 맛있어요 (2021년 11월 24일 수요일) - 작성자: moo**** (사진 21)
리뷰 217: 맛있었어요 (2021년 11월 9일 화요일) - 작성자: lqwel (사진 288)
리뷰 218: 굿 (2021년 12월 2일 목요일) - 작성자: 호그위드 (사진 2)
리뷰 219: 굿 (2021년 12월 2일 목요일) - 작성자: 열매3234 (사진 2)
리뷰 220: 좋아요 (2022년 2월 18일 금요일) - 작성자: 표준건설 (사진 7)
로드된 리뷰 개수: 230, 수집된 리뷰 개수: 220
리뷰 221: 좋아요 (2022년 3월 19일 토요일) - 작성자: ami**** (사진 57)
리뷰 222: D (2021년 11월 11일 목요일) - 작성자: scyk00 (사진 1)
리뷰 223: 굿 (2021년 10월 27일 수요일) - 작성자: 미밍1360 (사진 34)
리뷰 224:

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%99%95%EC%82%B0%EA%B3%A8/place/1164024954?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 왕산골
업종 카테고리: 한식뷔페
별점: 방문자 리뷰 43
방문자 리뷰: 6
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 30


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 도곡동의 한식뷔페인데
다른곳의 한식 뷔페보다 반찬종류는 많지는않아요
카드1만원
현금9천원
조금 아쉽습니다 (2024년 10월 24일 목요일) - 작성자: 제주 스텔스 (사진 2,289)
리뷰 2: 엄마가 해주는 맛있는 밥집^^
올때마다 기분 좋게 먹고 가용~
이 집 치킨도 맛남 (2024년 8월 23일 금요일) - 작성자: flo**** (사진 4)
리뷰 3: 이젠 2차에도 갑니다ㅋㅋㅋ
점심식사에서 2차맥주까지~!! (2024년 7월 8일 월요일) - 작성자: 배산임수85 (사진 485)
리뷰 4: 가까워서 좋아요 (2024년 11월 11일 월요일) - 작성자: 김귀임 교원맘 (사진 35)
리뷰 5: 음식이 금방 빠지면 같은 메뉴로 채워주세요~~~ ㅠ (2024년 6월 18일 화요일) - 작성자: 배산임수85 (사진 485)
리뷰 6: 음식은 맛있지만 가장 절대적으로 필요한것이라고 느끼는점은
오늘의 메뉴가 어떤건지 입구쪽이나 1층에 공지해주시면 좋겠습니다
근처가 직장이라 자주 지나가는데 호불호가 있는 음식이 나올까봐 그냥지나친적도많은데 무턱대고 들어갔다가 닭볶음탕(닭을 못먹음)나와있으면 성격상 나오지도못하고 기본찬들만 먹고 나온적도있습니다 제발 간곡히 부탁드립니다 번거롭더라도 오늘의 메뉴 공지해주시면 더 자주갈수있을것같아요
꼭 그런방법아니더라도 sns같은곳으로 공지해주는곳도 있더라고요
그냥 저의 희망사항 적어봤습니다 (2024년 4월 17일 수요일) - 작성자: chu**** (Unknown)
리뷰 7: 사장님 너무 친절하시고, 일단 맛이 집밥입니다.
엄마밥이 그리울땨 자주 가요!
그리고 2층에 있어그런지 밥먹으면서도 경치보며 먹어서 더 좋아요! (2024년 4월 16일 화요일) - 작성자: 심신수련 (사진 11)
리뷰 8: 여러가지 종류의 반찬이 다양

In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%B9%B4%EB%A6%B0%EC%A7%80%EB%A6%B0%EA%B0%80%EB%84%A4%EC%8A%A4%EB%82%B5%EB%B0%94%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/1850658022?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 카린지린가네스낵바 뱅뱅사거리점
업종 카테고리: 요리주점
별점: 방문자 리뷰 356
방문자 리뷰: 2,373
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 323


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 귀여운 소품이 한가득 있는 카린지 린가네 스낵바💕💕 제가 감탄을 하며 사진찍으니 여사장님께서 본인도 넘 좋아하셔서 귀여운것들 전시했다해요ㅎㅎ #양재데이트장소 로도 넘 추천이고! 친구들끼리 간술, 혹은 혼술도 좋을 것 같더라구요.
안주는 양은 적지만 그만큼 가상비도 최고!!!
부담없이 여러개 주문해도되서 좋드라구용ㅎㅎ
가게앞에서 포토존으로 사진찍어도 좋을듯ㅎㅎ #양재술집 으로 추천합니다~~~~~ (2024년 10월 19일 토요일) - 작성자: 지은4959 (사진 153)
리뷰 2: 정말 오랜만에 마음에 드는 일본식 술집을 발견했어요! 안주가 전체적으로 저렴한 편이라 처음엔 큰 기대 안했는데, 모든 메뉴가 다 맛있네요. 인테리어도 정말 예쁘게 해놓으시고 브금도 좋아서 일본 느낌 가득 ㅎㅎ~ 사장님도 친절하시구 강아지가 너무너무 예뻐서 힐링하구 가요. 다음에 또 놀러갈게요 :) (2024년 10월 10일 목요일) - 작성자: NANCY (사진 83)
리뷰 3: #양재데이트 왔다가 #양재역이자카야 찾아 온 곳이에요. 아 인테리어가 넘모 귀여워요 ㅠ 취향저격~~~🤍 아담하지만 그안에 볼거리•귀여운게 가득찬 곳으로 데이트코스로 좋은곳이에요💕

점심에는 카린지 , 저녁에는 린가네 로 운영중이에요 ㅋㅋㅋ 첨에는 이름을 두글자씩 끊어읽었는데ㅋㅋㅋ

암튼! 메뉴도 부담스럽지않게 1-2인분으로 가격도 저렴해요✨ 다양하게 주문할 수 있어서 좋고 남기지않아도되서 더욱 좋았던 곳이에요!!!!! #양재역데이트 코스 찾으신다면 추천추천~😆😆😆 (2024년 10월 19일 토요일) - 작성자: 찌니야1 (사진 1,041)
리뷰 4: 저녁식사 겸 들러본 곳인데, 점심엔 카린지(카레&카츠), 저녁엔 린가네 스낵바로 운영해서 이름이 린가네카린지스낵바 입니다

리뷰 55: 집 근처에 요래 맛진 곳이 있었다니! 맛도 좋고 분위기도 좋고 깔끔하고 편히가기 좋아요 😶 번창하세요 (2024년 4월 12일 금요일) - 작성자: min**** (사진 8)
리뷰 56: 완전맛있어요ㅎㅎㅎ 두번째 방문인데 잘먹고 갑니당 ㅎㅎ (2024년 6월 4일 화요일) - 작성자: 련화2 (사진 2)
리뷰 57: 맨날웨이팅있어서 기다렸는데 오늘은 바로들어왔어요! 가성비 너무 좋고 2차로 오기 너무좋아요💛 (2024년 2월 15일 목요일) - 작성자: 영띠32 (사진 56)
리뷰 58: 가성비 최고!! 분위기도 넘 좋아요 꼭 가세용❤️❤️ (2024년 2월 2일 금요일) - 작성자: kkkkkzzzz (사진 494)
리뷰 59: 추운겨울 따뜻한 사케를 찾아 우연히 들어간집
레트로한 일본감성이 맞이해주었습니다.
오뎅 모리와세와 따뜻한사케에 몸도 따뜻해지고 친철한 사장님에 마음까지 따뜻해졌습니다. 음식과 감성 모두 좋은집 추천합니다 (2023년 12월 23일 토요일) - 작성자: dusdn20 (사진 50)
리뷰 60: 2차로 간단히 마시기 좋은 곳이에요
안주들도 작게작게 다양하고 괜찮네용 !
(화장실은 별루..) (2024년 2월 27일 화요일) - 작성자: 하이빈나 (사진 727)
로드된 리뷰 개수: 70, 수집된 리뷰 개수: 60
리뷰 61: 귀여운 강아지가 있어요♡ 너무 귀여워요♡ 음식도 맛있어요~~! 2차 하기 딱입니다!! (2024년 2월 29일 목요일) - 작성자: jiw**** (사진 58)
리뷰 62: 분위기좋아요 근데 대부분예약석이라
예약하시고 방문추천해요. (2024년 2월 1일 목요일) - 작성자: 트래블랑 (사진 365)
리뷰 63: 맛있는 메뉴가 많아서 무엇부터 먹을지 고민이 많았네요.
음식도 빨리 나오고 무엇보다 너무나 맛있었네요.
인테리어도 아기자기하게 잘 꾸며있어서 볼거리도 많았답니다. 추운날씨에 김포에서 뱅뱅사거리까지 간 보람이 있었네요. 맛있게 잘 먹었습니다. (2023년 11월 18일 토요일) - 작성자: 스텔라1

리뷰 118: 레트로 느낌나고, 조명도 분위기 너무 좋고 여자친구랑 좋은 시간 보내다가 가요!!
물론 음식도 되게 깔끔하니 맛있게 먹었어요 😆 (2023년 10월 14일 토요일) - 작성자: 와이퍼45 (사진 16)
리뷰 119: 진짜 너무너무 좋아요!
제가 일본술집을 너무 좋아하는데 분위기도 너무 좋고 그리고 기린쇼맥 꼭 먹어야돼요 진짜 세번 네번 드세요 완전 강추입니다! (2023년 10월 7일 토요일) - 작성자: 술쟁이55 (사진 22)
리뷰 120: 안주가 다 너무 맛있어요 😋😋
분위기도 좋고 술 종류도 다양해서 좋아요!
간단하게 한 잔 하기 좋은 곳이에요🤩 (2023년 10월 5일 목요일) - 작성자: 모네55 (사진 34)
로드된 리뷰 개수: 130, 수집된 리뷰 개수: 120
리뷰 121: 뱅뱅사거리에 새로 생긴 일본식 스낵바

저녁에 방문했고 시그니처 메뉴인
파스타와 튜나햄카츠 테바사키 김치나베 먹음

양이 많진 않지만 가볍고 다양하게 메뉴
고른 후 술 먹기 좋음 (간도 술먹기에 적절)

인테리어도 예쁘고 접객도 친절 (2023년 10월 17일 화요일) - 작성자: 이상하59 (사진 160)
리뷰 122: 간단하게 한 잔 하기 딱 좋은 곳이에요.
선릉점과 다른게 패드로 주문 넣을 수 있었고 내부는 조금더 작았어요.
화장실은 돌아나가 끝에 있는데 조금 무서웠어요.
음식들은 다 맛있는데,,저 황도 저건 맛이 없어서 다 남겼네요ㅎㅎ개인 취향이니!
분위기도 엄청 좋아요ㅎㅎ (2023년 10월 9일 월요일) - 작성자: Komolebi (사진 369)
리뷰 123: 요리 너무 귀엽고 이쁘게 생겼어요! 맛도 너무 좋구 최고네요…
재방문 의사 150% 있습니다 (2023년 11월 4일 토요일) - 작성자: 김다운92 (사진 11)
리뷰 124: 메뉴도 다양하고 진짜 맛있어요! 자주 올 것 같아요☺️ (2023년 11월 20일 월요일) - 작성자: 지짜이 (사진 12)
리뷰 125: 혼밥존이 있다는 리뷰 보고 갔는데, 저녁에 혼밥하기엔 별로입니다. 사람들이 

로드된 리뷰 개수: 200, 수집된 리뷰 개수: 190
리뷰 191: 첨 와봤는데 너무 좋아요!!
저녁근무만아니면 오래 있다가고싶은데ㅜㅠ (2023년 8월 26일 토요일) - 작성자: sophie8692 (사진 111)
리뷰 192: 단품 안주류와 간단한 한잔하기
좋으네요
하이볼도 좋아요 (2023년 12월 8일 금요일) - 작성자: 나르샤25 (사진 186)
리뷰 193: 지나가다 새로 생겼길래 들어왔는데 분위기가 너무 좋네요 음식도 종류별로 시키기 좋구 안주도 맛나고 인테리어 너무 귀여워요 😍 😍
다음에 지인들 모시구 또올게요 (2023년 8월 16일 수요일) - 작성자: wij**** (사진 12)
리뷰 194: 가게가 너무 이뻐요 귀여운 피규어랑 사진으로 꾸며서 사진 찍기도 넘모 좋아요 ><
직원분들도 음식설명 친절하게 해주셧어요 사와처음 먹어보는데 너무 맛있고 하이볼도 맛있어요!! 무엇보다 음식 맛있어서 종종 밥먹으러도 올것같아요 가끔 한잔하러 오기 좋은 곳같아요 감사합니다 또오겠습니다^^ (2023년 8월 23일 수요일) - 작성자: 없어6576 (사진 1)
리뷰 195: 분위기가 너무 좋고 안주가 진짜 맛있어요 집 가까운 곳에 좋은 술집이 생겨서 좋네요 다들 오세요 ㅎㅎ (2023년 9월 16일 토요일) - 작성자: Ao렌지 (사진 35)
리뷰 196: 이색적인 이자카야! 분위기 갑! 맛있게 마셨습니다~ 술종류가 다양해서 신난단 (2023년 10월 19일 목요일) - 작성자: 오기948 (사진 5)
리뷰 197: 집주변에 이런곳이 생기다니 너무 행복합니다. 혼술하기에도 너무 좋을 것 같습니다! 일본 이자카야 온 느낌이에요! 안주도 가볍고 타파스느낌이라 가볍게 먹기에 너무 좋습니다! (2023년 11월 4일 토요일) - 작성자: 프라푸차니 (사진 16)
리뷰 198: 동네에 생겨서 와봤네요 간단히
한잔 하기 좋아요 (2023년 10월 27일 금요일) - 작성자: 요세후 (사진 9)
리뷰 199: 시킨 음식이 다 맛있었어요.
하이볼도 종류별로 있어서 

리뷰 277: 새로생겨서 와봤는데 너무 맛있어요~ 또올게요 (2023년 8월 26일 토요일) - 작성자: Cherylll (사진 8)
리뷰 278: 동네에 예쁜 술집 생겨서 들어와봤는데 시그니챠 오뎅탕도 너무맛있고 2차로 도란도란 오기 좋았어요 또 오게 될 것 같아요!! (2023년 8월 9일 수요일) - 작성자: 너굴너굴65 (사진 3)
리뷰 279: 성수에서 너무좋았던 경험
양재에서 이어가봅니다~~ (2023년 9월 26일 화요일) - 작성자: 고구마8125 (사진 6)
리뷰 280: 분위기도 너무 좋구 너무 친절하세요!! (2023년 9월 23일 토요일) - 작성자: 공육공육 (사진 35)
로드된 리뷰 개수: 290, 수집된 리뷰 개수: 280
리뷰 281: 새로 생겨서 와봤는데 분위기 귀엽고 좋아용 (2023년 8월 22일 화요일) - 작성자: 호두누낭 (사진 33)
리뷰 282: 인테리어랑 분위기 좋고 음식도 맛있어요!!>_< (2023년 9월 22일 금요일) - 작성자: 얄리7036 (사진 3)
리뷰 283: 혼밥이랑 혼술하기 좋은 곳이에요! 분위기도 넘 좋고 한국에서 우메슈 파는 곳 찾기 어려운데 있어서 넘 좋아요 🥹🤍 (2023년 9월 1일 금요일) - 작성자: Billionaire2 (사진 29)
리뷰 284: 맛있어요호☺️☺️ (2023년 9월 19일 화요일) - 작성자: 이서예66 (사진 10)
리뷰 285: 처음 왔는데 맛있습니다! 혼술하기 좋아요!!! (2023년 9월 19일 화요일) - 작성자: 라닝6146 (사진 8)
리뷰 286: 이색적인 인테리어에 끌려 들어왔는데 분위기도 너무 좋고 직원분들도 정말 친절하세요💓 (2023년 9월 5일 화요일) - 작성자: songyi486 (사진 11)
리뷰 287: 동네에 이런곳이 있는지 처음 알았어요!! (2023년 8월 14일 월요일) - 작성자: 잠실리뷰어 (사진 8)
리뷰 288: 여기는 하이볼찐맛집입니다. 꼭 와서 마셔보세요 맛있는 안주들 덕분에 하이볼이랑 좋은 페어링이 됐어요 ^^ (

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%98%90%EB%B4%89%EC%9D%B4%ED%86%B5%EB%8B%AD%20%EB%8F%84%EA%B3%A1%EC%A0%90/place/1281659956?c=14.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 또봉이통닭 서울도곡점
업종 카테고리: 치킨,닭강정
별점: 방문자 리뷰 7
방문자 리뷰: 5
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 2


로드된 리뷰 개수: 0, 수집된 리뷰 개수: 0
새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다... (재시도 1/3)
'더보기' 버튼이 더 이상 없습니다.
총 0개의 리뷰를 수집했습니다. (예상 총 리뷰 수: 2)
수집된 리뷰가 없습니다.


In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%9D%B4%EC%B2%9C%EC%8C%80%EB%B0%A5%20%EA%B0%95%EB%82%A8/place/1814776214?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 이천쌀밥
업종 카테고리: 한식
별점: 방문자 리뷰 26
방문자 리뷰: 6
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 18


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 시큼한 김치찌개와 쫀득한 쌀밥! 다시 생각하는 지금 입가에 침이 고이네요. (2024년 10월 8일 화요일) - 작성자: 이승준080 (사진 94)
리뷰 2: 반찬 리필 자유로운 편이고 친절해요 (2024년 9월 26일 목요일) - 작성자: 강남 맛집 탐방러 (사진 281)
리뷰 3: 밥을 솥에 바로해서 주시고 매일매일 반찬이 바뀌는데 가끔 집밥먹고싶을때 가는 곳이에요 (2024년 8월 5일 월요일) - 작성자: 코끼리다리78 (사진 12)
리뷰 4: 우리동네 최고 백반집 입니다 (2024년 7월 5일 금요일) - 작성자: 앙뢰 (사진 127)
리뷰 5: 이천쌀밥, 과식하게 만드는 곳 ㅠㅠ (2024년 7월 10일 수요일) - 작성자: 아름다운날에는 (사진 83)
리뷰 6: 밥맛이 좋고, 청국장, 제육볶음이 맛있어요^^ (2024년 4월 10일 수요일) - 작성자: lso**** (사진 9)
리뷰 7: 가정식백반이라 부담이 없네요 (2024년 5월 16일 목요일) - 작성자: 온새이 (사진 3)
리뷰 8: 좋아요 (2024년 8월 5일 월요일) - 작성자: his3325 (사진 9)
리뷰 9: 맛은 있는데 김치찌개 안에 고기가 얇고 적게 들어있어요 만원이나 하는데... (2024년 4월 5일 금요일) - 작성자: remu87 (사진 11)
리뷰 10: 좋아요 (2024년 9월 19일 목요일) - 작성자: 신복실 (Unknown)
로드된 리뷰 개수: 18, 수집된 리뷰 개수: 10
리뷰 11: 가까워요 (2024년 7월 24일 수요일) - 작성자: 김귀임 교원맘 (사진 35)
리뷰 12: 반찬 너무 깔끔하고 좋아요
밥이 진짜 맛있어요 최고최고 다시 방문 예정입니다 (2023년 11월 13일 월요일) - 작성자: 니키0313 (사진 252)
리뷰 13: 인심

In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%AA%85%EB%8F%99%ED%95%A0%EB%A8%B8%EB%8B%88%EA%B5%AD%EC%88%98%20%EB%B1%85%EB%B1%85%EC%82%AC%EA%B1%B0%EB%A6%AC%EC%A0%90/place/20642930?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 명동할머니국수 뱅뱅사거리점
업종 카테고리: 국수
별점: 4.03
방문자 리뷰: 198
블로그 리뷰: 8
리뷰 탭 클릭 완료
총 리뷰 수: 59


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 식사때를 놓쳐서 혼자식사하러갔는데
국수전문점이지만 오징어볶음밥 주문했는데
오징어도 실하고 맛있었어요
국물대신 나오는 국수도 너무 맛나요 (2024년 10월 29일 화요일) - 작성자: 제주 스텔스 (사진 2,289)
리뷰 2: 엄마손맛♡ 사장님이 아이 데리고 왔다고 다데기 넣은거 안넣은거 따로 주시고 양도 푸짐하게 주시고 아이도 예뻐해주셔서 감사했습니다^^ 넘 맛있게 잘 먹고 갑니다~~!! (2024년 8월 31일 토요일) - 작성자: 토닥맘29 (사진 165)
리뷰 3: 여기도 회사 근처라 가끔 가는데
가성비 괜찮음^~^ (2024년 7월 31일 수요일) - 작성자: 베야로드 (사진 333)
리뷰 4: 굿 (2024년 10월 24일 목요일) - 작성자: gol**** (사진 11)
리뷰 5: 양재에 이정도 음식 양에 가격이라니,, 놀랐습니다.. 보통 만원이상인데 말이에요
배 터지는 줄 알았습니다.. 국수양이 엄청 마나요
비빔국수 아니 사람 입맛돌게 합니다 이녀석.. 그리고 나름 식단을 챙기신다면 ‘두부국수’ 라는 녀석 추천합니다. 일단 멸치육수 베이스인데 크.. 진짜 그 국물의 깊이가.. 미칩니다. 같이 간 지인이 전날 술 옴총 마셨는데 그 국물로 해장했습니다 ㅋㅋㅋㅋ 담엔 다른 메뉴 먹으러 올라고용!! 벌써 단골분들이 몇분 계속 오시는거 같더라구여!! 아주머니도 너무 친근하시구 음식 만드실때 흥얼 노래 부르시는데 저까지 왠지 흥얼되게 되는 ㅎㅎㅎ 즐겁게 만들어주시니 맛도 더 있는 기분입니다😊🌸 봄날같은 음식점이었어요!! (2024년 4월 12일 금요일) - 작성자: 루피다히 (사진 225)
리뷰 6: 맛있는 국수 잘먹었어요 (2024년 8월 12일 월요일) - 작성자: 앙뢰 (사진 127)
리뷰 7: 잘 먹었습니다 (2024년 7월 17일 

In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%82%99%EC%97%AC%EC%82%BC/place/40774486?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 낙여삼
업종 카테고리: 낙지요리
별점: 4.12
방문자 리뷰: 181
블로그 리뷰: 31
리뷰 탭 클릭 완료
총 리뷰 수: 63


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 이곳은 기본적인 반찬이 매우 훌륭하게 나오는 좋은 식당입니다. 점심저녁 다 좋은데 특히 저녁 삼겹살의 맛과 파/콩나물 무침은 너무 훌륭합니다. 특히 오이소박이도 요즘 빼놓을 수 없는 반찬이네요. 사장님 너무 친절하시고 안에서 음식 조리해주시는 어머님도 손맛 그대로 느껴집니다. 무조건 추천!!! (2024년 7월 2일 화요일) - 작성자: 파워배추 (사진 1,515)
리뷰 2: 저녁에 먹는 삼겹살의 맛이 너무 좋아요. 같이 나오는 반찬이 어머니의 반찬같이 익숙해요~~ㅋ 고소하고 짠맛도 맛있어요 ㅎㅎ (2024년 6월 18일 화요일) - 작성자: 파워배추 (사진 1,515)
리뷰 3: 음식이 정말 맛있고, 반찬도 잘 나옵니다. (2024년 10월 2일 수요일) - 작성자: Jinny Kim (사진 178)
리뷰 4: 맛있어요 (2024년 9월 24일 화요일) - 작성자: mic**** (사진 350)
리뷰 5: 점심밥도 너무 좋아요ㅋㅋ 집밥 먹은것처럼 든든합니다~~~ (2024년 5월 29일 수요일) - 작성자: 파워배추 (사진 1,515)
리뷰 6: 언제나 맛있는 낙여삼입니다. 반찬. 메인메뉴 빼놓을게 없네요~!!! (2024년 5월 3일 금요일) - 작성자: 배산임수85 (사진 485)
리뷰 7: 오랫만 방문 (2024년 8월 28일 수요일) - 작성자: 김귀임 교원맘 (사진 35)
리뷰 8: 사장님은 언제나 친절하시고 점심/저녁 전부 맛있어요. 점심은 반찬도 잘나오고 매일 다른 구성으로 메뉴가 나옵니다. ㅎ (2024년 1월 3일 수요일) - 작성자: 파워배추 (사진 1,515)
리뷰 9: 이곳은 점심밥도 맛있고 저녁 삼겹살도 완전히 로컬맛입니다. 반찬구성도 좋고 점심메뉴도 끝내줍니다. 1만원의 메뉴로 구성되어있는데 비싼느낌보다는 든든히 잘먹었다 생각이 드

In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%8B%A4%EB%8F%88%EC%8B%9D%EB%8B%B9/place/1765708831?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 다돈식당
업종 카테고리: 돼지고기구이
별점: 방문자 리뷰 630
방문자 리뷰: 208
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 359


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 점심에 돈까스 먹으러 종종 왔는데 너무 맛잇게 먹었던 곳이라서 저녁엔 삼겹살도 판매한다고 하셔서, 직장 동료분들이랑 저녁에 꼭 한번 오자고 약속해서 오늘왔어요! 고기도 너무 부드럽고, 육즙도 팡팡 터지고, 직원분들이 친절히 하나하나 다꾸워주셔서 너무 편하게 먹을수 있엇던거 같아요! 잘 먹었습니다! (2024년 9월 3일 화요일) - 작성자: Ysoo96 (사진 5)
리뷰 2: 가격은 올랐지만 음식의 질은 떨어짐 (2024년 11월 6일 수요일) - 작성자: 분노의질주77 (사진 30)
리뷰 3: 오랜만의 치즈롤까스
맛이 좋구요!
양도 많구요!
친절하시고!
그래서 웨이팅을 해야합니다.
하지만 먹고나면, 그럴만 하구나 하고 납득이 되는 집입니다. (2024년 7월 1일 월요일) - 작성자: idisjien94 (사진 1,088)
리뷰 4: 돈까스가 생각나 방문한 다돈식당!
경양카츠가 있어 시켜보았는데 양송이와 함께 버무려진 소스가 맛있었어요 !
돈까스 두께도 두꺼워 먹는식감이 좋았습니다
이거 당분간은 경양카츠만 찾게될것같아요 ㅎㅎ (2024년 9월 2일 월요일) - 작성자: 로드니보데즈 (사진 5)
리뷰 5: 경양카츠로 주문해서 먹었는데
고기가 두껍고 소스가 맛있어요~ (2024년 10월 11일 금요일) - 작성자: eatandplay (사진 675)
리뷰 6: 처음에 고기를 구워주시는데
잘먹을수있게 구워주셔서 편하게 먹을수있었습니다
생각보다 가게도 넓어서 단체로 와도 맛있게 먹고 갈수 있을거같아요 (2024년 9월 3일 화요일) - 작성자: 죽기살기울기 (사진 1)
리뷰 7: 바쁜 점심 시간, 만족스러운 식사를 원하는 사람에게 "다돈식당"을 추천합니다. 빠른 속도로 조리된 돈까스는 겉은 야무지게 바삭하고, 속은 고소한 풍미로 점심 시간을 풍요

리뷰 76: 자리도 넓고 고기도 맛있습니다
트러플 소금에 목살 찍어먹는게 특히 맛있어요!!
고기도 직접 다 구워주셔서 편하게 먹을 수 있어요~ (2023년 6월 20일 화요일) - 작성자: 건덕후72 (사진 2)
리뷰 77: 굿 (2024년 3월 6일 수요일) - 작성자: 김귀임 교원맘 (사진 35)
리뷰 78: 쩔어요 (2023년 9월 22일 금요일) - 작성자: 분노의질주77 (사진 30)
리뷰 79: 다돈식당은 점심시간에 돈까스가 메인입니다
돈까스 전문점 보다 더 좋아요
모듬 돈까스 12,000원
새우까스도 꼭 먹어야 합니다
냄새 안나고 고기질과 튀김옷 모두 훌륭해요
다시 방문 할께요 (2023년 4월 12일 수요일) - 작성자: 남아음악 (사진 231)
리뷰 80: 정말 너무 맛있어요... 된찌는 미쳤다!! (2023년 6월 29일 목요일) - 작성자: millankaka74 (사진 18)
로드된 리뷰 개수: 90, 수집된 리뷰 개수: 80
리뷰 81: 매번 퇴근하고 방문하는 곳입니다. ^^
직원분들도 친절하시고 가성비 쵝오에요
돈까스도 맛있어용 😀😀 (2023년 6월 28일 수요일) - 작성자: 긔염둥이23 (사진 2)
리뷰 82: 양재에서 가장 맛있는 돈까스를 파는 곳입니다. 양도 많고 육질이 대박이에요 (2023년 5월 18일 목요일) - 작성자: 은박4 (사진 157)
리뷰 83: 굳 (2023년 9월 25일 월요일) - 작성자: 다나유 (사진 104)
리뷰 84: 갈매기살 무조건 드세요!!!!
부드럽고 입에서 살살 녹습니다 👍👍 (2023년 8월 2일 수요일) - 작성자: qscfthmkol (사진 11)
리뷰 85: 다소 날씨가 추운 날 얼큰한 국물과 고기가 생각나서 발길이 닿는 곳은 도까스와 돼지고기 구이 맛집인 뱅뱅사거리 다돈식당으로 ~~
이른 시각에 벌써 손님들로 가득했다.
맛도 좋지만 가성비가 으뜸으로 매니아들에게 익히 소문난 그대로~~

직원이 직접 구워준 통삼겹살, 400시간 숙성해서 너무 부드럽고 맛있어요

상로스 돈까스도 풍부한

리뷰 160: 겨울에만 먹을 수 있는 김치동가스 ㅠ 여름에는 또 모밀세트 먹어줘야됨. (2022년 12월 6일 화요일) - 작성자: 7409 (사진 518)
로드된 리뷰 개수: 170, 수집된 리뷰 개수: 160
리뷰 161: 점심때 자주 가는데 맛있습니다 (2022년 12월 15일 목요일) - 작성자: 콩치빠 (사진 13)
리뷰 162: 상로스세트 돈까스 맛있네요 ! 직원들 친절하고 매장이 넓고 쾌적합니다. (2022년 10월 4일 화요일) - 작성자: 쉥필귀정Agen (사진 101)
리뷰 163: 레몬소금과 겨자소스 진짜 넘 잘어울리고 고기는 겉바속촉 진짜 넘 맛나요>_< (2022년 12월 9일 금요일) - 작성자: 쏘쏘2226 (사진 37)
리뷰 164: 목살도 맛있고
된장찌개, 파김치고 굿입니다 (2022년 12월 6일 화요일) - 작성자: 맛동산4762 (사진 532)
리뷰 165: 고기집인데 돈까스만 계속 먹으러 가네요.
몇 가지 먹어봤는데 상로스까스가 제일 맛있네요. 김치나베도 좋구요. (2023년 1월 11일 수요일) - 작성자: 데커드 (사진 21)
리뷰 166: 주변에 돈가스집이 많아서 하나씩 뿌시고 있는데 제일 자주 가는 곳입니다 맛있어요
양이 많아요 (2022년 11월 8일 화요일) - 작성자: 냥냥쓰1 (사진 28)
리뷰 167: 고기집인데 돈까스가 주력이 된 것 같아요. 양도 많고 돈까스는 전문점 이상의 퀄리티입니다. (2023년 1월 4일 수요일) - 작성자: 데커드 (사진 21)
리뷰 168: 바삭 바삭한 ~~
육즙이 입에서 녹아요 ~~

주말을 목전에 둔 금요일, 드디어
점심시간에는 접하는(저녁은 다수 왔음) 뱅뱅사거리 돈까스 맛집 다돈식당에서 상로스를 너무 맛있게 먹었네요

일행과 함께 돼지 특수부위로 만든 상로스와 소바세트로 불금을 화려하게 장식하네요

식당이 넓고 편안하며 직원들도 친절하네요 ~~ (2022년 7월 8일 금요일) - 작성자: 쉥필귀정Agen (사진 101)
리뷰 169: 굳 (2023년 3월 15일 수요일)

로드된 리뷰 개수: 270, 수집된 리뷰 개수: 259
리뷰 260: 좋아요 (2022년 2월 14일 월요일) - 작성자: ey43 (사진 6)
리뷰 261: 점심에는 처음 와보는데 돈까스도 참 맛있네요~ 대기하는 사람들도 많고 맛집인가봐요:) (2021년 7월 29일 목요일) - 작성자: 리아4140 (사진 78)
리뷰 262: 맛있어요ㅎㅎ (2021년 10월 19일 화요일) - 작성자: 규현레이몬드 (사진 517)
리뷰 263: 모듬에 새우가 덜익은건지.. 잘모르겟지만… 그외엔 갠찮아요! (2021년 10월 18일 월요일) - 작성자: 뚱보55 (사진 5)
리뷰 264:  (2021년 10월 13일 수요일) - 작성자: 마성녀 (사진 866)
리뷰 265: 굿 (2022년 1월 3일 월요일) - 작성자: 천재5868 (사진 7)
리뷰 266: 돈가스맛있어요 (2021년 11월 5일 금요일) - 작성자: 김은미2656 (사진 2)
리뷰 267: 양재 돈까스 맛집입니다 (2021년 9월 16일 목요일) - 작성자: 마성녀 (사진 866)
리뷰 268: 양재에서 가장 맛있음 (2021년 8월 9일 월요일) - 작성자: 마성녀 (사진 866)
리뷰 269: 회사 근처 식당.
고기가 부드럽긴한데 맛은 그럭저럭.
아쉽넹. ㅠㅠ (2021년 7월 29일 목요일) - 작성자: yoozero30 (사진 183)
로드된 리뷰 개수: 280, 수집된 리뷰 개수: 269
리뷰 270: 돈까스 배터짐…! (2021년 10월 8일 금요일) - 작성자: bur**** (사진 6)
리뷰 271: 음식이친절하고 사장님이맛있어요! (2021년 9월 13일 월요일) - 작성자: 97**** (사진 7)
리뷰 272: 부드러운 히레 돈까스 졸맛 (2021년 7월 6일 화요일) - 작성자: 하코리타 (사진 35)
리뷰 273: 늦게가면 줄이 길어 기다려야하지만 회전율이 빠르고 주문을 미리 할수 있어서 점심시간에 딱 맞춰 먹을수 있었고 직원분들이 매우 친절하세요 (2021년 7월 7일 수요일) 

'더보기' 버튼이 더 이상 없습니다.
총 358개의 리뷰를 수집했습니다. (예상 총 리뷰 수: 359)
리뷰 데이터가 '다돈식당_reviews.csv' 파일로 저장되었습니다.


In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%98%88%EB%8B%B9%20%EB%8F%84%EA%B3%A1%EB%8F%99/place/18855701?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 예당
업종 카테고리: 한식
별점: 4.3
방문자 리뷰: 331
블로그 리뷰: 75
리뷰 탭 클릭 완료
총 리뷰 수: 120


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 저번에 김치수제비 먹고 맛있어서 이번엔 만두전골 먹으러 다녀왔어요 :) 만두전골인데 약간 샤브샤브 느낌이에요. 한우가 들어가서 그런지 1인분에 2만원이라 가격은 조금 있는데 수제비, 칼국수, 만두 들어가서 맛있고 든든합니다 👍 그리고 사장님이 친절하세용~~! 요즘같이 추운 날 먹기 좋은 것 같아용 (2024년 11월 6일 수요일) - 작성자: 행복한 빛블리양 (사진 1,365)
리뷰 2: 내년이면 30년이 되는 굉장한 업력의 식당이에요ㅎㅎ 이 동네 사람들은 거의 모르는 분은 없더라구요!!수제손만두는 못 참잖아요 그래서 주문한 만두전골!!! 그냥 몸이 싸악 풀리면서 자연스럽게 소주를 주문했네요… 진짜 국물을 들고 설거지해버린 맛집이였어요! 반찬도 직접 만드시고 같이 간 일행은 김치전골도 굉장히 맛있대요ㅋㅋㅋ 그리고 콩국수는 직접 멧돌에 갈아서 하신다니 다음 방문은 콩국수를 먹으러!!! (2024년 6월 23일 일요일) - 작성자: 란쫑 (사진 119)
리뷰 3: 맛있어요
음식도 깔끔하고 (2024년 8월 1일 목요일) - 작성자: 앙뢰 (사진 127)
리뷰 4: 굿 (2024년 10월 15일 화요일) - 작성자: 챨리53 (사진 194)
리뷰 5: 삼겹살도 맛있고 수제비도 맛나구 ,, 손만두전골도 맛나요 !! 점심에 자주먹는 해물칼제비 최고에요
사장님도 직원분도 친절하신 식당입니다 : ) (2024년 2월 29일 목요일) - 작성자: 하이빈나 (사진 727)
리뷰 6: 굿 (2024년 9월 15일 일요일) - 작성자: 챨리53 (사진 194)
리뷰 7: 굿 (2024년 9월 5일 목요일) - 작성자: 챨리53 (사진 194)
리뷰 8: 밑반찬도 고기도 된장찌개도 맛있어요 (2024년 5월 4일 토요일) - 작성자: 칼언니julia97 (사진 351)
리뷰 9: 칼국

리뷰 104: 소고기만두전골... 국물이 얼큰하지만 많이 맵거나 텁텁하지 않고 담백해요. 수제비 칼국수 소고기에 큼지막한 만두까지~ 건더기도 알차고 맛있어요. 밥말아 먹어도 굿! 해장에도 아주 좋을듯~~~ (2020년 5월 21일 목요일) - 작성자: moodin (사진 286)
리뷰 105: 칼국수 양이 너무 적었어요ㅠㅠ 콩나물보다 더 적음
해물 푸짐하고 맛도 있는데 칼제비가 메인인데 칼제비는 진짜 적음.. 놀랄 정도로 적음 (2021년 1월 28일 목요일) - 작성자: 33**** (사진 13)
리뷰 106: 만두전골 맛있어요 (2020년 8월 11일 화요일) - 작성자: 파파야86 (사진 1)
리뷰 107: 이 집 만두 진짜 맛있어요 (2020년 5월 27일 수요일) - 작성자: 웬디네취향 (사진 477)
리뷰 108: 김치 손만두 전골 맛있어요! (2020년 5월 12일 화요일) - 작성자: 몬테수지 (사진 16)
리뷰 109: 만두전골 맛있어요 (2020년 7월 3일 금요일) - 작성자: 나무나라83 (사진 17)
리뷰 110: 맛은 있는데 자리가 좁아요 (2020년 7월 30일 목요일) - 작성자: 3327450 (사진 110)
로드된 리뷰 개수: 120, 수집된 리뷰 개수: 110
리뷰 111: 맛있어요~ (2020년 7월 30일 목요일) - 작성자: 투보이스 (사진 13)
리뷰 112: . (2020년 6월 30일 화요일) - 작성자: namwoog (사진 1)
리뷰 113: 손만두 좋아요 (2020년 5월 14일 목요일) - 작성자: poo**** (사진 7)
리뷰 114: 맛있어요 (2020년 8월 9일 일요일) - 작성자: eun**** (사진 165)
리뷰 115: 굿 (2020년 8월 2일 일요일) - 작성자: eun**** (사진 165)
리뷰 116: 친절해요 (2020년 5월 14일 목요일) - 작성자: yoomee75 (사진 14)
리뷰 117: 굿 (2020년 3월 26일 목요일) - 작성자: iam**** (사진 8)
리뷰 1

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%20%EC%B0%B8%EC%A1%B1/place/38226320?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 양재족발 참족
업종 카테고리: 족발,보쌈
별점: 4.78
방문자 리뷰: 2,398
블로그 리뷰: 2,153
리뷰 탭 클릭 완료
총 리뷰 수: 2136


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 올만에 족발이 땡겨서 양재역 근처 참족 방문했는데
여섯시 전인데도 벌써 만석이었어요!
좀만 더 늦었으면 맛난 족발 못 먹을 뻔 했네요
고기 누린내 민감한편인데 전혀 느껴지지 않았고
쫄깃하고 탱글하니 넘 맛있어요
기본맛도 고기 고소하고 부드러워 맛있는데
매운족발 매콤하니 계속 땡기는 맛이에요
서비스로 나오는 꽃게탕 얼큰하구 시원하니
술안주가 따로 없어요 해물 왕창 들어갔는데 서비스라니
족발 양도 푸짐하구 구성도 넘 좋구 대만족이에요 (2024년 11월 1일 금요일) - 작성자: 맛집탐방가점민 (사진 150)
리뷰 2: 강남 맛집 참족에 처음 방문 해봤는데 족발 명인이 직접 삶는 족발로 이미 유명한 맛집이더라구요😆
특히 국내 최초 갈비 족발을 선보인 집답게, 갈비 양념의 깊은 풍미와 고기의 부드러운 식감이 인상적입니다. 한입 먹을 때마다 야들야들한 족발이 입안을 가득 채워, 정말 정성껏 준비된 음식이라는 느낌👍👍
족발과 함께 나오는 주먹밥과 막국수도 족발과 굉장히 잘어울리고 그 맛을 한층 더해주어, 족발을 다 먹을 때까지 계속 손이 가게 됩니다🤭
무엇보다 참족은 음식뿐만 아니라 서비스도 특별합니다. 족발을 주문하면 서비스로 제공되는 꽃게탕은 시원하고 깊은 국물 맛으로, 술 안주로도 정말 좋습니다. 이렇게 음식 하나하나에 정성과 신경이 가득 담긴 곳이라, 맛도 분위기도 만족스러운 식사를 할 수 있었습니다. 양재 맛집 인정! (2024년 9월 26일 목요일) - 작성자: kyu**** (사진 962)
리뷰 3: 양재 찐맛집 참족에 다녀왔습니당~ 족발이 땡기던 요즘 양재역 근처에 이런 맛집이 있는줄 몰랐네요ㅎㅎ 주말에 예약하고 방문했는데 거의 만석이더라고요😃 저는 반반족발하고 영혼의 단짝 막국수를 시켰는데 일단 반반족발은 기본하고 매운 족발하고 

리뷰 45: 끝내주게 맛있습니다. 가성비 미쳤는데 가성비 수준이 아니라 넘 고급스럽게 맛나요 (2024년 10월 15일 화요일) - 작성자: 넥슈어 (사진 67)
리뷰 46: 보기도 좋고
꽃게탕까지 ~
족발 양념 달고짜게 자극적이지도 않은데 맛있네요
살도 부드러워요
꽤 맛있어요^^ (2024년 11월 4일 월요일) - 작성자: jiy**** (사진 10)
리뷰 47: 몇년째 오는 족발집인데 맛도 변함없고 요즘 사람이 더 많아진듯요😭 저만 알고 싶은 곳인데.... 아숩 (2024년 10월 25일 금요일) - 작성자: bonnie68 (사진 25)
리뷰 48: 3년전부터 꾸준히 족발먹고싶으면 꼭 방문하는 곳이에요!!! 인기가 좋아서 혹시나 자리가 없을까 꼭 예약하고 오는데 올때마다 같이 오는 사람들마다 다 맛있다고 하고 좋아해서 실패없는 맛집이에요!!!
사이드로 해물탕이 나오는데 진짜 이걸로도 소주 한병 완전 가능하고 꼭 반반 시켜서 먹는거 추천합니당 (2024년 10월 28일 월요일) - 작성자: wdm**** (사진 5)
리뷰 49: 사장님께서 너무 친절하시구, 친절하다고 맛을 뺄수는없죠 진짜 입맛 까다로운편인데 먹는 마지막까지 웃으면서 나왔어요! 다시한번 감사드리고, 다른분들도 소중한 사람들과 소중한 시간 보내기에 적합한 맛집인거같아요!😀 (2024년 9월 13일 금요일) - 작성자: dc**** (사진 5)
리뷰 50: 족발 너무 맛있고 사이드메뉴도 하나같이 맛있네요~ 가족들끼리왔는데 모두 맛있게 잘 먹고갑니다 ^^ (2024년 10월 19일 토요일) - 작성자: 쏘굿12 (사진 25)
로드된 리뷰 개수: 60, 수집된 리뷰 개수: 50
리뷰 51: 이번이 4번째 방문이에요!!!
오늘은 예약없이 5시40분에 들어왔는데
조금만 늦었으면 못 먹을 뻔 했습니다
항상 세트로 시키는데 메뉴 구성도 너무 좋고
족발이.진짜 야들야들한게 맛있어요~~~ (2024년 10월 31일 목요일) - 작성자: Dori3871 (사진 53)
리뷰 52: 두달전 예약없이 왔다가 먹지못

리뷰 118: 세트가 구성이 좋고 단체 모임하기 좋아요!
족발도 맛있습니다!!! (2024년 11월 1일 금요일) - 작성자: 감자5188 (사진 10)
리뷰 119: 양재에서 족발이 너무 먹고싶어서 예약하고 왔는데 진짜 맛있네요..
그리고 양이 너무너무 많아서 배가 터질거 같이 많이 먹고 갑니다!! (2024년 11월 1일 금요일) - 작성자: PILLAR755 (사진 11)
리뷰 120: 고기 진짜 맛있아요
방송
방영된게 찐입니다

감자전은 꼭!! 필수
족발생각나면 또올것같아여 ㅎㅎ (2024년 10월 14일 월요일) - 작성자: 이경원4 (사진 10)
로드된 리뷰 개수: 130, 수집된 리뷰 개수: 120
리뷰 121: 남자친구랑 족발 먹구 싶어서 찾아 보던중 맛집이라 나와 있어서 먹었는데 예약으로 운영 되시는 것 같더라구여 그래도 받아 주셔서 감사 했습니다!! 맛도 진짜 맛있게 먹었어요 고기 잡내 없이 양도 엄청 푸짐해서 배터지도록 먹었습니닼ㅋㅋㅋㅋㅋ나중에 예약하고 또 오고 싶습니다😆🫠 (2024년 8월 13일 화요일) - 작성자: 바캉스가쟈 (사진 9)
리뷰 122: 너모 맛있고 분위기도 좋고 서비스도 좋아요!
모임하기 딱 인것 같아요!! 또올게요 (2024년 8월 30일 금요일) - 작성자: nan**** (사진 305)
리뷰 123: 밀크씨슬에 삼까지 챙겨주시는 사장님의 인심과 고객을 생각하는 마음부터 맛있는 가게라는것을 느끼고 먹으니 더더욱 맛있습니다 !
이 곳을 이제서라도 알아서 다행이라고 생각합니당 (2024년 10월 31일 목요일) - 작성자: 근짱구 (사진 8)
리뷰 124: 너무 맛있게 잘 먹었습니다! 양도 많고 신선하고 맛도 좋아서 다음에 또 오고 싶네요ㅎㅎㅎㅎ
청결한 맛집이라 추천드립니다! (2024년 10월 2일 수요일) - 작성자: leecw4u (사진 18)
리뷰 125: 너무 맛있어서 다 먹었어요 굳 (2024년 10월 30일 수요일) - 작성자: 바티골57 (사진 3)
리뷰 126: 야들야들 맛있습니다!!!👏👏 (2024년 

리뷰 198: 양 진짜 진짜 많고 족발 진짜 부드럽고 맛있어요 엄청 친절하십니다 (2024년 8월 24일 토요일) - 작성자: 킹네원 (사진 350)
리뷰 199: 기대 이상입니다
메뉴가 다 맛있어요 >_<
또 올게요 (2024년 9월 3일 화요일) - 작성자: 망포동포동통 (사진 247)
리뷰 200: 족발 맛이 최고네요~오늘 처음 방문했는데 또 찾게 될 거 같아요^^ (2024년 10월 4일 금요일) - 작성자: vicjihyun2 (사진 27)
로드된 리뷰 개수: 210, 수집된 리뷰 개수: 200
리뷰 201: 청모하러 왔는데 진짜 맛집 발견!!! (2024년 10월 11일 금요일) - 작성자: 망난이66 (사진 8)
리뷰 202: 족발 너무 부드럽고 맛있어요bbb (2024년 11월 12일 화요일) - 작성자: suh36 (사진 6)
리뷰 203: 와 진짜 맛있어요
역시 유명한 맛집입니다
최고 (2024년 10월 28일 월요일) - 작성자: 지혜4222 (사진 22)
리뷰 204: 맛있게 잘 먹었습니다! 소문대로네요 역시~! (2024년 10월 17일 목요일) - 작성자: Alan28 (사진 12)
리뷰 205: 갑자스런 번개로 약속장소로 정했는데 양도 푸짐하고 맛있고 친절하고 재방문할꺼에요!! (2024년 9월 23일 월요일) - 작성자: ddu**** (사진 8)
리뷰 206: 맛있어용!!!!! 반반 먹을수있어서 좋아용!! 세트메뉴 가성비 짱이에요 (2024년 10월 15일 화요일) - 작성자: 웹툰56 (사진 18)
리뷰 207: 혹시 몰라서 예약을 하고 방문 했는데 손님이 엄청 많아서 여기 찐 맛집인가보다 하구있었는디 오랜만에 족발 먹었는데 너무 맛있더라구요 ㅠㅠ

해물탕 서비스로 주신게 진짜 족발이랑 합이 완전 잘 맞아요 ㅠㅠ (2024년 8월 14일 수요일) - 작성자: jejehand (사진 31)
리뷰 208: 와 너무 맛있습니다….
이렇게 맛있는 족발 처음 먹어봐요
갈비족발 추천해요!!! (2024년 8월 14일 수요일) - 작성자: 

로드된 리뷰 개수: 290, 수집된 리뷰 개수: 280
리뷰 281: 회식으로 두번째 방문했어요.
넘 맛있게 잘 먹고 갑니다. (2024년 9월 11일 수요일) - 작성자: 김편집337 (사진 18)
리뷰 282: 너무 맛있게 잘먹었습니다.
다음에 또 방문할게요~ (2024년 11월 5일 화요일) - 작성자: seocho13243 (사진 5)
리뷰 283: 너무 맛있어서. 순싹했네요
족발중에 참족이 젤 맛있고 부드러워요ᆢ
제방문 하겠습니다ᆢ (2024년 10월 8일 화요일) - 작성자: cjd**** (사진 6)
리뷰 284: 오랜만에 왔어요~
역시나👍🏻 양도 푸짐하고 맛도 좋고
기분도 좋아요~ (2024년 10월 29일 화요일) - 작성자: 쯔끄미18 (사진 9)
리뷰 285: 맛도있고 양도 푸짐해요. 친절하게 대해주셔서 기분좋게 먹고갑니다! 다음에 또올게요!!! (2024년 10월 14일 월요일) - 작성자: kyu**** (사진 6)
리뷰 286: 내부 분위기도 좋고 음식이 정말 맛있었습니다 (2024년 10월 17일 목요일) - 작성자: 플래티나93 (사진 12)
리뷰 287: 족발 못먹는 친구와 같이 왔는데 다양한 소스와 밑반찬들 덕분에 맛있게 잘 먹네요

족발 야들야들하고 매운탕은 시원해서 소주없이는 먹을 수 없습니다 😭
전체적으로 음식이 맛있어요 👍 👍 👍 (2024년 8월 15일 목요일) - 작성자: 날군17 (사진 5)
리뷰 288: 소문으로만 듣다가 와봤는데 너무 맛있네요!! (2024년 10월 3일 목요일) - 작성자: 꼬냥8 (사진 37)
리뷰 289: 너무 맛있어용 구성도 너무 좋아요 (2024년 9월 26일 목요일) - 작성자: ala**** (사진 6)
리뷰 290: 족발이 엄청 쫄깃하고 맛있어요 ㅎㅎ 세트 메뉴도 알차고😍 (2024년 9월 27일 금요일) - 작성자: 링링링뽀 (사진 44)
로드된 리뷰 개수: 300, 수집된 리뷰 개수: 290
리뷰 291: 너무맛있어요 ᆢ (2024년 10월 21일 월요일) - 작성자: cjd**

...

로드된 리뷰 개수: 540, 수집된 리뷰 개수: 530
리뷰 531: 지나가다 볼때마다 항상 사람이 많아서 꼭 한번 먹어보고 싶었어요 진짜 맛있네영!! (2024년 9월 13일 금요일) - 작성자: 얀이ㅁ (사진 18)
리뷰 532: 진짜 너무 맛있고 분위기도 좋고 암튼 또 오겠습니다
너무너무 맛있어요 최고👍🏻 (2024년 7월 31일 수요일) - 작성자: Syh21 (사진 9)
리뷰 533: 짱 맛있어요 회식하러 왔는데 좋아요~~! (2024년 7월 17일 수요일) - 작성자: 소블리772 (사진 16)
리뷰 534: 너무너무 맛있고 양도 많고 세트가 좋습니다 (2024년 8월 23일 금요일) - 작성자: 불독9579 (사진 9)
리뷰 535: 단체모임하기 좋습니다.
꼭 예약하고 오셔야 해요!
맛있고 양도 많아요 😃 (2024년 7월 24일 수요일) - 작성자: 욜로812 (사진 13)
리뷰 536: 음식이 깔끔하고 맛있었습니다!! (2024년 9월 24일 화요일) - 작성자: see06212 (사진 1)
리뷰 537: 족발 엄청 야들야들하고 맛있어요~~~ 세트 구성도 좋아서 막국수 주먹밥 같이 먹을수 있어서 좋아요 ㅎㅎㅎ 모임하기 좋은곳!! (2024년 7월 31일 수요일) - 작성자: 누누370 (사진 87)
리뷰 538: 사장님이 친절하시고 엄청 맛있어요! (2024년 8월 19일 월요일) - 작성자: 홍355 (사진 17)
리뷰 539: 자주 오는 맛집이에요. (2024년 10월 25일 금요일) - 작성자: ano**** (사진 1)
리뷰 540: 회식으로 방문했는데 족발이 진짜 부드럽고 맛있습니다 (2024년 8월 21일 수요일) - 작성자: 영구9570 (사진 5)
로드된 리뷰 개수: 550, 수집된 리뷰 개수: 540
리뷰 541: 와우!!참족발 넘 맛나요 또올게요 (2024년 10월 24일 목요일) - 작성자: 2bb**** (Unknown)
리뷰 542: 너무맛있고 직원분들이 친절해요 (2024년 9월 20일 금요일) - 작성자: dud***

리뷰 625: 주차도 편하고 족발 간이 잘 되어있어서 맛있어요!! (2024년 9월 4일 수요일) - 작성자: lil**** (사진 1)
리뷰 626: 족발은 찰지면서 사르르 녹고 보쌈은 야들야들하게 너무 맛있어요! (2024년 9월 4일 수요일) - 작성자: gon**** (사진 2)
리뷰 627: 먼저 넘 친절하세요^^ 족발의 맛남과 꽃게탕 서비스로 녹아내리는 참 진심 ㅋㅋ 사장님 이모님 모두 너무 친절하십니다^^

맛집 인정!! (2024년 6월 15일 토요일) - 작성자: D3 공동환 (사진 4)
리뷰 628: 너무 맛있고 직원분들도 엄청 친절해요!!!!! (2024년 8월 22일 목요일) - 작성자: 안녕7685 (사진 11)
리뷰 629: 존맛 ㅠㅠ 인생 족발집이에여.... 서비스로 나오는 꽃게탕까지 전부 다 맛있고 막국수 꼭 드세요 !!!! 💓 오랜만에 인생맛집 찾아서 기분 좋습니다 ㅎㅎ (2024년 4월 27일 토요일) - 작성자: chanmiya (사진 157)
리뷰 630: 맛있어요!!!ㅎㅎㅎ🤍🤍🥳 (2024년 8월 28일 수요일) - 작성자: 센9650 (사진 19)
로드된 리뷰 개수: 640, 수집된 리뷰 개수: 630
리뷰 631: 양도 많고 구성이 알차요! 고기도 너무 부드러워요!! (2024년 8월 28일 수요일) - 작성자: ㅇㅁㅇ30 (사진 18)
리뷰 632: 소문대로 정~~말 맛있는 참족 이네요~
서비스 찌개도 짱 맛있어요
꼭 가보세요 (2024년 9월 19일 목요일) - 작성자: eun**** (사진 2)
리뷰 633: 너무부드럽고 맛나네요~~ㅎㅎ (2024년 8월 30일 금요일) - 작성자: 여름 나무 (사진 6)
리뷰 634: 등산후 들렀는데 매콤 달콤 담백 맛이 좋아요. 서비스로 해장국까지 시원하게 잘 먹었어요. (2024년 7월 16일 화요일) - 작성자: tru**** (사진 11)
리뷰 635: 너무너무 맛있는 참족 언제와도 만족스럽게 먹고갑니다!.! 재방문 해야지~ (2024년 8월 26일 월요일) - 작성자: 

리뷰 719: 직장동료 추천으로 왔는데 너무 맛있어요 :) (2024년 8월 17일 토요일) - 작성자: li1**** (사진 12)
리뷰 720: 맛있게 먹었습니다. 족발 오랫만에 많이 먹었어요 ㅎ
막국수는 달아요. 여쭤봤더니 원래 소스가 그렇다고하시네요;;
다음에 또 올께요 (2024년 5월 28일 화요일) - 작성자: 쭌쭌89 (사진 27)
로드된 리뷰 개수: 730, 수집된 리뷰 개수: 720
리뷰 721: 완전 맛있구 친절해요!!! 재방문할게요!! (2024년 8월 13일 화요일) - 작성자: 방울35 (사진 7)
리뷰 722: 데이트맛집으로 짱좋아요 (2024년 7월 9일 화요일) - 작성자: yel**** (사진 5)
리뷰 723: 반찬과 서비스로 나오는 탕 너무 맛있었고 족발도 잡내 안 나고 너무 부드러웠어요 ! 오랜만에 폭식하고 갑니다 직원분들도 너무 친절하세요 ! (2024년 7월 15일 월요일) - 작성자: bel**** (사진 4)
리뷰 724: 맛있었습니다! 족발이 실해요 (2024년 8월 16일 금요일) - 작성자: 강원준70 (사진 2)
리뷰 725: 직원분들 친절하고 너무 맛있고 좋아요 (2024년 8월 16일 금요일) - 작성자: 담닝이 (사진 2)
리뷰 726: 진짜 족발중의 왕인듯 너무 맛있어요!! (2024년 5월 31일 금요일) - 작성자: 규현57 (사진 33)
리뷰 727: 친절하시고 항상 맛있어요 (2024년 9월 12일 목요일) - 작성자: ide**** (사진 4)
리뷰 728: 족발 너무 부드럽고 맛있어요!!!강남 족발은 여기입니다ㅎㅎ (2024년 7월 3일 수요일) - 작성자: haaawol (사진 4)
리뷰 729: 족발도 맛있고 서비스로 주신 탕도 너무 맛있습니다!! (2024년 7월 3일 수요일) - 작성자: 맹랑구리 (사진 4)
리뷰 730: 처음 방문인데 너무 맛있어요~ (2024년 7월 3일 수요일) - 작성자: kjk**** (사진 5)
로드된 리뷰 개수: 740, 수집된 리뷰 개수: 730
리뷰 

리뷰 816: 갈비 족발이 특히 맛있었습니다. (2024년 8월 31일 토요일) - 작성자: ksl**** (사진 4)
리뷰 817: 여러가지 재미있는 족발집이네요 친절한 직원들 덕분에 잘 먹었습니다 (2024년 6월 22일 토요일) - 작성자: 진짜바보 (사진 64)
리뷰 818: 친구들과 간만의 모임
세트메뉴가 주먹밥에 막국수까지
저렴하게 잘 먹었습니당^^ (2024년 7월 13일 토요일) - 작성자: 킬러4 (사진 1)
리뷰 819: 음식이 진찌 맛있습니다,
족발이 육질이 부드럽고 윤기가 좋습니다!
특히 갈비 족발이 정말 특이하고 맛있었습다!!!! (2024년 6월 3일 월요일) - 작성자: 유군3164 (사진 3)
리뷰 820: 맛있어요~ 양이 정말 많네요 (2024년 7월 25일 목요일) - 작성자: 현복98 (사진 2)
로드된 리뷰 개수: 830, 수집된 리뷰 개수: 820
리뷰 821: 쫄깃쫄깃하니 너무 맛있어요~~ (2024년 6월 20일 목요일) - 작성자: myn**** (사진 5)
리뷰 822: 함께 온 분들이 너무 맛있다고 하셔서
뿌듯~~ 담에 또 와야겠어요~^^ (2024년 6월 26일 수요일) - 작성자: hwam6981 (사진 6)
리뷰 823: 양재참족에서 회식했는데 직원분들이 친절하셔서 기분좋게 먹고갑니다! 또 오고싶어요! (2024년 6월 11일 화요일) - 작성자: 돼장토마토 (사진 51)
리뷰 824: 청첩모임 왔는데 음식도 맛있고 친절하세요!! (2024년 10월 17일 목요일) - 작성자: Joooooh2 (사진 12)
리뷰 825: 맛있게 잘 먹었습니다🩷 (2024년 8월 29일 목요일) - 작성자: jsu**** (사진 11)
리뷰 826: 진짜 맛있어요! (2024년 8월 29일 목요일) - 작성자: 이남주47 (사진 4)
리뷰 827: 밀크씨슬도 주시고 좋아요 맛있어요~ (2024년 7월 26일 금요일) - 작성자: peonyyy (사진 236)
리뷰 828: 맛잇어요 또 갈거에요~ (2024년 8월 28일 수요일

로드된 리뷰 개수: 920, 수집된 리뷰 개수: 910
리뷰 911:  (2024년 9월 6일 금요일) - 작성자: dlaehdgus23 (사진 11)
리뷰 912: 너무 맛있고 친절하십니다~!! (2024년 8월 9일 금요일) - 작성자: KKJH (사진 3)
리뷰 913: 회식때 족발 먹자하면 항상 오는곳입니다 잘먹고 갑니다 (2024년 8월 9일 금요일) - 작성자: znz**** (사진 4)
리뷰 914: 맛있다고해서 찾아왔는데 진짜 맛이썽요 ㅎㅎㅎㅎ굿굿 또 올거에요!! (2024년 8월 8일 목요일) - 작성자: 졍3733 (사진 13)
리뷰 915: 맛 있 어 요 짱! (2024년 10월 11일 금요일) - 작성자: 보리61 (사진 3)
리뷰 916: 맛이있어요 (2024년 9월 11일 수요일) - 작성자: 잘살자 다같이 (사진 3)
리뷰 917: 맛있어요 (2024년 10월 3일 목요일) - 작성자: pos03025 (사진 1)
리뷰 918: 양도 많고 맛있어요 !!
매뉴도 특별해요!! (2024년 6월 25일 화요일) - 작성자: nis**** (사진 20)
리뷰 919: 지인분들과 맛있게 잘 먹었습니다.
옆 테이블 고객분들 목소리가 커서 대화하기 불편했습니다.
다음엔 주인분을 통해서 불편함을 말씀드려야겠어요. (2024년 7월 15일 월요일) - 작성자: hwam6981 (사진 6)
리뷰 920:  (2024년 9월 5일 목요일) - 작성자: BKKEEM (사진 10)
로드된 리뷰 개수: 930, 수집된 리뷰 개수: 920
리뷰 921: 참족에는 특별한 맛이 있어요 (2024년 6월 3일 월요일) - 작성자: jin**** (사진 8)
리뷰 922: 너무 맛있고 양도 많고 서비스도 좋습니다 (2024년 7월 5일 금요일) - 작성자: 1awa3a4a (사진 3)
리뷰 923: 맛도리 맛도리 입니다! 야들야들 JMT 🐷🐷🐷🐷🎀✨🩵 (2024년 6월 27일 목요일) - 작성자: Cetacea88 (사진 16)
리뷰 924: 맛있네용 (2024년 9

In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%20%EC%95%84%EC%9B%8C948/place/772273676?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: OUR948
업종 카테고리: 카페,디저트
별점: 방문자 리뷰 98
방문자 리뷰: 5
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 40


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 깨끗하고 친절하시고
대화하기 편합니다~ (2024년 7월 4일 목요일) - 작성자: 양배추dear맘 (사진 536)
리뷰 2: 친절하세요 (2024년 7월 4일 목요일) - 작성자: 쩡2867 (사진 209)
리뷰 3:  (2024년 9월 11일 수요일) - 작성자: 황금9477 (사진 169)
리뷰 4: 가성비 좋은 곳이라 점심식사 후 직장동료들과 항상 방문하는 곳입니다.
점심식사 시간에 가면 항상 사람 많더라구요~ (2024년 3월 13일 수요일) - 작성자: KKOB (사진 71)
리뷰 5: 커피 가격 저렴하고 맛있습니다. (2024년 4월 7일 일요일) - 작성자: 지켜봄으로51 (사진 17)
리뷰 6: 굿 (2024년 6월 12일 수요일) - 작성자: soon33 (Unknown)
리뷰 7: 굿 (2024년 3월 15일 금요일) - 작성자: gam**** (Unknown)
리뷰 8: 굿 (2024년 5월 21일 화요일) - 작성자: 루카엔 (사진 1)
리뷰 9: 굿 (2024년 4월 24일 수요일) - 작성자: 루카엔 (사진 1)
리뷰 10:  (2024년 3월 15일 금요일) - 작성자: 분양신5094 (사진 102)
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 굿 (2024년 4월 2일 화요일) - 작성자: 루카엔 (사진 1)
리뷰 12: 커피 마시기 넓고 좋아요 (2023년 5월 24일 수요일) - 작성자: sou**** (사진 42)
리뷰 13: 여기는 감기차가 맛있습니다. (2023년 9월 11일 월요일) - 작성자: Jinny Kim (사진 178)
리뷰 14: 비올때 기분이 좋네여 (2023년 4월 25일 화요일) - 작성자: 유리시아나68 (사진 22)
리뷰 15: 매장이 넓어요 가격이 저렴한 편이라 

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%20%EB%AC%B4%ED%99%94%EC%9E%A0%20%EA%B0%95%EB%82%A8%EC%A0%90/place/18783735?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 무화잠
업종 카테고리: 게요리
별점: 4.35
방문자 리뷰: 750
블로그 리뷰: 467
리뷰 탭 클릭 완료
총 리뷰 수: 569


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 사장님(?), 매니저님(?)께서 친절히 코스 가이드 해주셨고 킹크랩 코스로 선택했어요. 킹크랩 퀄리티에 충분히 만족하구요. 직원분들도 친절하셨고 볶음밥과 매운탕 퀄리티에도 충분히 만족했습니다. 여친과 즐거운 시간 보낼 수 있어 좋았어요. 돈만 많이 벌면 되겠습니다 ㅎㅎ (2024년 8월 26일 월요일) - 작성자: 삼돌22 (사진 70)
리뷰 2: 이름값 못하네요 다소 저렴한 주말코스라 그런건가요? 내용이 부실하고 특히 메인인 대게…
활대게 바로 찐게 맞는지 의심스러울정도로 단맛은 전혀없고 죽은대게에서 나는 짠맛만 남아있어요
살도 푸석하고 먹는내내 기분이 별로 안좋을정도
그리고 이런곳은 프라이빗한 공간과 직원응대의 서비스 비용도 포함되어 다른곳보다 높은 금액을 지불하고 편하게 먹으려고 가는겁니다
직원분 응대교육 받으셔야 합니다 먹는내내 눈치가 보이고 불편하네요 2인 286,000원이면 적게 먹고 간것더 아닌데 말이죠 (2024년 9월 7일 토요일) - 작성자: schmoe (사진 78)
리뷰 3: 신랑생일기념해서 예약방문했어요. 늘 느끼는거지만 직원분들모두 친절하시고 대접받는.느낌들어서 늘 기분좋게 식사하고 오는.단골집입니다. 매니저님 께서 센스있는 생일밥상과 주임님의 이날서비스로 우리부부 행복한 저녁시간 보냈네요. 주차관리해주시는 우리선생님두 감사합니다 맛난 대게먹고 몸보신한듯 건강한 느낌이예요❤️ (2024년 6월 25일 화요일) - 작성자: 박엘리사벳 (사진 104)
리뷰 4: 중요한날, 손님모시고 식사 할때 늘 먼저 찾게됩니다 조용하고 음식맛도 깔끔합니다 (2024년 8월 9일 금요일) - 작성자: 책방주인하고파 (사진 41)
리뷰 5: 몇년이 흘러도 반찬이 동일한거 같아요. 먹을만한 맛있는 반찬으로 메뉴구성 변경이 필요한거 같고 간장게장의 경우는 

로드된 리뷰 개수: 70, 수집된 리뷰 개수: 60
리뷰 61: 오늘도 맛있는 보리굴비정식
푸짐하고 맛있게 잘 먹었습니다. (2023년 9월 22일 금요일) - 작성자: 피요나슈렉 (사진 975)
리뷰 62: 언제나 믿고먹는 무화잠
대게 사시미는 굿입니다. (2023년 11월 15일 수요일) - 작성자: ydy**** (사진 9)
리뷰 63: 가족식사하러 방문했고 코스로 62만원 결제했고 , 대게코스이고 회랑 전복 너무 신선!
친절하시고 코스 중간중간에 서비스 잘해주셔서
전부 맛있게 식사하며 즐거운 분위기였어요💕
대게를 계속 따뜻하게 먹을수 있는점도 좋았구요.

재방문 의사 100000% (2023년 7월 2일 일요일) - 작성자: AJ330 (사진 38)
리뷰 64: 옛날에 한번 가보고 오랜만에 생각나서 1:30 예약하고 다녀왔습니다 조금더 여유롭게 가는게 좋을거같습니다
주말가족특선먹었는데 가성비 좋고 룸으로 되서 조용히 먹기 좋았습니다 설명을 조금 더 해주시면 좋을거같긴한데 아무래도 촉박하게가서 그런거같았고 모두 친절하셨습니다 (2023년 8월 27일 일요일) - 작성자: 메모장 (사진 6)
리뷰 65: 맛있어요 (2023년 12월 14일 목요일) - 작성자: 당근305999 (사진 209)
리뷰 66: 오늘도 보리굴비정식 맛있게 먹었어요.
다음에도 꼭 바싹 구워달라고 요청해야겠어요.
갠적으로 바싹 구운게 전 맛있더라구요. (2023년 7월 26일 수요일) - 작성자: 피요나슈렉 (사진 975)
리뷰 67: 대게 킹크랩 재료가 비싸긴하지만 중요한 손님 모시기에 좋은장소임 맛도 좋구 서비스도 친절하고 주차하기 편해서 넘 좋습니다
신경쓰는 손님 모시기에 좋은장소로 강추합니다^~^ (2023년 8월 25일 금요일) - 작성자: sm**** (Unknown)
리뷰 68: 너무 맛있고 게사시미를 정말 좋아해요.
게다리 스테이크도 맛있습니다. (2023년 5월 20일 토요일) - 작성자: wonhee8 (사진 44)
리뷰 69: 초밥은 평일 점심만 가능하며 전화로 예

리뷰 147: 버터대게구이는 말이필요없죠
너무 맛있어요 (2022년 4월 25일 월요일) - 작성자: BoNNie (사진 886)
리뷰 148: 지난번 4인 모임로 식사했던 룸으로 요청했는데
이미 예약된 룸이라 하셔서..아쉬웠어요.

그런데 당일에 생각지도 못했던
그 룸으로 안내해주셔서
너무 감사했습니다.
(조정을 해주신건지 섬세한 배려 감동했어요) (2021년 10월 27일 수요일) - 작성자: J ar (사진 44)
리뷰 149: 항상 깔끔하고 맛있네요 (2022년 6월 7일 화요일) - 작성자: hl**** (사진 13)
리뷰 150: 보리굴비정식 먹는데 오늘 유난히 연두부샐러드에 기름쩐내가 너무 심함. 제발 기름좀 자주 갈아 튀겨주세요. (2022년 3월 16일 수요일) - 작성자: 피요나슈렉 (사진 975)
로드된 리뷰 개수: 160, 수집된 리뷰 개수: 150
리뷰 151: 3번째 방문 맛은 물론이고 항상 친절하셔서 기분 좋은 식사하고 오네요~ (2022년 3월 16일 수요일) - 작성자: 썬55 (사진 6)
리뷰 152: 엄마 생신날 방문 했어요! 생신 전주에 파티를 따로해서 당일 저녁 급하게 부모님댁 근처에 룸으로 된 곳을 찾다가 방문하였습니다.🎂
킹크랩 정말 최고였구욤 💗 회는 물론 문어숙회, 튀김류, 알밥, 꽃게탕 등등 음식 다 맛있었습니다!
또 생신이라고 회케이크에 초도 꼽아주시고, 함께 생신축하해주셔서 너무 감사드려요🥰 덕분에 좋은시간 보낼 수 있었습니다!
룸 온도조절도 잘해주시고 적절한 서빙속도와 친절에 부모님이 만족하셔서 기분 좋네용 💛
다음에 또 방문할게요! 감사합니다 ~~ 😍 (2021년 11월 8일 월요일) - 작성자: pinkpink11 (사진 15)
리뷰 153: 맛있는데 서빙 주실때 노크 주셨으면 (2022년 8월 19일 금요일) - 작성자: jus**** (사진 25)
리뷰 154: 부모님 모시고 식사했는데 좋아하셔요 (2022년 6월 4일 토요일) - 작성자: 2580ls (사진 23)
리뷰 155: 맛있어요. (2022

리뷰 237: 좋아요 (2021년 6월 1일 화요일) - 작성자: 율라30 (사진 14)
리뷰 238: 괜찮습니다 (2021년 6월 26일 토요일) - 작성자: SuperPower (사진 1)
리뷰 239: 점심특선 다 맛있어요 대게 전문점이라 당연 대게는 물론 대구매운탕이랑 보리굴비 너무 맛있어서 또 먹고 싶네요 다들 엄청 친절하시고 음식도 빨리 나와서 좋았습니다. 빠른 시일내에 또 들리겠습니다. (2021년 1월 27일 수요일) - 작성자: 블랙공주30 (사진 5)
리뷰 240: 코로나땜에 룸시설이 완벽히 되어있는 식당을
찾다가, 무화잠에서 저녁을 먹었습니다
Vip B코스로 먹었는데, 배가 터질정도 입니다
맛도 너무 맛있었구요 (2020년 12월 30일 수요일) - 작성자: 정법 행정사사무소 (사진 39)
로드된 리뷰 개수: 250, 수집된 리뷰 개수: 240
리뷰 241: 늘 만족합니다~~ (2021년 4월 20일 화요일) - 작성자: 김용석9 (사진 1)
리뷰 242: 킹크랩 가격은 올랐는데, 장도 쓰고 많이 아쉬웠습니다. (2021년 3월 24일 수요일) - 작성자: 이또한91 (Unknown)
리뷰 243: 1년만에 재방문했습니다. 위치가 변경되어서 좀 당황했구요. 맛은 동일하네요. (2021년 3월 11일 목요일) - 작성자: cs**** (사진 46)
리뷰 244: 맛, 서비스 모두 늘 값어치를 하는 장소.
특별한 날, 특별한 즐거움을 줘서 좋다.
(외국인 서빙 직원 교육은 필요해보임) (2021년 1월 30일 토요일) - 작성자: win**** (사진 5)
리뷰 245: 음식, 서비스 모두 만족! (2021년 4월 12일 월요일) - 작성자: INKYUNG93 (Unknown)
리뷰 246: 친절하고 맛있어서 재예약해서 갔어요 당연 이번에도 만족하고 갑니다. 수고들 하세요~ (2021년 2월 25일 목요일) - 작성자: 블랙공주30 (사진 5)
리뷰 247: 음식은 깔끔하고 좋은데 룸이 좀 시끄러웠어요 (2021년 1월 23일 토요일) - 작성

리뷰 317: 늘 만족하며 먹는 곳 (2020년 5월 29일 금요일) - 작성자: lia**** (사진 38)
리뷰 318: 역시는 역시 (2020년 6월 4일 목요일) - 작성자: 맛없으면안가요 (사진 488)
리뷰 319: 개별실이라서 조용하고 좋았습니다. 서비스도 친절하고 좋았습니다! (2020년 5월 4일 월요일) - 작성자: 김무니 (사진 27)
리뷰 320: 분위기가 고급스러워서 좋아요. (2020년 5월 8일 금요일) - 작성자: jdw64 (사진 22)
로드된 리뷰 개수: 330, 수집된 리뷰 개수: 320
리뷰 321: 어버이날에 기분좋게 식사하고 왔습니다~^^ (2020년 5월 8일 금요일) - 작성자: 김민경98 (사진 12)
리뷰 322: 맛잇어요 (2020년 6월 7일 일요일) - 작성자: ali**** (사진 20)
리뷰 323: 점심때 법인카드로 가끔씩 왔는데
아내 생일로 저녁에 와보게 되었습니다. 분위기도 좋고 만족스러웠습니다. (2020년 4월 21일 화요일) - 작성자: 다니엘2392 (사진 9)
리뷰 324: 대게코스인가 먹었는데 너무 맛있었어요~~~ (2020년 5월 1일 금요일) - 작성자: aa8119 (사진 17)
리뷰 325: 짱맛있었어용 (2020년 5월 28일 목요일) - 작성자: 111140 (Unknown)
리뷰 326: 잘먹고 갑니다. (2020년 5월 28일 목요일) - 작성자: 백곰6066 (Unknown)
리뷰 327: 좋아요 ㅎㅎ (2020년 7월 21일 화요일) - 작성자: 미키랑랑 (Unknown)
리뷰 328: 생일기념으로 가긴 갔는데 재방문 의사는 없습니다.
개별룸으로 되어 있는건 좋으나 너무 조용해요..
(음악같은거 신경 좀 더 쓰시면 좋을 것 같습니다.)
솔직히 말씀 드리면 가격대비 음식이 좀 별로예요.
둘이서 30만원 조금 넘게 나왔는데 코스에 나오는 음식이 일반한정식 집과 다를바가 없어요.. ;;
특별한게 없어요.. 퀄리티도 마찬가지고요..
그리고 음식 나오는 시간도 너무 길어요..ㅠㅠ 

로드된 리뷰 개수: 410, 수집된 리뷰 개수: 400
리뷰 401: 움식 깔끔하고 맛나요 시어머니랑 했던 대화가.. 상견례때 밥집보다 더 맛있다고 했어용 ㅎ ㅎ (2019년 11월 7일 목요일) - 작성자: 마이헌86 (사진 8)
리뷰 402: 방으로 나누어져 있어서 편했습니다 (2019년 11월 12일 화요일) - 작성자: popeyes00 (Unknown)
리뷰 403: 대게랑 튀김이 같이 나와서 애매했습니다. (2019년 10월 27일 일요일) - 작성자: yongstyle (사진 20)
리뷰 404: A세트(인당 12만원) 구성과 양이 매우 만족스러웠습니다. 분위기 좋고 음식맛도 뛰어났으며 직원분들도 친절하였습니다. (2019년 11월 6일 수요일) - 작성자: 랄쿠하됴 (사진 12)
리뷰 405: 이렇게 맛있는 게 오랜만에 먹었어요~!! 완전 대 만족입니다!!! (2019년 11월 21일 목요일) - 작성자: 하하하5568 (사진 26)
리뷰 406: 깔끔했습니다. 가격은 조금 비싼편인것 같아요 (2019년 11월 21일 목요일) - 작성자: 백수가과로사 (사진 1)
리뷰 407: 너무 좋습니다. 다음에 또 방문하겠어요. (2019년 11월 16일 토요일) - 작성자: Minchae35 (사진 68)
리뷰 408: 만족해요 잘먹었습니다 (2019년 12월 18일 수요일) - 작성자: 이뎅궁 (사진 15)
리뷰 409: 맛있었어요 !! (2020년 2월 8일 토요일) - 작성자: 루리루리59 (사진 28)
리뷰 410: 맛도 맛인데 세심한 서비스가 너무 좋았어요 (2019년 11월 9일 토요일) - 작성자: 도뉴 (Unknown)
로드된 리뷰 개수: 420, 수집된 리뷰 개수: 410
리뷰 411: 너무 고급스럽고 맛있고 친절하고 ~~ 좋았어요
코스요리 강추해요 (2019년 9월 11일 수요일) - 작성자: 댄스여왕41 (사진 20)
리뷰 412: 이번이 세번째 방문입니다 그전에 너무 맛있게 먹어서 재방문했으나 먹어본 대게중에 최악이네요
그전에는 게를

리뷰 486: 비싼만큼 맛있네요~ (2019년 1월 8일 화요일) - 작성자: 박민수102 (사진 6)
리뷰 487: 대게런치.세트 메뉴 먹으러 갔는데
연말이라 그런지 없애 버려서
그냥 버터구이 대게 먹고 왔는데
큰 감동 없이 so... so...입니다.

다시 가고 싶다는 생각이.안드네요 ㅡㅜ (2018년 12월 20일 목요일) - 작성자: 지지5599 (사진 11)
리뷰 488: VIP C코스 가격대비 보통이었네요. 나쁘진 않았지만, 재방문할 것 같지는 않아요~ (2019년 1월 10일 목요일) - 작성자: 스위밍7561 (Unknown)
리뷰 489: 맛있고 분위기 좋고 너무 좋았어요 (2019년 1월 5일 토요일) - 작성자: bsh**** (사진 10)
리뷰 490: 대게코스 먹었는데 싱싱하고 맛있어요.
같이나오는 회랑 음식들도 맛있었구요~
룸도 조용해서 얘기하기도 좋았고,
이번은 친구들이랑 송년회였는데
가족들이랑도 한번 가야겠어요! (2018년 11월 24일 토요일) - 작성자: pl**** (사진 2)
로드된 리뷰 개수: 500, 수집된 리뷰 개수: 490
리뷰 491: 전체적으로 맛은 있으니 가격대비 음식이 비싸요
분위기좋고 서비스좋고 주차장 편하고 조용하게 식사하기위해 비용지불한다 생각하면 비싼건 아니고요 음식자체만보면 비쌈 (2018년 12월 1일 토요일) - 작성자: euni3182 (사진 1)
리뷰 492: 서비스,분위기, 맛 어느하나 나무랄대가 없지만 대게튀김이 덜 바삭하더라구요ㅜㅜ 튀김의 생명은 바삭함인데ㅠㅠ 그래도 살이 꽉찬 대게는 실망을 주지않았어요~ 튀김 조금만 더 신경써주세요~^^ (2018년 10월 21일 일요일) - 작성자: jih**** (사진 179)
리뷰 493: 각각 룸으로 되어 있어서 조용히 식사할 수 있습니다. 담당 작원분도 친절하시고, 코스 서빙 타이밍이 아주 적절해서 흐름 끊기지 않고 즐거운 시간이었어요. (2018년 11월 10일 토요일) - 작성자: 도뉴 (Unknown)
리뷰 494: 음식나오는데1시간걸려

로드된 리뷰 개수: 560, 수집된 리뷰 개수: 550
리뷰 551: 맛있게 잘먹었습니다~ (2017년 9월 25일 월요일) - 작성자: 밍디879 (사진 33)
리뷰 552: 비싸지만 분위기좋고 맛있었습니다. 서비스를 좀주시면좋았을텐데 아쉽습니다. (2017년 8월 5일 토요일) - 작성자: 실버6822 (사진 7)
리뷰 553: 친절해요 비싸지만.. (2017년 7월 30일 일요일) - 작성자: 아기다람지 (사진 89)
리뷰 554: 친절했고 맛있게 잘 먹었습니다.
문을 일찍 닫네요.. (2017년 7월 28일 금요일) - 작성자: 카하만요 (사진 3)
리뷰 555: 가격은 비싼편인데 룸이라 조용하고 좋았습니다. 음식도 깔끔해요. (2017년 4월 22일 토요일) - 작성자: 민조이94 (사진 23)
리뷰 556: 다시 갈 일 없음!! 저녁 메인 c코스 주문하려는데 작은 대게가 없어서 c코스 주문이 안된다며?!! a,b 코스 주문하랜다. 다른분후기 보니 모두c코스 주문하던데 왜 더 과한 메뉴를 억지로 주문해야하지? 그건 매장사정아닌가? 예약까지 했고 전날 전화해서 코스메뉴만 가능하다는 답변도 들은터라 황당했음.설령c코스 주문이 안되면 예약자에게 미리 전화라도 해야하지 않나?없다고 하면 다인가? 그리고 중요한건 미안하다는 답변을 단 한마디도 들을 수 없음!! 먹던지 말던지 맘대로 하라는건가?! 클레임 거니 그제서야 b코스를 c코스 가격에 준다며.그래서 분명 직원에게 말했음! 매장사정으로 이렇게 된거면 선 양해or사과 이후 상황설명이 먼저가 아니냐고.이후에도 사과없음. 가격에비해 맛도 소소!! 한정식집이 훨 좋음 (2017년 5월 12일 금요일) - 작성자: 와리바시공주 (사진 22)
리뷰 557: 여자 친구와 좀 무리해서 방문했는데요.
전혀 후회하지 않을정도로 만족스러운 식사였습니다. 들어갈 때 인테리어부터 음식의 맛, 서비스 모두 만족스러웠습니다. 다음에 기회가 되면 부모님 모시고도 가고싶네요. (2017년 5월 6일 토요일) - 작성자: dan**** (

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%B6%81%EA%B2%BD%EB%B0%98%EC%A0%90/place/850010002?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 북경반점
업종 카테고리: 중식당
별점: 방문자 리뷰 80
방문자 리뷰: 9
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 33


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 가격이 저렴하고, 친절하시고, 음식도 맛있습니다. (2024년 10월 16일 수요일) - 작성자: Jinny Kim (사진 178)
리뷰 2: 볶음밥 맛집입니다. 정말 친구들에게도 추천하고싶은 곳이에요! (2024년 6월 11일 화요일) - 작성자: 아름다운날에는 (사진 83)
리뷰 3: 간만에 기교없는 중화요리 먹었어요 (2024년 6월 27일 목요일) - 작성자: 코끼리다리78 (사진 12)
리뷰 4: 짜장면 먹고싶어서 급으로 가서 ㅎㅎ
맛나요 편한분위깁니당 (2024년 5월 7일 화요일) - 작성자: 송드류 (사진 97)
리뷰 5: 맛잇어용! (2024년 6월 19일 수요일) - 작성자: 농약같은가스나86 (사진 185)
리뷰 6: 회사 점심시간
짬뽕밥 맛나여 (2024년 2월 21일 수요일) - 작성자: 송드류 (사진 97)
리뷰 7: 좋아요 (2024년 2월 29일 목요일) - 작성자: don**** (Unknown)
리뷰 8: 짬뽕밥이 맛남 (2023년 11월 21일 화요일) - 작성자: 송드류 (사진 97)
리뷰 9: 좋아요 (2024년 2월 13일 화요일) - 작성자: yoa**** (사진 320)
리뷰 10: 굿 (2023년 11월 7일 화요일) - 작성자: 콩나물구나무 (사진 367)
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 정통 수제는 아닌 듯... (2023년 9월 1일 금요일) - 작성자: 망고20 (사진 2)
리뷰 12: 탕짜면 먹엇어요~
평일 점심시간보다 살짝 일찍가서 겨우 앉앗고 저희 앉고 바로 웨이팅 생기더라구요!
가성비👍 (2023년 5월 25일 목요일) - 작성자: suj**** (사진 371)
리뷰 13: 적당히 괜찮아요! (2023년 3월 6일 월요일) - 작성자: 룰루랄라785 (사진 4)


In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EA%B9%A1%EB%8F%88/place/37902481?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 깡돈
업종 카테고리: 돼지고기구이
별점: 4.38
방문자 리뷰: 281
블로그 리뷰: 19
리뷰 탭 클릭 완료
총 리뷰 수: 129


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 좋아요~ (2024년 10월 31일 목요일) - 작성자: 양배추dear맘 (사진 536)
리뷰 2: 알고보면 북어국 맛집입니다. (2024년 8월 12일 월요일) - 작성자: 아름다운날에는 (사진 83)
리뷰 3: 간단히 밥먹기 좋은곳! (2024년 8월 9일 금요일) - 작성자: 아름다운날에는 (사진 83)
리뷰 4: 맛있어요 (2024년 9월 6일 금요일) - 작성자: 바다코끼리61 (사진 1)
리뷰 5: 알고보면 가성비 최고의 식당입니다. (2024년 6월 25일 화요일) - 작성자: 아름다운날에는 (사진 83)
리뷰 6: 좋아요 (2024년 7월 2일 화요일) - 작성자: 바다코끼리61 (사진 1)
리뷰 7: 맛있어요 가성비좋아요 (2024년 3월 8일 금요일) - 작성자: YJS700 (사진 112)
리뷰 8: 뒷고기나 덜미살 생각날 때 갈 것 같아요.
특이한 기름안튀는 불판도 좋고, 밑반찬 구성과 쌈도 좋아요.
김치랑 파채도 맛있고 기본으로 찌개, 계란찜, 김치전까지 나오는데 고기 가격도 인근에 비해 저렴하네요.
옛날 노포느낌 입니다.
잘 먹고 갑니다~ (2023년 8월 8일 화요일) - 작성자: ViViAN (사진 602)
리뷰 9: 회사옆이라 고기먹으로 두번정도 방문힌듯.
잘 모르는 부위고기도 먹어볼수잇엇네요 (2023년 12월 21일 목요일) - 작성자: 송드류 (사진 97)
리뷰 10: 점심메뉴 8000원인데 가성비 좋고 맛있습니다
음식이 간이 좀 세긴한데 맛있어요 (2023년 11월 3일 금요일) - 작성자: ㅋyle (사진 82)
로드된 리뷰 개수: 20, 수집된 리뷰 개수: 10
리뷰 11: 처음 방문
회사근처라서 방문 (2023년 11월 23일 목요일) - 작성자: 송드류 (사진 97)
리뷰 12: 좋아요 (2023년 12월 28

리뷰 125: 맛있어요 가성비 좋아요 (2020년 3월 13일 금요일) - 작성자: mo**** (사진 2)
리뷰 126: 맛있어요 (2020년 3월 13일 금요일) - 작성자: 달다란 (사진 2)
리뷰 127: 반찬도맛있고 가격도저렴해요 좋습니다 (2019년 6월 7일 금요일) - 작성자: mumu3 (사진 28)
리뷰 128: 굿굿 (2019년 6월 10일 월요일) - 작성자: mumu3 (사진 28)
리뷰 129: 직장인들이 찾기좋은곳
빠르게 메뉴가 준비되요. (2019년 3월 11일 월요일) - 작성자: Aloha RJ (사진 14)
'더보기' 버튼이 더 이상 없습니다.
총 129개의 리뷰를 수집했습니다. (예상 총 리뷰 수: 129)
리뷰 데이터가 '깡돈_reviews.csv' 파일로 저장되었습니다.


In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%B4%89%ED%94%BC%EC%96%91%20%EC%96%91%EC%9E%AC%EC%A0%90/place/37997726?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 봉피양 양재점
업종 카테고리: 한식
별점: 4.31
방문자 리뷰: 625
블로그 리뷰: 251
리뷰 탭 클릭 완료
총 리뷰 수: 288


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 냉면이랑 갈비먹러 왔어요~! 예약하니 룸 안내해주시네요.
반찬이랑 고기 정갈하고 맛있아요🥰🥰🥰 서버분이 맛나게 구워주셔서 신나게 먹고왔네용😆😆❤️ 냉면 말해모하나요~ 깔끔하고 맛있어요♡♡굿굿굿👍👍

또와야지! (2024년 10월 6일 일요일) - 작성자: 줄리공쥬 (사진 721)
리뷰 2: 유명한 평양냉면집인데 기회가 없어서 방문하지못하다가 근처올 일이 있어서 들렀습니다 다른 평양냉면이랑 외관상 다른 특이한점은 얼갈이배추 물기를 꼭짠 고명을 올려준다는 점 그리고 계란지단을 채썰어서 올려준다는 점 정도입니다 육수에서 육향이 매우 진하게 느껴지는게 인상적이었고 면은 잘끊어지는식감을 보완하려고 그런지 다른 곳보다는 탱글한 식감이 있었습니다을밀대랑은 확실히 다른 맛인것 같았습니다
맛있게 잘먹었어요 (2024년 8월 23일 금요일) - 작성자: 레이77777 (사진 1,081)
리뷰 3: 여사님들 친절하시고 고기도 잘 구워주시고 안내도 잘해주셨습니다. 점심 예약을 했음에도 제가 동행을 오래 기다렸어야 했는데 양해해 주셔서 감사했습니다. 음식은 다 맛있고 정갈하고 좋았습니다. 담에 가면 다른 식사류도 먹어보고 싶었습니다. (2024년 9월 6일 금요일) - 작성자: weemee (사진 165)
리뷰 4: 봉피양 대치는 가봣어도 양재점은 첨인데 친절하셔서 감동이엇어요
맛도 맛이지만 직원분들께서 친절 하셔서 녹두전 쵝오 더 맛잇고 냉면 속이 편하게 맛났어요 감사히 잘 먹었습니다 (2024년 7월 22일 월요일) - 작성자: 바이올렛 76 (사진 379)
리뷰 5: 음식은 값을 못하고 서비스도 형편 없다못해 과도하게 불친절합니다. 일단 재료가 없어서 못만드는지 특정 메뉴를 팔아먹으라고 교육을 받은건지 메뉴 유도를 부모님이 불편하실때까지 엄청 합니다. 일단 넘어갔습니다

로드된 리뷰 개수: 70, 수집된 리뷰 개수: 60
리뷰 61: 비쌈..근데 맛있음 (2023년 10월 19일 목요일) - 작성자: mul**** (사진 6)
리뷰 62: 평양냉면 입문자도 부담없이 먹을 수 있는 냉면맛집이에요. (2023년 6월 2일 금요일) - 작성자: 도로그 (사진 120)
리뷰 63: 봉피양 다른 지점들 좋아했는데 여긴 오늘 이후로 안갈 듯... 들어갔는데 안내해주는 사람이 아무도 없어서 기다리다가 2층 올라갔더니 1층에서 안내 받고 올라와야하니까 다시 내려가라고 1층에 안내할 사람 있다고 내려보냄, 근데 또 다시 부르더니 아무 자리나 앉힘. 그럴거면 무전은 왜 끼고 일하시는지. 여튼 주문하고 소주 병 딱 열었는데 갑자기 30분만에 먹고 나가라고 함 예약 있다고. 그럼 앉히거나 음식 내기 전에 미리 말을 해줘야죠? 어이없는 표정으로 쳐다보니까 그럼 30분 먹다가 다른 데로 자리 옮겨야 한다고 함 ㅋㅋㅋ 이런 집 처음 봤네요. 그러고도 미안하단 말 한 마디 없고 기분 완전 나빠서 빨리 먹고 일어났더니 계산할 때 되어서야 미안하다고 하고. 진짜 이 정도로 기본 안 된 식당 오랜만이었습니다. (2023년 8월 4일 금요일) - 작성자: 민무늬양복33 (사진 6)
리뷰 64: 가격이 계속 오르는 것이 아쉽지만, 언제 먹어도 양곰탕은 최고입니다! (2023년 6월 12일 월요일) - 작성자: 돌돔과바다 (사진 339)
리뷰 65: 양곰탕도 점심 후식메뉴에 추가해주시면 좋겠습니다!! 돼지 정식도 정말 양도 맛도 훌륭합니다. (2023년 6월 16일 금요일) - 작성자: 돌돔과바다 (사진 339)
리뷰 66: 와! 평양냉면 너무 맛있네요!
좀 비싸긴 하지만 너무 맛있어요!! 다시 생각나는 맛! (2023년 5월 26일 금요일) - 작성자: 빛1358 (사진 32)
리뷰 67: 슴슴하고 맛있는 봉피양 평양냉면 16,000원 (2023년 6월 18일 일요일) - 작성자: Saguaro NP (사진 1,538)
리뷰 68: 봉피앙 맛있어요 (2023년 7

리뷰 140: 가격대에 맞는 양념갈비는 인정, 그러나 일하시는분이 너무 바뻐서 봉피양의 급에 맞는 서빙을 하기 힘들고 중간중간 뜬금없는 영업에 민망했습니다. 간만에 가족모임에 대화중인데 봉피양 세트 영업들어오는건 아닌거 같구요, 양념갈비와 냉면의 품질과 급에 맞는 서비스에 집중하셔야될거 같습니다. (2022년 5월 1일 일요일) - 작성자: jun**** (사진 37)
로드된 리뷰 개수: 150, 수집된 리뷰 개수: 140
리뷰 141: 서버가 고기를 굽다말고 자리비우고, 덜익거나 타질않나, 반찬도 순서없이 나오고 (고기먹다 샐러드 나옴), 반찬 접시 정리도 슬라이딩에 갈비는 구우라며 집게는 가져가면 젓가락으로 구워요? 매우 저렴한 삼겹살집보다 더 서비스가 불만족스러움. 아래 다른 불친절 후기봤으면 룸잡고 먹으러 안갈듯함. 다시는 봉피양 안갈것임. (2022년 5월 20일 금요일) - 작성자: GN3K (사진 1)
리뷰 142: 평냉만 먹다가 만두 추가했는데 만두도 맛있어요 (2022년 8월 8일 월요일) - 작성자: Liiiiiiiii (사진 126)
리뷰 143: 존맛탱 (2022년 7월 12일 화요일) - 작성자: 몰뱌 (사진 769)
리뷰 144: 봉피앙의 맛의 비밀은...?!ㅎ (2022년 7월 4일 월요일) - 작성자: 두맨이 (사진 40)
리뷰 145: 가격이 비싸다 (2022년 9월 21일 수요일) - 작성자: 럭셔리장 (사진 1)
리뷰 146: 양재지나가다 평양냉면 땡겨서 우래옥
생각하며 갔다가 우래옥갈껄 100번 후회한 집
고기를 구워주는 테이블밖에 안남았다며 어쩔거냐기에 그때 빠른 걸음으로 나갔어야 했다.
고기식을때쯤 30분만에 나온 평양냉면은
테이블끝에서 슬라이딩해주고 태어나서 이런 고깃집에서 이런 서빙은 처음받아봄. 메뉴판은 치워주지도 않고 냉면가위는 물어보지도 주지도 않고 쌩하고 떠나버려 셀프로 가져옴.
88000원짜리 밥먹었는데 만족도는 8800원이였음. (2022년 1월 8일 토요일) - 작성자: RyuPlay (사진 180)
리뷰

리뷰 217: 친절하고 음식이 정갈하고 맛있습니다. (2021년 3월 14일 일요일) - 작성자: 송파럭셔리킴 (사진 2,171)
리뷰 218: 그냥 (2021년 6월 26일 토요일) - 작성자: dana9249 (사진 68)
리뷰 219: 평양냉면 생각날 때 가까워서 들러요♥ (2021년 3월 7일 일요일) - 작성자: jadecrush55 (사진 152)
리뷰 220: 봉피양 좋아하고 맛있는데..
양재점은결제시 주문내역도 안 맞고 직원들이 너무산만해서 안정적인 식사가 이뤄지지않네요..

손님이 더 떠나가기전에 개선이 필요할듯하여 리뷰 남깁니다. (2021년 2월 27일 토요일) - 작성자: 이수쭈꾸미 (사진 899)
로드된 리뷰 개수: 230, 수집된 리뷰 개수: 220
리뷰 221: 조아요!! (2022년 1월 16일 일요일) - 작성자: wyy**** (사진 162)
리뷰 222: 맛있어요 (2021년 7월 22일 목요일) - 작성자: jamie98 (사진 9)
리뷰 223: 맛있지만 좀 더 매장정리랑 위생에 더 신경써야 할 것 같아요 컵에 물 따르고 나서 봤더니 립스틱 자국이 그대로 남아있는 컵이더라구요...음식값만큼 매장 정리에 더 신경써야 할 듯요 (2021년 2월 26일 금요일) - 작성자: 민트초코념념굿 (사진 313)
리뷰 224: 맛있어요 (2021년 10월 1일 금요일) - 작성자: okl**** (Unknown)
리뷰 225: 주말에는 사람이 적어서 좋아요 (2021년 3월 7일 일요일) - 작성자: 남승현57 (사진 113)
리뷰 226: 좋아요 (2021년 6월 24일 목요일) - 작성자: spr**** (Unknown)
리뷰 227: 굿나 (2021년 5월 21일 금요일) - 작성자: 서울밴드단장 (사진 24)
리뷰 228: 가격이 장난아니네요 맛보기냉면이 9천원~
맛은 있어요 (2021년 3월 31일 수요일) - 작성자: 화이팅8458 (사진 16)
리뷰 229: 맛있어요 (2021년 5월 21일 금요일) - 작성자: ali***

In [40]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%A7%A5%EC%BC%84%EC%B9%98%ED%82%A8/place/18360454?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 맥켄치킨
업종 카테고리: 치킨,닭강정
별점: 4.71
방문자 리뷰: 196
블로그 리뷰: 37
리뷰 탭 클릭 완료
총 리뷰 수: 75


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 간만에 친구랑 먹고 마시다가 아쉬워서 2차로 별 생각없이 들어간 집.

추억의 캐챱+마요네즈 사라다.
닭다리살 야들하고 촉촉쓰하지만 겉은 바싹한 맛도리 순살치킨.

화장실은 정말 최고! 깨끗하고 몬가 럭셔리해... 친구랑 역대급 화장실이라고 화장실 이야기로 낄낄대면서 맥주 한잔 순식간에 비움 ㅋ (2024년 8월 14일 수요일) - 작성자: 야뇌청룡 (사진 242)
리뷰 2: 강남구 원탑 치킨집
진짜 개맛있습니다
전반적으로 후라이드도 양념도 매콤한 맛이 디폴트
사장님의 친근한 친절함이 정있게 느껴졌어요

야장 자리있으면 일단 앉으세요 (2024년 5월 17일 금요일) - 작성자: Junhwan6 (사진 66)
리뷰 3: 치킨에서 고소한 맛이 난다
옛날식 양배추 샐러드와 병사이다..
바삭한 치킨, 파닭으로 소주 두병이 순삭..
주인 어르신께서 친절도 하시고
열심이신듯 ~
건강하게 오래 하세요~ (2024년 3월 27일 수요일) - 작성자: 전국 맛 빵집 찾아라 (사진 254)
리뷰 4: 맛있어영 감사합니다 (2024년 8월 30일 금요일) - 작성자: hua**** (사진 75)
리뷰 5: 일끝나구 회사동료랑 ㅎㅎ
치킨먹으로 ㅎㅎ 맛남 (2024년 4월 18일 목요일) - 작성자: 송드류 (사진 97)
리뷰 6: 기본안주 팝콘도 맛있고 옛날 느낌의 치킨도 좋아요 (2024년 4월 23일 화요일) - 작성자: wawa67 (사진 295)
리뷰 7: 제가 알기로 25년 넘은 가게로 알고 있어요.
엄청 오랫동안 같은 동일한 가게 운영이 가능하신 이유가 다 있겠죠?
맛있습니다. 가격도 합리적이고요!! (2024년 2월 21일 수요일) - 작성자: BeRich (사진 235)
리뷰 8: 파닭 맛있어요. 퍽퍽살 안좋아해서 순살 잘 안 먹는데 여기 순살치킨 괜찮습니다

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%8A%A4%ED%83%80%EB%B2%85%EC%8A%A4%20%EC%84%9C%EC%B4%88%ED%83%9C%EC%9A%B0%EB%B9%8C%EB%94%A9%EC%A0%90/place/1685227032?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 스타벅스 서초태우빌딩점
업종 카테고리: 카페
별점: 방문자 리뷰 40
방문자 리뷰: 10
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 25


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: "승상, 저는 도무지 이해가 되지 않습니다. 어찌하여 근처에 2개의 스벅이 있는데 이곳에 또 세우셨습니까?"
"우선 이 땅 위로 커피의 역사를 계속 흐르게 하고 싶었네. 짧았지만 강렬했던 커피의 인상을 남겼던 곳인 만큼, 이곳에는 다른 세력이 오게 해서는 안 되었지."
"그래도 다른 두 곳 때문에 상대적으로 부진..."
"허허. 이 사람. 그렇지 않아도 내 오늘 다녀왔네. 매장도 넓고, 좋은 향도 나고 이야기를 나누기엔 더없이 좋은 공간이었네."
"그래도 사람이 올까요?"
"뱅뱅사거리를 끼고 있어 사통팔달 교통이 편한데다, 정류장이 눈 앞인데 뭐가 걱정인가? 거기다 🌳집, 🦀집, 🚗집, 💰집이 인근에 있어서 걱정이 없다네."
"그래도..."
"허어. 이 사람! 어느 매장을 가도 같은 품질이네. 됐나?" (2024년 10월 4일 금요일) - 작성자: 탈퇴한 가입자 (사진 1,140)
리뷰 2: 집근처 뱅뱅사거리점 없어지구ㅜ 바로 맞은편에 스벅 새로 생겼네요 넘좋아요~~집이랑 젤 가까운곳🩷 역시 항상 사람 많네용😊 (2024년 10월 30일 수요일) - 작성자: 연어초밥사랑해 (사진 415)
리뷰 3: 오늘 오픈한 서초태우빌딩점...2만원이상 머그컵 증정 행사가 있길래 꾸역꾸역 2만원 구매해서 받음ㅋㅋㅋ
머그컵이 모라고 ㅋㅋㅋㅋ
집근처라 앞으로 자주 방문할거 같음. (2024년 9월 29일 일요일) - 작성자: 피요나슈렉 (사진 975)
리뷰 4: 얼마전까지 공사하구 있었는데
오늘 소니깐 오픈한거 같아서
들어가 봤는데^~^
분위기 좋음 (2024년 10월 4일 금요일) - 작성자: 베야로드 (사진 333)
리뷰 5: 좋아요.급 작업해야해서 들어왔는데
집중 최고 잘 됩니다. (2024년 9월 30일 월요일) - 작성자: Chriyang (사진 

In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%82%A8%EA%B0%95%EB%A7%A4%EC%A0%90/place/35112336?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 남강매점
업종 카테고리: 백반,가정식
별점: 4.31
방문자 리뷰: 148
블로그 리뷰: 12
리뷰 탭 클릭 완료
총 리뷰 수: 75


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 건물 지하에 있는 주차장 옆에 있는 매점입니다.
근데 매점에서 식사도 하네요.ㅎㅎㅎ
지상에서는 간판이 없어서
여기에 식당이 있는줄 알기 어려울텐데...
우연히 알게돼서 가봤는데요.
어릴적 향수를 불러오는 그런 곳이네용~
친절하시고, 분위기도 좋구요! ㅎㅎㅎ
음식 맛은 평타입니다!
하지만, 분위기가 좋아서 종종 가보려고 합니다~ (2024년 9월 27일 금요일) - 작성자: idisjien94 (사진 1,088)
리뷰 2: 진심을 다한 요리를 내어주셔서 감사합니다. (2024년 10월 23일 수요일) - 작성자: DH31 (사진 193)
리뷰 3: 사장님 늘 친절하셔서 밥이 더 맛있게 느껴집니다. (2024년 10월 18일 금요일) - 작성자: DH31 (사진 193)
리뷰 4: 동태탕 맛이 끝내줍니다! (2024년 11월 13일 수요일) - 작성자: DH31 (사진 193)
리뷰 5: 정성가득한 집밥같은 음식 내어주셔서 감사합니다. (2024년 8월 1일 목요일) - 작성자: DH31 (사진 193)
리뷰 6: 우리 엄마보다 친절하시고 요리도 잘하심 (2024년 8월 6일 화요일) - 작성자: DH31 (사진 193)
리뷰 7: 반찬이 정말 잘 나옵니다. 반찬에서 정성을 느낄 수 있습니다. 떡만두국 잔치국수 각각 8500원으로 좀 비싼 느낌이었는데 반찬과 집에서 어머니가 만들어주는 수준으로 나오는 만두국 국수에 가격이 납득이 됩니다. 강남/서초의 숨은 맛집 인정 (2024년 4월 8일 월요일) - 작성자: 토끼 꿀 (사진 60)
리뷰 8: 한여름에 좀 시원하지 않은것 빼고는 다 괜츈 (2024년 7월 29일 월요일) - 작성자: DH31 (사진 193)
리뷰 9: 여사님들께서 친절하게 응대해주심 (2024년 7월 25일 목요일) - 작성자: DH31 (

In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%B2%84%EB%93%9C%EB%82%98%EB%AC%B4%EC%A7%91%20%EC%84%9C%EC%B4%88%EB%B3%B8%EC%A0%90/place/11627634?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 버드나무집 서초동본점
업종 카테고리: 한식
별점: 4.26
방문자 리뷰: 1,516
블로그 리뷰: 633
리뷰 탭 클릭 완료
총 리뷰 수: 717


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 국물보다 갈비가 더 많은곳!!
갈비 덜고 국물 먹으면서 갈비뜯어먹으니 진짜너무맛있네요 :)
깍두기와 오징어젓갈도 한몫합니다!!
한우갈비에 양이 1.5인분이어서 가격은 나가지만 재방문 의사 있어요! (2024년 10월 22일 화요일) - 작성자: 룰루랄라3211 (사진 169)
리뷰 2: 대기표 22 23 24번 받고
갈비탕 먹음^^

토욜이라
오픈런 손님들이 꽤 있었음~

몆번을 먹어도 맛남^^
근데 내가 배꼴이 커진건지
묘하게 양이 줄어든 느낌적인 너낌🙊

수시로 반찬 리필해주시고
맛나게 잘 먹어씀👍 (2024년 8월 31일 토요일) - 작성자: 외유내강369 (사진 1,983)
리뷰 3: 동네 친구들이랑 저녁 먹으려고 방문했어요
주차 편하고 직원분들 친절하시고 소고기도 훌륭하네요
물냉면도 맛있었어요 잘 먹고 갑니다😊😊 (2024년 10월 10일 목요일) - 작성자: 구구구구구꾸꾸구구 (사진 2,119)
리뷰 4: 30년전부터 다녔지만
이 집 갈비만큼 만족스러운 곳을 아직 못찾겠더라.
같이 나오는 반찬이며 김치에 쌈장까지
뭐하나 맛없는게 없다.
고기도 맛있게 구워주시고 필요한 반찬들
알아서 척척 가져다주시는 베테랑 이모님들 덕분에
늘 만족스러운 곳이다.
제철과일과 매실차에 이어 아이스크림까지
디저트가 늘어난 것도 너무 좋다.^^ (2024년 8월 4일 일요일) - 작성자: 프로추앙러 (사진 1,481)
리뷰 5: 소고기는 버드나무집 👍
생갈비 안창 등심 먹었어요
생갈비살이 특히 맛있어요
가끔 예약안하고 저녁에가면 품절될때가 있으니
생갈비살 드실땐 예약하고 가시는걸 추천합니다:)
사이드메뉴 된장찌개도 짜지도 않고 정말 맛나요 👍 (2024년 10월 23일 수요일) - 작성자: ㅇㄱㄱㄳㄷ (사진 4)
리뷰 6: 예전에 먹었던 맛 그대로 맛있게

리뷰 64: 💁‍♀️버드나무집

버드나무 집 갈비탕은 늘 최고예요.

생갈비는 미리 예약하는 게 좋겠더라고요.
없어서 양념으로 먹었는데 좀 아쉬웠어요.
생갈비가 먹고 싶었거든요. ㅠㅠ

그래도 맛있게 잘 먹었습니다😊 (2023년 10월 12일 목요일) - 작성자: 경험디자이너 (사진 420)
리뷰 65: 좋아요 (2024년 6월 24일 월요일) - 작성자: don**** (Unknown)
리뷰 66: 일본으로부터 푸디 지인들이 서울을 방문해서 늘 좋아하는 이곳으로 갔는데 미리 최근 아쉬웠던 서비스 이슈를 말씀드리고 잘 부탁드렸더니 예전처럼 이것저것 세심히 살펴주셨습니다. 음식도 맛있어서 지인들도 대만족. (2023년 10월 8일 일요일) - 작성자: kkk9320 (사진 246)
리뷰 67: 갈비탕 먹으려고 오픈런했는데,
맛있네요 ㅎㅎ (2024년 1월 14일 일요일) - 작성자: 푸우푸우17 (사진 30)
리뷰 68: 미슐랭 맛집이라길래 가봤습니다.
전체적으로 맵거나 짠 음식없이 조금 단맛이었어요.
외국인 손님과 함께가기에 괜찮을 것 같은 음식들!
가격대가 높긴하지만 한 번씩 가볼만하네요. (2024년 1월 28일 일요일) - 작성자: 율리8213 (사진 64)
리뷰 69: 점심메뉴 갈비탕 & 소고기국밥 먹었습니다
2명에서 메뉴 하나씩 시켜서 같이먹는거 추천드려요
갈비 많다고 생각했는데
바닥에는 비계가 많이 깔려있네요..;;
소고기국밥은 맵지 않아서 좋았어요 (2023년 9월 23일 토요일) - 작성자: 밝을소영 (사진 294)
리뷰 70: 좋아하는 맛집 ~~
실장님 ~~직원분들 너무 친절하십니다 ~~ (2024년 3월 7일 목요일) - 작성자: 덤보아빠 (사진 11)
로드된 리뷰 개수: 80, 수집된 리뷰 개수: 70
리뷰 71: 소고기국밥에 한우고기도 엄청 많이 주시지만 후식으로 제가 좋아하는 아이스크림까지 넘넘 맛있어요~~ (2023년 10월 21일 토요일) - 작성자: jas**** (Unknown)
리뷰 72: 점심특선도 맛있어요. (2024년 1월

리뷰 137: 맛있는데 비싸고 비싼데 맛있는 곳^^ (2023년 8월 7일 월요일) - 작성자: 뜨끈뜨끈호떡 (사진 387)
리뷰 138: 맛있어요. (2023년 10월 2일 월요일) - 작성자: 오늘또감사해 (사진 209)
리뷰 139: 맛있어요 (2023년 12월 8일 금요일) - 작성자: Let it be31 (사진 5)
리뷰 140: 1일100그릇 한정!! 번호표받아서 먹는 한우갈비탕 맛집이예요~
반찬가짓수도 많은데 다 맛있는 반찬맛집
식후에 주시는 식혜도 달달 깔끔해요
디저트 아이스크림도 챙겨먹고왔어요
갈비탕 32000원 가격은 사악하나 맛은 인정👍🏼 (2023년 2월 19일 일요일) - 작성자: 샹들리유 (사진 1,753)
로드된 리뷰 개수: 150, 수집된 리뷰 개수: 140
리뷰 141: 👍 (2024년 1월 8일 월요일) - 작성자: 자곰 (사진 115)
리뷰 142: 추억의 가게😊 (2023년 8월 27일 일요일) - 작성자: 이수쭈꾸미 (Unknown)
리뷰 143: 버드나무집 오래된 맛집이어서 가족식사로 예약하고 방문했는데 노포맛집이라고 하기에…오랜만에 방문한 버드나무집은 서비스가 너무 실망스럽다 못해 불쾌했어요.

이모님 보통 룸을 체크할 때는
부족한 부분이 없는지 물어보시고
반찬을채워주시거나 편안한 식사를 돕기 위한 체크를 하시는데
식사 시작한지 30분만에 고기 추가 안하시면 불을 빼겠다고 하시고 식사중에 그릇 정리하시고 공기밥에서는 머리카락이 나왔는데 아이들이 있어서 바꿔달라고만 이야기했지만 죄송하다는 사과도 없으셨어요.

전화로 서비스가 불편했다고 말씀드리니 늦게 도착했냐고 저희한테 원인을 찾는 질문을 하시는 거 보고 그 사이 많이 바뀐건지 더 이야기할 필요가 없겠다 싶었습니다..

4인 70만원 식사로 오는 레스토랑이 이런 서비스일꺼라 상상 못했습니다. (2023년 7월 10일 월요일) - 작성자: 별쫑91 (사진 45)
리뷰 144: 좋아요 (2023년 10월 29일 일요일) - 작성자: 마노4211 (사진 640)
리뷰 145

로드된 리뷰 개수: 230, 수집된 리뷰 개수: 220
리뷰 221: 국밥 맛나요 (2023년 4월 17일 월요일) - 작성자: hyunji0426 (사진 36)
리뷰 222: 비싸지만 음식맛은 괜찮아요 (2023년 5월 15일 월요일) - 작성자: 빵파34 (사진 3)
리뷰 223: 굿~~ (2023년 4월 11일 화요일) - 작성자: gam**** (사진 159)
리뷰 224: 국밥 따뜻하게 잘먹었습니다!! (2022년 12월 15일 목요일) - 작성자: KHS28 (사진 1,229)
리뷰 225: 갈비 정식 먹었는데 맛있습니다. 반찬들도 맛있었네요. 후식으로는 수정과와 파인애플이 나오고 나오는길에 아이스크림이나 커피도 있어요 (2022년 9월 13일 화요일) - 작성자: 으스으74 (사진 456)
리뷰 226: 소고기국밥이 너무 맛있습니다. (2023년 1월 13일 금요일) - 작성자: 은박4 (사진 157)
리뷰 227: ^^ (2023년 4월 20일 목요일) - 작성자: 1l7 (Unknown)
리뷰 228: 갈비정식 1인에 43,000원짜리 식사하고 왔어요
고기는 직원분이 구워주셨고 맛은 부드럽고 괜찮았지만 과연 이 가격에 먹는게 맞는지 의문이 드는 한끼였어요 찰밥을 추가했는데 누리끼리하고 딱딱해서 원래 이렇게 나오는지 여쭤보니 밥을 잘못 지어서 나온것같다고 하시면서 새걸로 바꿔주셨는데 누가봐도 그 전에 나온건 정말 오래된 음식이었네요 공깃밥도 전체적으로 누렇고 질고 어느부분은 딱딱하고 반찬도 겉부분이 살짝 말라있었는데 건조해서 그렇겠지하고 그냥 먹었는데 음식체크 안하고 그냥 손님상에 올려놓으시나봐요 솔직히 고기랑 된장찌개 빼고는 먹을만한게 없었네요 미쉐린가이드도 받고 워낙 유명한 가게라 기대하고 왔는데 실망이 참 컸습니다 서비스부터 시작해서 할 말이 정말 많은데 글자수가 걸려서 일단 !!강력비추천!!입니다^^ (2022년 12월 12일 월요일) - 작성자: yuf**** (사진 13)
리뷰 229: 종업원 교육이 필요합니다.
항상 어른들만 모시고 

리뷰 313: 들어오면서 카운터에서 인사를 참 밝게하셔서 기분좋게 앉았는데 식사하는 내내 서비스가 너무 엉망이라 기분않좋게 나왔습니다.
식당 이미지가 안좋을것 같아요.
많이 아쉽습니다. (2022년 6월 28일 화요일) - 작성자: 여우처럼 (사진 84)
리뷰 314: 예전에 갈비탕으로 돌아온맛~
아이스크림 기계까지 생겼네요 (2022년 10월 10일 월요일) - 작성자: kuk**** (사진 262)
리뷰 315: 굿 (2023년 5월 6일 토요일) - 작성자: 슈퍼밀크 (사진 190)
리뷰 316: 자주 이용하는 곳이에요 (2022년 11월 3일 목요일) - 작성자: sof**** (사진 54)
리뷰 317: 소고기국밥에 고기도 많고 맛있어요~~ (2022년 7월 13일 수요일) - 작성자: 석고87 (사진 111)
리뷰 318: 갈비 항상 맛있어요 (2022년 8월 9일 화요일) - 작성자: Fa cai (사진 274)
리뷰 319: 음..재방문 의사 없네요~ (2022년 9월 3일 토요일) - 작성자: odo**** (사진 9)
리뷰 320: ㆍ (2022년 9월 25일 일요일) - 작성자: soo**** (사진 13)
로드된 리뷰 개수: 330, 수집된 리뷰 개수: 320
리뷰 321: 음식은 괜찮은것 같은데...직원들 서비스가 어수선하고 성의 없어서 밥맛이 떨어졌습니다. (2022년 6월 28일 화요일) - 작성자: joj**** (사진 27)
리뷰 322: 좋아요 (2023년 1월 22일 일요일) - 작성자: daechi9 (Unknown)
리뷰 323: 비싸지만 맛있어요! 직원분들이 정말 친절하세요! (2022년 8월 18일 목요일) - 작성자: HONGVENUS (사진 143)
리뷰 324: 굿~^^ (2022년 10월 26일 수요일) - 작성자: sprite1999 (사진 2)
리뷰 325: 좋아요 (2022년 10월 25일 화요일) - 작성자: 스킨100 (사진 4)
리뷰 326: 굿 (2022년 10월 25일 화요일) - 작성자: 10**

리뷰 423: 굿 (2022년 4월 29일 금요일) - 작성자: iml**** (Unknown)
리뷰 424: 한마디로 절대 비추천!
갈비탕 먹으러 갔더니 그것도 26,000원????@.@ 한우갈비탕이라나? 그럼15,000원 정도 하는 다른곳 한우갈비탕은 가짜한우인가? 그마저 한정판매라 없단다 점심 12시 45분경
갈비정식이랑 육회비빔밥 시켰는데 정식(34,000원)은 2인이상! 시켰으나 그 이유를 알겠음. 뼈 큰거 두개에 살조금. 1인분시키면 작은 고기가 4~5점일테니 1인분짜리 내놓을 수 없음. 게다가 양념게장 맛 더럽게 없고(양념한지 며칠은 된 느낌) 반찬 다 맛없음.김치전(아기손바닥만하거) 차갑디차갑고. 반찬 고기 다 나왔는데 밥이랑 된장찌게 안 나옴. 밥 달라하니 찌개가 끓는 시간 있어서 좀 늦게 나온다고 밥이라도 줘요? (이건 뭐 얻어먹는것도 아니고) 맛없는 반찬으로 반 이상 밥 먹고 찌개나옴. (2021년 11월 25일 목요일) - 작성자: 눈누나나71 (사진 74)
리뷰 425: 좋아요 (2022년 6월 16일 목요일) - 작성자: a61**** (사진 1,066)
리뷰 426: 갈비탕 멋져요 (2022년 1월 27일 목요일) - 작성자: Goskymoon (사진 148)
리뷰 427: 굿 (2022년 3월 20일 일요일) - 작성자: 조용한 바람 (사진 998)
리뷰 428: 일일한정 한우 갈비탕 먹고 갑니다 국물이 진하고 맛있어요~ (2021년 12월 26일 일요일) - 작성자: feel조아 (사진 717)
리뷰 429: 맛은 있는데 내 돈 주고 사 먹을 만큼인지는 모르겠어요. 반찬도 평범합니다. (2022년 1월 28일 금요일) - 작성자: 탄천너구리 (Unknown)
리뷰 430: 굿 (2022년 6월 21일 화요일) - 작성자: okl**** (Unknown)
로드된 리뷰 개수: 440, 수집된 리뷰 개수: 430
리뷰 431: 좋아요 (2022년 6월 21일 화요일) - 작성자: pyh**** (Unknown)
리뷰 432: 가격은 비

로드된 리뷰 개수: 540, 수집된 리뷰 개수: 528
리뷰 529: 조아요~~~~ (2021년 11월 15일 월요일) - 작성자: 왕밤빵5475 (사진 20)
리뷰 530: 굿 (2022년 2월 22일 화요일) - 작성자: yhk**** (사진 84)
리뷰 531: ㅇㅇ (2022년 5월 27일 금요일) - 작성자: 천사여인 (사진 4)
리뷰 532: 좋아요 (2022년 1월 18일 화요일) - 작성자: pyh**** (Unknown)
리뷰 533: 굿 (2022년 1월 18일 화요일) - 작성자: okl**** (Unknown)
리뷰 534: 국밥은 옛날 맛 그대로 입니다.
너무 맛있어요 ㅎ (2021년 8월 20일 금요일) - 작성자: 뭘줄까 (사진 1)
리뷰 535: 식사를 하기에도 좋고, 고기맛도 있고 좋습니다. (2021년 6월 7일 월요일) - 작성자: Dam114 (사진 39)
리뷰 536: 친절하고 맛있어요 ㅎ ㅜ ㅎ (2021년 8월 10일 화요일) - 작성자: jh910908 (사진 107)
리뷰 537: 좋아요 (2022년 1월 30일 일요일) - 작성자: daechi9 (Unknown)
리뷰 538: 굿 (2022년 1월 13일 목요일) - 작성자: okl**** (Unknown)
로드된 리뷰 개수: 550, 수집된 리뷰 개수: 538
리뷰 539: 굿 (2022년 1월 13일 목요일) - 작성자: pyh**** (Unknown)
리뷰 540: 갈비탕 맛있어요 푸짐하고 (2021년 7월 11일 일요일) - 작성자: 0a6**** (사진 42)
리뷰 541: 소고기 맛있어요 (2021년 8월 5일 목요일) - 작성자: jss2594 (사진 1,523)
리뷰 542: 좋아요 (2022년 1월 13일 목요일) - 작성자: pyh**** (Unknown)
리뷰 543: 친절 (2022년 1월 13일 목요일) - 작성자: 임하돈 (Unknown)
리뷰 544: 종아요 (2022년 1월 4일 화요일) - 작성자: wlswn46 (Unknown)
리

리뷰 642: 오전 11시까지 가면 한우 갈비탕 푸짐하게 먹을수 있어요 (2020년 11월 24일 화요일) - 작성자: 메리3135 (사진 22)
리뷰 643: 좋아요 (2021년 8월 10일 화요일) - 작성자: 나나5489 (사진 741)
리뷰 644: 점심 회식하기 좋아요. (2021년 2월 8일 월요일) - 작성자: 계프 (사진 1,005)
리뷰 645: 예전에는 고기도 푸짐
밑반찬도 푸짐 했었는데
오랜만에 가보았는데
선지국도 정식에는 안준다고 해요
오곡밥도 6천원이나 하드라구요
물론 기본은 맛집이지만 좀 실망 하고 왔어요
점심시간 갈비탕은 아직도 훌륭해 보였습니다 (2020년 7월 22일 수요일) - 작성자: 오 캐롤 (사진 342)
리뷰 646: 최상의 서비스를 신선한 재료로 준비한 맛집 중 맛집 입니다. 반찬이 정갈하고 맛나 더욱 좋습니다. (2020년 10월 15일 목요일) - 작성자: Trollsj (사진 7)
로드된 리뷰 개수: 660, 수집된 리뷰 개수: 646
리뷰 647: 가격대비 별로 (2020년 10월 15일 목요일) - 작성자: 맛있게먹은녀석들 (사진 491)
리뷰 648: 좋아요 (2021년 3월 29일 월요일) - 작성자: 센트네 (사진 1)
리뷰 649: 맛있어요.
늦게가면 못 먹어요. (2020년 9월 9일 수요일) - 작성자: 헤이체 (사진 104)
리뷰 650: 맛있게 잘 먹었습니다 (2020년 10월 30일 금요일) - 작성자: sunrise845 (Unknown)
리뷰 651: 한우갈비탕 맛있어요 (2020년 10월 22일 목요일) - 작성자: 날마다새롭게87 (사진 10)
리뷰 652: 비싸지만 맛있어여..😭 (2020년 12월 23일 수요일) - 작성자: dbf**** (사진 359)
리뷰 653: 양이 푸짐합니다.. 가성비까진 잡지 못하지만... (2020년 7월 28일 화요일) - 작성자: 거위의꿈43 (사진 270)
리뷰 654: 갈비탕이 맛있다고 하여 근처에 볼 일이 있어서 간김에 방문. 어마어마한 양에 놀

In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%B9%B4%ED%8E%98%EB%B8%94%EB%9E%91131/place/1214592701?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 카페블랑131
업종 카테고리: 카페
별점: 방문자 리뷰 1
방문자 리뷰: 2
블로그 리뷰: 없음
오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0089EBD3+24307]
	(No symbol) [0x00828D74]
	(No symbol) [0x0070C323]
	(No symbol) [0x0074DC86]
	(No symbol) [0x0074DECB]
	(No symbol) [0x0078B9F2]
	(No symbol) [0x0076FED4]
	(No symbol) [0x00789579]
	(No symbol) [0x0076FC26]
	(No symbol) [0x0074219C]
	(No symbol) [0x0074311D]
	GetHandleVerifier [0x00B48D93+2818227]
	GetHandleVerifier [0x00BA542E+3196750]
	GetHandleVerifier [0x00B9D9D2+3165426]
	GetHandleVerifier [0x0093DA70+675216]
	(No symbol) [0x00831B3D]
	(No symbol) [0x0082EA18]
	(No symbol) [0x0082EBB5]
	(No symbol) [0x00821640]
	BaseThreadInitThunk [0x7701FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x771E809E+286]
	RtlGetAppContainerNamedObjectPath [0x771E806E+238]

리뷰 데이터가 '카페블랑131_reviews.csv' 파일로 저장되었습니다.


In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%98%A4%EB%8A%98%EC%9D%80%20%EC%96%B4%EB%96%A4%EB%8B%AD/place/1854564675?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 오늘은 어떤 닭
업종 카테고리: 닭요리
별점: 방문자 리뷰 8
방문자 리뷰: 4
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 8


로드된 리뷰 개수: 8, 수집된 리뷰 개수: 0
리뷰 1: 닭개장은 혼자 먹기에도 충분할 정도로 양이 많고 계란찜은 무슨 계란폭탄찜으로 주시네요🤯
계육볶음도 매콤하니 술안주로 딱인 거 같아요
다음에도 꼭 오고 싶습니다!! (2024년 10월 8일 화요일) - 작성자: 스랭킷 (사진 7)
리뷰 2: 거래처 근처에 새로 들어와서 가봄. (지난주 오픈날 재료소진 떠서 입구컷 당함…) 닭개장과 닭곰탕 하나씩 시켜서 서로 조금씩 맛봤는데 둘 다 진짜 훌륭했다. 간판에 적힌 대로 닭곰탕은 진짜 깊고 담백하고 닭개장은 딱 좋게 맵고 칼칼함. 무엇보다 양이 진짜 푸짐. 밥 말아서 떠먹는데 다른 국밥집은 숟가락에 올라간 양이 밥>건더기인데 여긴 밥<건더기 ㄷㄷㄷ 깍두기도 어떤 데는 너무 달던데 단맛 덜헤서 좋았고 백김치는 깔끔해서 매운 닭개장이랑 잘 맞았다.국밥집인데도 실내가 쾌적하고 직원분들도 친절하셔서 좋았다. (2024년 8월 29일 목요일) - 작성자: 치즈달9 (사진 11)
리뷰 3: 계육볶음이란 메뉴가 궁금해서 주문해봣어요. 매콥하면서 닭갈비느낌도 나고 깻잎향이 확 나서 술안주로 딱이엇어요! 닭곰탕은 담백하고 닭고기가 진짜 많아서 배불럿어요 ㅎㅎㅎ 또 오고싶네요 (2024년 9월 4일 수요일) - 작성자: boo**** (사진 9)
리뷰 4: 오픈일 점심에 바로 가서 먹어본 닭개장도 칼칼하면서도 집에서 끓인 것처럼 건강한 맛이었는데, 어제 지인들과 저녁에 방문해서 먹은 닭도리탕도 맛있었고(단짠이 아니어서 진짜 좋았음!) 특히 처음 보는 닭전이 완전 취향이었습니다. 통다리살로 구운 전인데 기대했던 맛! 양념장도 진짜 맛있었고요. 가까운 곳에 생겨서 자주 방문할듯! 사진 올리면서 보니까 닭전 또 먹고 싶네요^^ 점심 저녁 직원분들도 모두 친절하셨어요. 매장도 깔끔합니다. (2024년 8월 27일 화요일) - 작성자: 

In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%AC%B8%ED%99%94%EC%8B%9C%EB%AF%BC/place/1442991337?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 문화시민 서울
업종 카테고리: 카페,디저트
별점: 방문자 리뷰 382
방문자 리뷰: 874
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 272


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 디저트 오마카세는 처음이었는데, 색다르고 맛있었어요!!
생전 처음먹어보는 맛인 디저트도 있었는데 ㅋㅋ 재밌었어요
다음 오마카세는 다른 종류라던데 바뀌면 또 가보고싶어요 (2024년 11월 10일 일요일) - 작성자: Liz4432 (사진 50)
리뷰 2: 테마에 영감을 받은 독특한 디저트들을 먹어볼 수 있었어요 디저트오마카세라는 것도 처음 접해보았는데 데이트 하러 오기도 좋아요 특이하고 처음보는 디저트 체험해보고 싶다면 꼭 방문해볼만해요😆😆 (2024년 8월 23일 금요일) - 작성자: 힝구37 (사진 99)
리뷰 3: 한시간 동안 태국 여행하고 온 느낌이었어요!! 디저트 하나하나가 맛과 식감이 휼륭하고 너무 강하지 않아 재료들 맛이 다느껴지고 발란스가 좋더라구요!! 티도 향이 너무 좋았어요!! 한시간 동안의 행복이었습니다!! (2024년 8월 21일 수요일) - 작성자: 난난의일상 (사진 162)
리뷰 4: 디저트 오마카세라니 .. 특별한 날 힐링하고 갑니다! 예측가능하지 않은, 신선한 맛들의 향연~~ 하나 하나 정성들여 만드시는 모습에 감동ㅎㅎ 다음에 또 갈 수 있길🫶 (2024년 9월 14일 토요일) - 작성자: 올리뷰영 (사진 1,170)
리뷰 5: 오마카세 디저트의 구성이 다채롭고 너무나 맛있었어요. 입안에서 바삭거리던 머랭쿠키도 향긋한 부추오일과 어우러지는 칠리아이스크림도 인상적이었어요.

보기만해도 예쁘고 귀여워서 선뜻 먹기가 아깝기도 했답니다. (2024년 9월 11일 수요일) - 작성자: 의록 (사진 53)
리뷰 6: 급 맛있는 디저트가 먹고싶어서 방문했는데 매우 만족!!!
팟타이 컨셉으로 진짜 고기가 들어간 디저트라니..부추맛과 생강맛이 나는 크럼블까지 문화충격인데 존맛탱
가게가 넓진 않지만 평일에 가니 아늑하고 좋았

리뷰 55: 저번에 갔을 때는 디저트 코스 시즌 8이었는데요
딱 시즌 10이 시작된 날! 일부러 맞춰서 잽싸게 다녀왔어요
제가 감을 정말 좋아해서 (홍시 연시 단감 대봉 다 환장함)
스타터부터 완전 취향 저격. . 슈톨렌 마무리까지 완벽했어요

구움과자가 워낙 맛있는곳이라 참을 수 없어서
구움과자까지 추가해서 먹었답니다 . .
문화시민은 항상 사랑이에요 하트 (2023년 11월 1일 수요일) - 작성자: 계리계리 (사진 3,878)
리뷰 56: 이쁘고 맛있는 디저트가 다양하게 있어요! (2024년 4월 30일 화요일) - 작성자: 별나라미소년 (사진 454)
리뷰 57: 나른한 오후에 들른 『문화시민 서울』
디저트를 즐기는 분들이라면 이미 알고 계실, 예술같은 디저트 오마카세를 만날 수 있는 곳이에요.

시즌별로 달라지는 메뉴 구성을 즐길 수 있어서 시즌별로 방문하는 재미도 있을것 같네요.
시그니처 음료인 질소 플랫치노도 정말 맛있었지만 오마카세와 페어링하기에는 깔끔한 티가 좋을 것 같아요.

코스메뉴는 총 4가지로 구성되어있는데, 처음부터 끝까지 이어지는 느낌의 구성이라 뭐 하나 빼놓을 것이 없구나,, 싶더라구요. 덕분에 오랜만에 만난 지인과 근사한 시간을 낼 수 있어서 좋았습니다 : ) (2023년 10월 18일 수요일) - 작성자: noocar (사진 248)
리뷰 58:  (2024년 6월 6일 목요일) - 작성자: 코스모스7410 (사진 963)
리뷰 59: 크리스마스, 연말에 여사친들 옹기종기 모여서 힐링하기 딱이에요 눈으로 보눈 것도 즐겁고 달고 고소하고 새콤 달콤하게 골고루 디저트 구성이 잘되어 있어요! 그리고 특히 매니저님이 다정하시고 설명도 잘 해주시고 (잘 생기심) 다이어트와 건강까지 챙겨주십니다 메뉴 바뀔 때쯤 한번 더 힐링하러 꼭 다시 갈께요! (2023년 11월 28일 화요일) - 작성자: 알뽀또 (사진 36)
리뷰 60: 제 취향이던 메뉴 2가지 골라봤어요!ㅎㅎ (2024년 2월 15일 목요일) - 작성자: iai**** (사진 17

로드된 리뷰 개수: 120, 수집된 리뷰 개수: 110
리뷰 111: 디저트 맛집이고 친절하시고 엄청 예쁩니닷
근데 전체적으로 달아요
가성비는.. 음 뭐 늬낌맛집이니까! (2023년 7월 23일 일요일) - 작성자: 미친스리 (사진 721)
리뷰 112: 솔직히 기대 안 하고 갔는데 아주 만족스러웠어요. 스타터의 바질 맛은 바질 광인인 저로서 조금 아쉬웠지만... 그 후에 나온 페스츄리 디저트가 미친놈이었습니다. 얇게 카삭거리는 수제 감자칩 너무 맛있어요. 복숭아 디저트는 여름에 어울리게 시원하고 상큼했습니다. 휘낭시에도 생각보다 밀도 있고 쫀쫀해서 좋았어요. 코스 바뀌면 또 방문하고 싶어요~ (2023년 8월 7일 월요일) - 작성자: Kimpoke (사진 164)
리뷰 113: 디저트오마카세 첫데뷔하는 동생이 대만족 했어요♡ 최애들 생일이라 신경쓰고싶어서 해피벌스 플레이팅까지 너무완벽해서 최고의생일♡ 옥수수좋아하는 동생이 옥수수스쿼시
는 특히 100번 먹어야된다고 했어요! 친절하시고 작은공간에서 엄청 힐링했네요💓

평가: ★★★☆☆ (2023년 3월 27일 월요일) - 작성자: 갔던집또안가는리뷰어 (사진 2,804)
리뷰 114: 특별한 날 디저트 오마카세 경험해보기 좋아요 (2023년 9월 23일 토요일) - 작성자: 지9075 (사진 84)
리뷰 115: 디저트 짱맛! 포장도 정성스러워서 선물하기 좋아욭 (2023년 10월 23일 월요일) - 작성자: 바콜브 (사진 502)
리뷰 116: 차도 맛있구 디져트도 특색있어서 특별한 날 가기 좋아요☺️ (2023년 10월 20일 금요일) - 작성자: azre (사진 46)
리뷰 117: 너무 너무 예쁘고 맛있고 새로웠어요! 다양하고 고급진 디저트는 쉽게 접하기 어려운데 경험할 수 있어서 좋았습니다 😆

특히 메인 디저트가 너무 맛있었습니다!! 참외를 메인으로한 디저트인데 한입에 다같이 넣었을때 완전 행복했어요...🥰

가게 분위기도 조용하면서 분위기있고! 셰프님들도 완전 친절하시고 설명 듣는 재미도 쏠쏠 했어

리뷰 168: 정말정말 만족스러웠던 오마카세
코스 하나하나마다 사장님이 정성스럽게 플레이팅하고 설명해주시는데 이런 조합이 이렇게 잘어울린다고? 싶을 정도로 색다른데 심지어 맛도 있어.
마지막에 어느게 가장 좋았고 어느게 아쉬웠는지 꼼꼼하게 체크하시는 모습까지 완벽..
얼마나 연구하고 디저트에 진심인지 알 수 있어서 정말 좋은 시간이었어요.
시즌마다 계속 방문하고 싶은 집. (2022년 10월 3일 월요일) - 작성자: jhr99116 (사진 6,957)
리뷰 169: 제가 애정하는 카페에요💗🫢 (2023년 4월 13일 목요일) - 작성자: hiluv (사진 180)
리뷰 170: 너무너무너무너무너~~~~무 친절하시고
디저트도 맛있고 커피도 맛있고
들기름커피는 단짠꼬소하고
저번시즌 반응 좋아서 단품으로 나왔던 버터어쩌구 진짜 꼬소하고 꼬숩고 서비스로 주셨던(저번시즌 거라 소진될 수 있음주의) 얼그레이 아이스크림은 얼그레이 그 잡채, 그리고 친구가 먹었던 대파아이스크림음 진짜 충격적인 맛ㅋㅋㅋㅋㅋ 대파향이 나는데 맛있어서.. 그리고 가장 좋았던 건 상큼한 레몬소르베…. 계속 생각나는 맛ㅠㅠㅠㅠ 다음 시즌에듀 가고 시퍼여✨ (2022년 10월 7일 금요일) - 작성자: 30387 (사진 269)
로드된 리뷰 개수: 180, 수집된 리뷰 개수: 170
리뷰 171: 오.. 찐맛탱이에여 필링도 알차구 계산 해주시는 분도 친절스 (2023년 4월 16일 일요일) - 작성자: dozi (사진 2,284)
리뷰 172:  (2023년 5월 14일 일요일) - 작성자: clzls (사진 345)
리뷰 173: 넘맛ㅅ있었어요 (2023년 4월 28일 금요일) - 작성자: min**** (사진 224)
리뷰 174: 롯데 팝업스토어로 사먹어봤는데 맛잇었어요. 마들렌이 조금 달아서 아메리카노랑 먹으면 잘 어울릴거같아요 (2023년 4월 8일 토요일) - 작성자: SJ9145 (사진 40)
리뷰 175: 3만원에 이 퀄리티면 후회 안 하실 겁니당 (2023년 7월 27일 목요일) -

리뷰 243: 맛있어요 (2023년 4월 29일 토요일) - 작성자: 투수니13 (사진 4)
리뷰 244: 구움과자 마들렌 휘낭시에는
여기가 제일 맛있는 집이네요.
제가 먹었던 구움과자중에 상큼한 필링이라니
넘버 원 마들렌 맛집 (2022년 7월 1일 금요일) - 작성자: 산타루치아30 (사진 2,627)
리뷰 245: 평범하지 재료들로 어울리는 맛을 내서 너무 좋았어요. 디저트 좋아하는 분들에게 강추입니다. (2022년 4월 19일 화요일) - 작성자: 기가지닝 (사진 1,609)
리뷰 246: 마들렌 진짜 맛있어요 (2023년 4월 9일 일요일) - 작성자: thd**** (사진 261)
리뷰 247: 사장님이 친절하시고요!
들기름 커피 한번 마셔봤는데 맛있어서 또 왔어요 ! 오늘은 다행스럽게도 마들렌이 있어줘서 맛있게 먹었어요 엄청 부드러워요! 여기 맛집 등록 (2022년 6월 3일 금요일) - 작성자: Hyo225 (사진 187)
리뷰 248: 디저트는 달아야한다는 분들에게 최고👏🏻 (2022년 7월 4일 월요일) - 작성자: 사코라 (사진 101)
리뷰 249: 점심시간쯤 맞춰서오니, 휘낭시에가 많아서 좋더라고요! 일찍 온 보람있어요 ㅎㅎㅎ 맛있습니다! (2022년 6월 5일 일요일) - 작성자: Hyo225 (사진 187)
리뷰 250: 들기름 커피 너무 생소하지만
한입 마시는순간 정말 새로운 경험이에요
그리고 맛있어요 들기름의 고소함이 정말 잘 어울려요 (2022년 8월 5일 금요일) - 작성자: 워나82 (사진 35)
로드된 리뷰 개수: 260, 수집된 리뷰 개수: 250
리뷰 251: Soso (2023년 5월 5일 금요일) - 작성자: 100**** (Unknown)
리뷰 252: 마들렌이 맛있어요 (2022년 10월 2일 일요일) - 작성자: 쿠오 (Unknown)
리뷰 253: 마들렌 맛집!! 마들렌에 들어가는 크림이 정말 맛있어요 굳굳 (2022년 7월 30일 토요일) - 작성자: 송이송이초코랄라 (Unknown)
리뷰 254: 특별한 

In [48]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EA%B3%B0%EB%B0%94%EB%A1%9C%EA%B3%B0%ED%83%95/place/1148852390?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 곰바로곰탕
업종 카테고리: 한식
별점: 4.62
방문자 리뷰: 3,773
블로그 리뷰: 485
리뷰 탭 클릭 완료
총 리뷰 수: 3332


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 강남역 국밥 곰바로곰탕 왔어요 !! 국물이 진하고 맛있어요!! 몸보신 하는것처럼 땀 흘리면서 먹고 왔답니다 ~ 연예인들도 많이 찾는 맛집으로 고기가 한우이고 크게 잘 나와요!! 게다가 엄청 부드럽습니다!! 가게 바로 옆쪽으로 주차도 가능하니 편했습니다!! 유튜브 장사의 신에 출연했었는데 지금도 매장에 계시고 엄청 친절하세요!! 점심 식사로 방문해보시는것도 완전 강추입니다!! 뜨거울 때도 맛있고 식어도 맛있어요!! (2024년 10월 26일 토요일) - 작성자: 포포이동 (사진 917)
리뷰 2: 역삼동 맛집
강남역점심맛집
곰탕 개맛집
육수도 엄청 맛있지만 안에 들어간 수육도 엄청 부드럽고 부위도 다양하다
진심 개존맛 (2024년 11월 7일 목요일) - 작성자: 문라잇선데이 (사진 8,693)
리뷰 3: 개운하고 맛있는 곰탕맛집 곰바로~~
육수와 쫄깃하고 부드러운 국내산 한우 조합이 제대로임ㅎㅎ
맛보니까 한우곰탕 맛집 바로 인정되네여!!
육수 향이 너무 좋고 맛도 좋아용
보통시켰는데 양 엄청 많아서 배부르게 먹었어여!
엄청 친절하시구 잘 챙겨주셔서 기분좋게 식사했답니당 ㅎㅎ
부모님께서도 딱 좋아하실 거 같아서 같이 오고싶네용
재방문 100% (2024년 10월 26일 토요일) - 작성자: 구르르미 (사진 411)
리뷰 4: 항상 만석이라 벼르다 벼르다 1년만에 다시왔습니다!
근처 곰탕중 최고구요. 서비스로 만두 주시는데 만두별로 안좋아하는 딸도 너무 맛있다 합니다.
당연 너무 친절하시구요 다음에 또 갈것 같습니다! ㅎ (2024년 11월 5일 화요일) - 작성자: 점심 맛집 (사진 751)
리뷰 5: 모든 메뉴가 맛있어요 ㅋㅋㅋㅋ심지어 만두 깍두기까지 맛잇어요.. 국밥은 당연하고 만두진짜 맛나니 꼭 드셔보세요 고기도 부드럽고 양도 많고 수육의 고기는

리뷰 48: 너무 맛있게 먹었습니다~~ 다들 엄청 감탄했어요!
사장님 친절하게 대해주셔서 더 맛있게 먹었어요 :)

유튭 보고왔는데 기대했어도 기대완전 충족됐어요! (2024년 11월 12일 화요일) - 작성자: ji_**** (사진 14)
리뷰 49: 재방문 역시나 건강한 맛
사장님 덕분에 건강해졌어요^^ (2024년 10월 15일 화요일) - 작성자: 하얀곰3696 (사진 85)
리뷰 50: 유튜브에서 보고 가야지 하다가 이제야 방문했어요!
김치랑 석박지 둘 다 맛있었고
고기도 엄청 두껍고 크게 많이 들어가있습니다!!
만두도 서비스로 주십니다!
친절하게 응대해주셔서 행복하게 식사했습니다
감사합니다!! (2024년 10월 12일 토요일) - 작성자: xuxxn30 (사진 21)
로드된 리뷰 개수: 60, 수집된 리뷰 개수: 50
리뷰 51: 일단 내부도 너무 깨끗하고 모든 직원들이 너무너무 친절하십미다ㅠㅠ
음식도 너무너무 양많고 맛은 두말할것도 없어요
역시 사람많은곳은 이유가 있네요.
한우고기곰탕인데 가격도 너무너무 착합니다! (2024년 9월 7일 토요일) - 작성자: 콜라3755 (사진 3)
리뷰 52: 직원분들 너무 친절하시고 김치랑 곰탕 너무 맛있어요! 11-15시는 소면도 무제한이라 넉넉하게 먹을 수 있습니다 ㅎㅎ 또 방문하고 싶은 곳 😍 (2024년 10월 24일 목요일) - 작성자: zip무새 (사진 22)
리뷰 53: 벌써 N번째 방문이에요.
국물 고기 김치 맛이..👍🏻
음식도 빨리나오고 깔끔해서 퇴근 후 식사나 친목으로 방문하기 좋습니다😍 (2024년 11월 4일 월요일) - 작성자: 쩡훈엄마55 (사진 27)
리뷰 54: 매장에 들어오면 사장님께서 반갑게 인사 해주십니다.
거기서부터 맛있습니다.

고기가 참 부드럽고 맛이 좋습니다.
국물이 깊고 진합니다.
상황버섯과 마늘이 우러나서
특유의 국물맛이 있습니다.
저는 맛있게 잘 먹었습니다.

매장에서는 조용한 미야쟈키 감독 OST들 너무 좋았습니다. 전체적으로 매장 분위기가 편안하고 좋습니다.


리뷰 108: 배고플때 제일먼저 생각나는 곰바로입니당! 추석연휴 준비하기위해 몸보신하러 주말에 들렀습니당 ㅋㅋ 다들 곰바로먹고 건강해지세요! (2024년 9월 14일 토요일) - 작성자: 릿사 (사진 28)
리뷰 109: 강남에 약속이 있어 오면 생각나는 맛집입니다
항상 먹는 한우곰탕~~ 후회없습니다 (2024년 10월 19일 토요일) - 작성자: ssony1092 (사진 5)
리뷰 110: 오랜만에 친구와 함께 강남 곰바로곰탕을 방문했습니다. 🐮 이곳은 한우 곰탕 전문점으로 유명한데, 특히 상황흑마늘 한우곰탕이 이색적이라 주문해보았습니다. 🍲 깊고 진한 한우 곰탕의 맛에 상황버섯과 흑마늘이 더해져 건강하면서도 풍부한 맛이 인상적이었습니다. 🌿 깔끔하고 담백한 국물에 재료의 조화가 뛰어나, 몸도 마음도 든든하게 채울 수 있었습니다. 친구와 함께 오랜만에 맛있는 음식을 나누며 즐거운 시간을 보냈고, 다음에도 다시 찾아가고 싶은 곳입니다. 😊 (2024년 8월 24일 토요일) - 작성자: Yutopapa80 (사진 27)
로드된 리뷰 개수: 120, 수집된 리뷰 개수: 110
리뷰 111: 사무실 근처 일상찐맛집
곰탕 고기 입에서 녹아요 (2024년 11월 11일 월요일) - 작성자: 국민대포 이승엽 (사진 8)
리뷰 112: 따끈한 곰탕 너무 맛있어용!!
벌써 5번 넘게 방문입니다ㅎㅎㅎㅎ
날씨 추워지니까 더 자주 와야겠어요
소면 무한리필인 것도 너무 좋습니당!
반찬도 다 맛있어용 (2024년 10월 2일 수요일) - 작성자: 요오히이 (사진 46)
리뷰 113: 항상 자주 오다가 두어달만에 왔는데
역시는 역시!
진짜 제일 가성비 좋고 맛있는 곰탕입니다
오늘 유독 고기도 신선하고 많았어요
술 마실 생각 없었는데 먹다가 탁주 바로 시킴
계속 장사해주셔서 감사합니다^^
또 올게요!!! (2024년 9월 24일 화요일) - 작성자: 유자차798 (사진 14)
리뷰 114: 워낙에 맛집이고 입소문이난집이라 다른말이 필요없을듯! 강추! 역삼동 맛집이이요! (2024년 11

리뷰 174: 이제까지 먹은 곰탕들중에 가장 맛있는 곰탕
노원에 사는데 여기까지 와서 먹어요ㅠㅠ진짜 너무맛있움 (2024년 9월 21일 토요일) - 작성자: 인생은한방6458 (사진 54)
리뷰 175: 진짜 너무너무 맛있어요. 건강해지는 그런 맛입니다!
국물 맛이 깊어서 속에 쌓인 피로가 모두 풀리는
그런 느낌 입니다 ㅎㅎㅎ (2024년 9월 14일 토요일) - 작성자: 46**** (사진 10)
리뷰 176: 역시 국물이 직접 우려낸거라 엄청 찐하고 진국이네요 (2024년 11월 7일 목요일) - 작성자: geon4411 (사진 13)
리뷰 177: 벌써 세번째 방문중인 곰바로곰탕 입니다!
처음에는 장사의 신 유튜브에서 보고 알게 돼서 방문했다가
맛이 너무 좋아서 가끔씩 곰탕 생각날 때마다 방문하고 있어요!

고기가 국내산 한우라 그런지 잡내 없이 고소한 맛이 일품입니다!

강남역 근처에 오시게 될 일 있으시다면 꼭 방문하셔서
곰탕 한 번 맛보세요! 한 번 맛보고 나면 곰탕 생각날 때마다 여기로 오게 되어 있습니다. (2024년 6월 13일 목요일) - 작성자: foodhwan (사진 666)
리뷰 178: 곰탕도 맛있고, 오징어젓도 맛있고, 왕만두도 맛있고, 고기고 많이 든 역삼동 맛집~~~~~~!!!
곰바로 곰탕! 맛짱 (2024년 10월 29일 화요일) - 작성자: 지로299 (사진 1)
리뷰 179: 너무 맛있어요!!! (2024년 11월 7일 목요일) - 작성자: daniell8282 (사진 15)
리뷰 180: 진짜미친 인생곰탕집.. 고기양 미쳐써요. 무조건 드세요 (2024년 10월 17일 목요일) - 작성자: 구노0708 (사진 21)
로드된 리뷰 개수: 190, 수집된 리뷰 개수: 180
리뷰 181: 수육 만두 둘다 너무 맛있어용🥹🥹 국물도 쥑이고 사장님도 친절하셔서 너무 즐거운 시간 보내고왔습니다 ㅎㅎ💗💗💗 (2024년 8월 20일 화요일) - 작성자: ann**** (사진 154)
리뷰 182: 너무 맛있었어요
국물이깔끔하고 정갈한 한

리뷰 250: 밥먹느라 곰탕은 못찍었네요ㅋㅋ 회사 근처라 점심에 자주와요!! 건강을 생각해서 자주 먹어요!! (2024년 11월 4일 월요일) - 작성자: 박지영5864 (사진 6)
로드된 리뷰 개수: 260, 수집된 리뷰 개수: 250
리뷰 251: 3번째 방문입니다 김치랑 깍두기랑 국밥 만두 그냥 다 맛있어요..! (2024년 11월 6일 수요일) - 작성자: har**** (사진 3)
리뷰 252: 회사 앞이라 자주 오는데 곰탕 국물이 진짜 끝내주고 고기도 실하니 너무 맛있습니다! 소면도 따로 주셔서 넣어서 먹을 수 있어요 ㅎㅎ 저녁에 술 마시러 따로 오고 싶습니다!! (2024년 8월 26일 월요일) - 작성자: hyu**** (사진 30)
리뷰 253: 곰바로곰탕 너무 유명해서 와서 먹었네요
고기도 연하고 너무 맛있습니다
다음에 또 오겠습니다~^^ (2024년 9월 6일 금요일) - 작성자: 안녕하세요8012 (사진 3)
리뷰 254: 곰탕이 먹고 싶어서 찾아왔는데 와보니 이미 놔본적이 있었던곳 !!!
수육이 맛있다고 친구 추천으로 왔었던곳!!!!! 그때도 너무 맛있게 먹었는데 역시는 역시!!!
혼밪 하시는 분들도 많고!!
곰탕 역시 맛있고 건강해 지는 느낌!!!!
국물이 너무 진하고 맛있어요!!!! (2024년 6월 24일 월요일) - 작성자: 306**** (사진 144)
리뷰 255: 정말 맛있게 먹엇었는데 또와서 기대중입니다 (2024년 10월 31일 목요일) - 작성자: hig**** (사진 6)
리뷰 256: 곰탕 너무 맛있어서 또 왔어요!! 국물이 프랜차이즈맛 안나고 찐해서 맛있어요!!! (2024년 9월 12일 목요일) - 작성자: seulseul37 (사진 22)
리뷰 257: 너무 맛있어요!!
처음 방문했는데 고기 너무 맛있고 육수 진짜 진해요~ (2024년 8월 22일 목요일) - 작성자: no**** (사진 38)
리뷰 258: 국물이 찐하고 큰 고기 덩이가 많이 있네요. 김치도 너무 맛있어요. 앞으로 점심 메뉴로 자주 먹을듯~~

...

리뷰 587: 김치랑 국밥이 너무 맛있어서 자주들려요 너무 맛있어요 (2024년 10월 4일 금요일) - 작성자: ils**** (Unknown)
리뷰 588: 국물이 진하고 아주 맛있어요!! 다음에도 올게요 (2024년 6월 25일 화요일) - 작성자: 뀨259 (사진 27)
리뷰 589: 음식에 대한 진심이 느껴져요 (2024년 10월 6일 일요일) - 작성자: moo**** (Unknown)
리뷰 590: 와... 곰탕 먹으러 왓다가 수육에 소주 한잔 햇는데 진짜 찐맛집입니다!!! 🤦 여러 부위가 수육으로 나와서 먹는 재미가 쏠쏠하고 부위마다 식감과 맛이 달라서 재밌는 맛집입니다. 완전 추천!! 👍 (2024년 5월 10일 금요일) - 작성자: Jerryk (사진 188)
로드된 리뷰 개수: 600, 수집된 리뷰 개수: 590
리뷰 591: 국물이 뜨끈하니 너무 맛있어용!!
스트레스가 풀리는 맛이였습니당 (2024년 10월 4일 금요일) - 작성자: yunghoih (사진 1)
리뷰 592: 너무 맛있어요 ㅎㅎ 등산후 꿀맛! (2024년 6월 16일 일요일) - 작성자: 사당동날다람쥐 (사진 686)
리뷰 593: 맛집이라고해서 찾아가서 먹음
서비스만두도 주심 좋아요 가격은 좀 비쌈 (2024년 9월 3일 화요일) - 작성자: 시아 버지 (사진 4)
리뷰 594: 맛있게 잘 먹었습니다! (2024년 10월 31일 목요일) - 작성자: 길3624 (사진 1)
리뷰 595: 맛있어요😆 집 가까워서 자주와요~ (2024년 8월 8일 목요일) - 작성자: 홍엽72 (사진 2)
리뷰 596: 동네 곰탕맛집입니다
양도 많고 맛도 좋습니다 (2024년 10월 2일 수요일) - 작성자: jun**** (Unknown)
리뷰 597: 매일 전통 초대형 무쇠가마솥에서 정성들여 만드는 보양곰탕 맛집 곰바로곰탕😊
국내산 한우를 사용하여 매일 전통 가마솥에서 장시간 걸쳐 소고기와 신선한 야채를 정성껏 삶아 우려내 고소하면서도 깔끔한 맛이 일품이 곰탕 맛집😊 곰탕 육수, 진한 국

리뷰 669: 맛있어서 자주 오게돼요 양도 많아서 항상 배부르게 먹고 갑니다 (2024년 9월 14일 토요일) - 작성자: Annavvv (사진 5)
리뷰 670: 먼저 음식은 너무 깊고 맛있어서 생략!
친절하시고 손님들 식사에 방해되지않게
음식 나오는 종은 살짝살짝 눌러주시고
손님이 입,퇴시 어서오세요,감사합니다는 아주 우렁차게!! 너무 인상깊었습니다.
다음에 또 오겠습니다.
사장님 잘 먹고갑니다!👍👍 (2024년 5월 12일 일요일) - 작성자: cla**** (사진 19)
로드된 리뷰 개수: 680, 수집된 리뷰 개수: 670
리뷰 671: 소면있고 김치 존맛이에요 주말 점심에왔는데 바로 입장 가능했습니다!! (2024년 9월 21일 토요일) - 작성자: 사람9194 (Unknown)
리뷰 672: 접근성이 좋고 맛있습니다 (2024년 9월 21일 토요일) - 작성자: 콜트권총68 (Unknown)
리뷰 673: 너무너무 친절하시고 맛있습니다 (2024년 9월 21일 토요일) - 작성자: asd**** (사진 1)
리뷰 674: 고기양도 많고 맛있네요 ㅎㅎ 보양하고 감사합니다! (2024년 8월 15일 목요일) - 작성자: 스페이스 메이커 (사진 3)
리뷰 675: 음식이 너무 맛있고 사장님 서비스가 좋아서..일부러 영수증 나눠서 달라고 요청했습니다!
너무 맛있고 항상 술마셔도 생각나고 해장하고싶을때도 생각나요..사장님!! 오래 영업해주시길 항상바래요!! (2024년 7월 11일 목요일) - 작성자: rem**** (사진 1)
리뷰 676: 김치부터 먹어보았는데 이 집 맛집이다 싶었어요ㅋㅋ
곰탕 나오는데 고기냄새가 너무 좋았어요!! 너무 마시따 (2024년 7월 1일 월요일) - 작성자: YuY20 (사진 4)
리뷰 677: 맛있게 잘먹었습니당. 고기가 부드러워서 좋아요! (2024년 7월 17일 수요일) - 작성자: hooning76 (사진 2)
리뷰 678: 세상에 너무너무 맛나요
울덩내 곰탕맛집은 요기다!!!!
자주올게요~~^^* (2024년 10월

리뷰 756: 너무 맛있어요 항상 야근하면 오는 곳입니다 (2024년 6월 20일 목요일) - 작성자: 유뜰 (사진 23)
리뷰 757:  (2024년 10월 22일 화요일) - 작성자: 이쁜언니95 (사진 2)
리뷰 758: 뜨끈한 곰탕으로 든든하게 점심 식사했습니다! (2024년 8월 12일 월요일) - 작성자: 진07 (사진 22)
리뷰 759: 장사의신땜에 왔는데 정말 친절하시고 정말 맛있네용! 😭🫣👍 (2024년 7월 5일 금요일) - 작성자: asasiii (사진 2)
로드된 리뷰 개수: 770, 수집된 리뷰 개수: 759
리뷰 760: 정말 맛있습니다. 가격도 쌉니다. (2024년 8월 6일 화요일) - 작성자: 강효상24 (사진 3)
리뷰 761: 운 좋게 대기 없이 바로 들어왔어요!
맛있게 잘 먹었어요! (2024년 8월 6일 화요일) - 작성자: jis**** (사진 1)
리뷰 762: 집 근처 곰탕 완전맛집~ (2024년 9월 30일 월요일) - 작성자: Yoon4023 (사진 1)
리뷰 763: 매번 오지만 항상 너무 맛있게 먹고가요!
깔끔하게 맛잇어용 (2024년 7월 3일 수요일) - 작성자: 빵삐 (사진 47)
리뷰 764: 남자친구가 너무너무 궁금해해서 왔는데!! 국물 먹자마자 둘다 감탄했어요..너무 맛있어요!! (2024년 5월 2일 목요일) - 작성자: 핼1141 (사진 63)
리뷰 765: 국물이 시원하고 맛있습니다! (2024년 9월 7일 토요일) - 작성자: swo123 (Unknown)
리뷰 766: 맛있는집
친절한집
한결같이 맛있게 먹고 있어요^^ (2024년 7월 24일 수요일) - 작성자: wildwolf (사진 4)
리뷰 767: 오늘도 맛있네요~ 잘먹고가요! (2024년 7월 22일 월요일) - 작성자: jkm610 (사진 12)
리뷰 768: 항상 너무 맛있어요👍🏻👍🏻❤️❤️ (2024년 7월 18일 목요일) - 작성자: kim**** (사진 8)
리뷰 769: 무슨 말이 필요해요???
회사 앞 단골이구요!
가

로드된 리뷰 개수: 860, 수집된 리뷰 개수: 849
리뷰 850: 여름에 몸보신으로 곰탕 찾았는데 너무 맛있어요~~
또 오고 싶은 맛 입니다!! (2024년 7월 20일 토요일) - 작성자: gks**** (사진 1)
리뷰 851: 매운갈비찜 흑마늘도 특이하고 맛있어요! 갈비탕 육수와 소면도 나와서 좋아요 (2024년 6월 29일 토요일) - 작성자: 뚜이41 (사진 2)
리뷰 852: 국밥은 너무나 당연하게도 정말 맛있는 국밥입니당 첨 먹었을때 너무 맛있구 강남 올때마다 생각나서 두번째 방문으로 왔습니당 사장님 너무 친절하시두 국밥 퀄리티 넘 좋아요~ 서비스로 주신 만두와 콜라도 넘 잘 먹었습니당 최고에여 👍 번창하세요 🫶 (2024년 5월 7일 화요일) - 작성자: ah16 (사진 7)
리뷰 853: 점심시간에 자주 애용해요 맛있어오 (2024년 7월 2일 화요일) - 작성자: 민댕이42 (사진 14)
리뷰 854: 음식도 맛있고 친절하세요 (2024년 8월 21일 수요일) - 작성자: sob**** (Unknown)
리뷰 855: 사장님도 친절하시고 음식도 맛있습니다! (2024년 8월 21일 수요일) - 작성자: jjh**** (사진 11)
리뷰 856: 김치도 너무 맛있고 수육이 두껍고 맛나요 (2024년 7월 24일 수요일) - 작성자: jy**** (사진 3)
리뷰 857: 곰탕 중에서도 깔끔한 국물 좋아하는데 여긴 정말 찐 고기로 진하고 맑게 우려낸 국물 맛이예요! 고기도 너무 부드럽고 맛있습니다 (2024년 7월 19일 금요일) - 작성자: jjo**** (사진 12)
리뷰 858: 국물도 진하고 따끈해요
김치도 적당히 달고 먹을만해요 (2024년 7월 15일 월요일) - 작성자: GOS걸린아이폰 (사진 2)
리뷰 859: 국물도 진하고 고기가 넘 부드럽고 맛있어요!!
제대로 보양했어요~~:) (2024년 7월 29일 월요일) - 작성자: 갱이8763 (사진 25)
로드된 리뷰 개수: 870, 수집된 리뷰 개수: 859
리뷰 860: 맛있어

리뷰 942: 고기도 튼실하고 든든하니 하루종일 배가 안고플 것 같아요!
😎 (2024년 7월 8일 월요일) - 작성자: 서한결36 (사진 1)
리뷰 943: 국물이 진하고 맛있어요 ~~ 사장님도 친절하셔서 재방문의사 있음!! (2024년 6월 29일 토요일) - 작성자: pof**** (사진 3)
리뷰 944: 김치 깍두기 곰탕 다 너무 맛있어요 강추 ❤️ (2024년 6월 29일 토요일) - 작성자: 송민희1659 (사진 8)
리뷰 945: 최근 국밥이 너무 땡겨서 방문했는데 정말정말 맛있고 좋았어요! 기본적인 양도 많은데 소면도 점심시간에는 무제한 제공이라 배부르게 먹고 왔어요~ (2024년 5월 31일 금요일) - 작성자: Junbeom7 (사진 4)
리뷰 946: 곰탕이 뜨끈하고 양이많습니다. 소면사리도 좋아요 (2024년 7월 2일 화요일) - 작성자: IDHUN (사진 5)
리뷰 947: 맛있어요~ (2024년 9월 23일 월요일) - 작성자: sophie52 (사진 231)
리뷰 948: 맛있게 잘 먹고 갑니다!
고기가 너무 실하고 마시써요!!! (2024년 6월 25일 화요일) - 작성자: haribo37 (사진 17)
로드된 리뷰 개수: 960, 수집된 리뷰 개수: 948
리뷰 949: 맛있어서 자주 와요😀
오늘도 한 그릇 뚝딱입니다!! (2024년 6월 25일 화요일) - 작성자: freudig3 (사진 15)
리뷰 950: 진짜 맛있어요 (2024년 8월 28일 수요일) - 작성자: 호로롤로로롤 (사진 9)
리뷰 951: 고기가 엄청 부드럽고 국물도 다른 곰탕이랑 다르게 특별했습니다.너무 맛있게 잘 먹었습니다. (2024년 6월 10일 월요일) - 작성자: JS4179 (사진 6)
리뷰 952: 고기엄청 순하고 야들야들 닭고기같은 식감느낌처럼 부드러워요
냄새안나고 맛있어요
곰탕좋아해서 찾아왔는데 너무맛있어요!!! (2024년 3월 17일 일요일) - 작성자: 소리244 (사진 62)
리뷰 953: 너무 좋아요!! (2024년 10월 18일

In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%B9%B8%EA%BC%AC%EC%8B%9C/place/1084480642?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 칸꼬시
업종 카테고리: 맥주,호프
별점: 3.77
방문자 리뷰: 42
블로그 리뷰: 4
리뷰 탭 클릭 완료
총 리뷰 수: 12


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 편안하게 술 마시기 좋아요 (2023년 10월 11일 수요일) - 작성자: 열나요더워서 (사진 196)
리뷰 2: 또또또간집 ㅠㅠ 너무 맛있는데 평점이 낮아서 리뷰 계속 올려보겠읍니다...! 교촌보다 더 맛있어서 기절ㅎㅎ 그래서 치킨 맨날 품절... 사장님들도 최고 친절해용! 갈때마다 감동 이빠이ㅠㅠ 알라뷰입니다!!💕 (2023년 4월 26일 수요일) - 작성자: 화이팅9840 (사진 3)
리뷰 3: 가성비 굿 (2023년 6월 12일 월요일) - 작성자: 아나킨1213 (Unknown)
리뷰 4: 가성비 좋아요 음식 맛있어요 (2022년 12월 2일 금요일) - 작성자: 열나요더워서 (사진 196)
리뷰 5: 맛나요 (2023년 3월 27일 월요일) - 작성자: jeongic0210 (팔로워 4)
리뷰 6: 긋 (2022년 12월 31일 토요일) - 작성자: 곰돌부 (사진 14)
리뷰 7: 굳 (2022년 12월 6일 화요일) - 작성자: 뭐임뫄어쩔 (사진 284)
리뷰 8: 치킨이 맛있어요^^ (2022년 12월 16일 금요일) - 작성자: hah**** (사진 10)
리뷰 9: 음식이 맛나요.
가성비가 좋아요. (2022년 1월 24일 월요일) - 작성자: 이우현LeeWooHyun (사진 16)
리뷰 10: 좋아요 (2021년 11월 3일 수요일) - 작성자: won**** (Unknown)
로드된 리뷰 개수: 12, 수집된 리뷰 개수: 10
리뷰 11: 굿 (2021년 5월 11일 화요일) - 작성자: 수연2344 (Unknown)
리뷰 12: 노가리 맛있어요~~ (2020년 8월 18일 화요일) - 작성자: chs**** (사진 1)
'더보기' 버튼이 더 이상 없습니다.
총 12개의 리뷰를 수집했습니다. (예상 총 리뷰 수: 12)
리뷰 데이터가 '칸

In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EB%B4%89%ED%8F%89%EC%B0%A9%ED%95%9C%EB%A9%94%EB%B0%80/place/37491529?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 봉평착한메밀 본점
업종 카테고리: 한식
별점: 4.04
방문자 리뷰: 463
블로그 리뷰: 58
리뷰 탭 클릭 완료
총 리뷰 수: 164


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 점심시간을 지나서 갔더니 한바탕 직장인들의 점심 전쟁이 휩쓸고간 웬지 모를 썰렁함이 느껴졌어요. 그래서 그런건지, 기분탓인지 몰라도 전에 갔을때 먹었던 맛과 정성이 느껴지지 않았어요. ㅜㅜ 시끄럽고 정신없어도 점심때 가서 따끈하게 준비해놓은 음식을 먹어야하려나봐요. (2024년 8월 7일 수요일) - 작성자: 민작가 (사진 486)
리뷰 2: 밑반찬이 저렇게 나오구요.
주문한 음식은,
메밀 수제비
메밀 들께 수제비
비빔국수
보리밥
그리고 메밀전
아주 배부르게 먹고 왔습니다.
매우 근강한 맛이에용!!
사람 많아서 기다리는 사람들도 있습니다. (2024년 3월 18일 월요일) - 작성자: idisjien94 (사진 1,088)
리뷰 3: 메밀로 만든 수제비라서 건강에 좋을 것 같아요.
서비스 하시는 분들이 친절해요 (2024년 4월 19일 금요일) - 작성자: bubbleno1 (사진 823)
리뷰 4: 비빔막국수 주문했습니다
메밀을 직접 뽑아내는 거다보니까 소요시간이 다소 걸려요, 그만큼 건강한 맛 나는거겠죠??
근데 양념이 너무 쎄서 메밀 맛이 안느껴지더라구요 ㅠㅠ
반찬은 깔끔하고 장조림(계란) 나오니 괜찮았으나 기본 반찬의 국은 안주셨어요 ㅜ (2024년 1월 15일 월요일) - 작성자: 산딸기갱 (사진 528)
리뷰 5:  (2024년 4월 19일 금요일) - 작성자: 푸른날개21 (사진 183)
리뷰 6:  (2024년 3월 28일 목요일) - 작성자: ho**** (사진 140)
리뷰 7: 수제비에 밥한공기 말아먹음 아주 든든해요..들깨수제비도 맛있어요! (2024년 1월 16일 화요일) - 작성자: pesu76 (사진 244)
리뷰 8: 요즘은 겨울이라 들깨수제비먹으러 갔네요. 사람많은 맛집입니다. 건강한맛이에요. (2024년 1월 26일 

리뷰 98: 좋음 (2022년 1월 5일 수요일) - 작성자: 밥반찬89 (사진 113)
리뷰 99: 맛나요 (2021년 12월 26일 일요일) - 작성자: 밥반찬89 (사진 113)
리뷰 100: 맛있어요 (2022년 1월 10일 월요일) - 작성자: 잠만보927 (Unknown)
로드된 리뷰 개수: 110, 수집된 리뷰 개수: 100
리뷰 101: 굳 (2021년 12월 18일 토요일) - 작성자: 밥반찬89 (사진 113)
리뷰 102: 좋아요 (2022년 1월 10일 월요일) - 작성자: amw**** (Unknown)
리뷰 103: 굿 (2021년 12월 1일 수요일) - 작성자: bes**** (사진 36)
리뷰 104: 굿굿 (2021년 11월 22일 월요일) - 작성자: oko**** (사진 20)
리뷰 105:  (2021년 10월 31일 일요일) - 작성자: jykim7891 (사진 12)
리뷰 106: 굿 (2021년 12월 11일 토요일) - 작성자: sde**** (Unknown)
리뷰 107: 굿 (2021년 11월 25일 목요일) - 작성자: 3222234 (사진 3)
리뷰 108: 좋아요 (2022년 3월 14일 월요일) - 작성자: youji841 (Unknown)
리뷰 109: 메뉴주문확인 두번을 협박성있게 하시고
메뉴가 한 세번쯤 잘못나옵니다ㅋㅋㅋ
점심12시에 도착하니 메뉴가 30분걸려요... 회사 점심시간에 갈곳은 아닌듯합니다
물론 30분기다려서 먹을 맛은 더더욱아니구요 (2021년 6월 9일 수요일) - 작성자: 받종 (사진 96)
로드된 리뷰 개수: 120, 수집된 리뷰 개수: 109
리뷰 110: 좋아요 (2022년 3월 14일 월요일) - 작성자: 아힌6795 (사진 2)
리뷰 111: 좋아요 (2021년 11월 8일 월요일) - 작성자: 김경26 (Unknown)
리뷰 112: 맛있어요 (2021년 10월 18일 월요일) - 작성자: 잠만보927 (Unknown)
리뷰 113: 근처에서 5년동안 간판만 보았는데
완전

In [54]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%ED%86%B5%EB%8F%BC%EC%A7%80%EB%91%90%EB%A3%A8%EC%B9%98%EA%B8%B0%EA%B9%80%EC%B9%98%EC%B0%8C%EA%B0%9C%EC%A0%84%EB%AC%B8%EC%A0%90%20%EC%97%AD%EC%82%BC/place/35103360?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 통돼지두루치기김치찌개전문점
업종 카테고리: 육류,고기요리
별점: 4.33
방문자 리뷰: 462
블로그 리뷰: 89
리뷰 탭 클릭 완료
총 리뷰 수: 220


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 여기는 진짜 찐 맛집❤️코로나19도 이겨낸 곳입니다😆 김치찌개와 두루치키 그리고 미나리삽겹살이 주메인인데 다 맛있어요! 재료가 다 국내산~~ 개운하니 시원한 찌개는 진짜 예술이에요~ 그리고 사이드메뉴 계란말이도👍🏻👍🏻🎈 (2024년 10월 6일 일요일) - 작성자: Choi8D (사진 1,146)
리뷰 2: 고기도 너무 맛있고 미나리가 너무 고소해서 흡입했습니다 또 올게요 찌개도 완전 맛있거 자주올ㄹ거같아요 ㅎㅎㅎ (2024년 10월 29일 화요일) - 작성자: 대포8996 (사진 91)
리뷰 3: 이모가 친절하시고 냉장고기라서 고기 맛이 꽤나 괜찮아요. 다만 삼겹살의 비계가 조금 많은 게 흠이에요. (2024년 10월 28일 월요일) - 작성자: 있는그대로 솔직후기 냥 (사진 13,463)
리뷰 4: 통돼지두루치기 정말 맛나요ㅎㅎ 육즙 가득한 돼지고기에 매콤한 양념이 어우러져 입맛 당기네요. (2024년 11월 8일 금요일) - 작성자: cry**** (사진 26)
리뷰 5: 아늑한 분위기 속에서 맛있는 두루치기 즐기기 딱 좋네요! 양도 푸짐하고, 정말 든든하게 잘 먹었어요^^ 다음에 또 와야겠다~ (2024년 11월 15일 금요일) - 작성자: jyi**** (사진 15)
리뷰 6: 신선한 재료와 정성이 돋보이는 맛에 감탄했습니다 직원분들의 친절한 서비스도 인상적 ㅎㅎ (2024년 10월 28일 월요일) - 작성자: sca**** (사진 17)
리뷰 7: 친절한 서비스에 감동했어요! 두루치기 양도 푸짐하고 맛있게 잘 먹었습니다~ (2024년 11월 11일 월요일) - 작성자: tab**** (사진 11)
리뷰 8: 매콤하면서도 담백 통돼지두루치기 강추! 🍲 매장도 넓고 쾌적한 분위기라 좋았어요! 😋 (2024년 10월 29일 화

로드된 리뷰 개수: 80, 수집된 리뷰 개수: 70
리뷰 71: 밑반찬도 깔끔하게 잘 나오는데 메인음식 김치두루치는 더 맛있어요. 고기가 두툼하면서 잡내도 없고 부드러워요. 남은 양념으로 밥도 볶아 먹을수 있으니 환상의 조합이에요. (2024년 1월 20일 토요일) - 작성자: for**** (사진 97)
리뷰 72: 고기도 큼직막하게 있고 김치와 같이 먹으면 그것이 바로 환상의궁합 술안주로 진짜 딱 너무 맛있어요 양도 많고! 친구들이랑 왔다 너무 맛있게 먹고갑니다 (2024년 1월 17일 수요일) - 작성자: 넴모세삼 (사진 46)
리뷰 73: 점심식사하러 방문했습니다~ 두루치기 양념이 너무 맛있어서 밥에 쓱쓱 비벼먹었더니 한공기 뚝딱했어요! (2024년 5월 9일 목요일) - 작성자: 7917 (사진 18)
리뷰 74: 회사분들이랑 식사하려고 왔어요!! 고기 냄새도 안나고 김치랑같이 먹는데 너무 맛있어요! 저녁식사하러왔다 술한잔하기도 너무 좋아요! (2024년 1월 17일 수요일) - 작성자: 뜨뜨ㄱ뜨 (사진 34)
리뷰 75: 삼겹살에 미나리 올려먹으니 완전 JMT이에요! 안먹어 보신분들은 꼭 미나리 삼겹살 추천드려요! 계란말이랑 마무리 볶음밥까지 야무지게 먹고 왔습니다 ! (2024년 2월 19일 월요일) - 작성자: 남매맘5959 (사진 9)
리뷰 76: 우연히 들리게된 맛집이였어요! 고기양도 많고 김치 가 맛있으면 음식은 다한거죠~ 고기랑김치랑 같이먹으면 짱짱맛! 집밥같은 느낌! (2024년 1월 16일 화요일) - 작성자: 닫비짱 (사진 54)
리뷰 77: 빠르게 입장했고 맛있네요. (2024년 3월 22일 금요일) - 작성자: mpa**** (사진 7)
리뷰 78: 점심때 김치찌개 먹었는데 김치가 좀 적게 들어있어요 (2024년 2월 14일 수요일) - 작성자: rub**** (사진 416)
리뷰 79: 친구랑 오랜만에 들린곳인데..고기도 두툼해서 씹감도 좋고 비린내도 안나고 묵은지도 넘 자극적이지 않아서 먹기 좋아요 밥두그릇은 기본인거 같아요 

리뷰 150: 김치찌개가 맛있어요 (2022년 11월 12일 토요일) - 작성자: 현아577 (Unknown)
로드된 리뷰 개수: 160, 수집된 리뷰 개수: 150
리뷰 151: 그냥그래요. (2022년 11월 4일 금요일) - 작성자: Chic Seung (사진 66)
리뷰 152: 찌개가 맛있네요 (2022년 12월 25일 일요일) - 작성자: 최지나41 (사진 5)
리뷰 153: 김치찌개는 집에서 먹는 맛이고
계란말이가 예술적으로 푸짐하고 맛있어요 (2022년 8월 21일 일요일) - 작성자: 찐현맘 (사진 15)
리뷰 154: 김치찌게 맛있고 계란말이가 너무 좋네요 (2022년 8월 24일 수요일) - 작성자: 천하의김선달 (사진 14)
리뷰 155: . (2023년 1월 5일 목요일) - 작성자: mok**** (사진 3)
리뷰 156: 김치찌개에 국물이 별로 없어서 아쉬웠습니다. 맛도 조금 아쉽...근데 고기는 많이 들어 있어서 좋았어요. 근데 물먹으려고 컵 봤는데 컵 2개에 입닿는 부분에 김조각 작은게 들러붙어있어서... 실망이었습니다. (2022년 5월 24일 화요일) - 작성자: jinnie123 (사진 58)
리뷰 157: 좋아요 (2022년 10월 25일 화요일) - 작성자: 오드리18 (사진 12)
리뷰 158: 굿 (2022년 10월 25일 화요일) - 작성자: eeet082932 (사진 5)
리뷰 159: 친절해요 (2022년 12월 6일 화요일) - 작성자: tkd**** (사진 436)
리뷰 160: 좋ㅇㆍ요 (2022년 10월 15일 토요일) - 작성자: phr007 (Unknown)
로드된 리뷰 개수: 170, 수집된 리뷰 개수: 160
리뷰 161: 친절해요 (2022년 10월 25일 화요일) - 작성자: tkd**** (사진 436)
리뷰 162: 제손모가지다걸고싶습니다 어떻게증명해야되나요
맛도별로인데 불친절에 밥은 성겁게 3/2도안되게 담아서줘요 반찬 콩나물 콩자반 간장(간장은뭐찍어먹으라고주는지.?)이게다고,김치찌게도 별

In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
# 왕산골
# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 페이지로 이동
driver.get('https://map.naver.com/p/search/%EC%B2%AD%EB%8B%B4%EB%B0%B0%EC%A7%B1%EC%9D%B4/place/1776020426?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 이름 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색 (없으면 "없음" 출력)
    try:
        rating_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
        )
        rating_text = rating_element.text
        rating = rating_text.replace("별점", "").strip()
    except (NoSuchElementException, TimeoutException):
        rating = "없음"

    # 방문자 리뷰 검색 (없으면 "없음" 출력)
    try:
        visitor_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
        )
        visitor_review = visitor_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        visitor_review = "없음"

    # 블로그 리뷰 검색 (없으면 "없음" 출력)
    try:
        blog_review_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
        )
        blog_review = blog_review_element.text.split("리뷰")[1].strip()
    except (NoSuchElementException, TimeoutException):
        blog_review = "없음"

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    print("리뷰 탭 클릭 완료")

    # 리뷰 총 개수 가져오기
    review_count_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
    )
    total_reviews = int(review_count_element.text.replace(',', ''))
    print("총 리뷰 수:", total_reviews)
    print('\n')
    total_reviews = min(total_reviews, 1000)  # 최대 1000개의 리뷰로 제한

    reviews = []
    review_text_set = set()
    retry_count = 0
    max_retries = 3

    # '더보기' 버튼이 없어질 때까지 리뷰 수집
    while len(reviews) < total_reviews and retry_count < max_retries:
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        loaded_review_count = len(review_elements)
        print(f"로드된 리뷰 개수: {loaded_review_count}, 수집된 리뷰 개수: {len(reviews)}")

        if loaded_review_count > len(reviews):
            retry_count = 0  # 새로운 리뷰가 로드되면 리트라이 카운트 초기화
            for review_element in review_elements[len(reviews):]:
                try:
                    # 작성자 이름
                    try:
                        author_name = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[1]/span/span').text.strip()
                    except NoSuchElementException:
                        author_name = "Unknown"

                    # 리뷰 개수
                    try:
                        review_count_text = review_element.find_element(By.XPATH, './/div[1]/a[2]/div[2]/span[2]').text.strip()
                    except NoSuchElementException:
                        review_count_text = "Unknown"

                    # 리뷰 내용과 날짜
                    try:
                        review_text = review_element.find_element(By.XPATH, './/div[5]/a[1]').text.strip()
                        review_date = review_element.find_element(By.XPATH, './/div[7]/div[2]/div/span[1]/span[2]').text.strip()
                    except NoSuchElementException:
                        try:
                            review_text = review_element.find_element(By.XPATH, './/div[4]/a').text.strip()
                            review_date = review_element.find_element(By.XPATH, './/div[6]/div[2]/div/span[1]/span[2]').text.strip()
                        except NoSuchElementException:
                            try:
                                review_text = review_element.find_element(By.XPATH, './/div[6]/a').text.strip()
                                review_date = review_element.find_element(By.XPATH, './/div[8]/div[2]/div/span[1]/span[2]').text.strip()
                            except NoSuchElementException:
                                review_text = "내용 없음"
                                review_date = "날짜 없음"

                    # 중복 방지 및 저장
                    review_key = (author_name, review_text, review_date, review_count_text)
                    if review_key not in review_text_set:
                        reviews.append((review_date, review_text))
                        review_text_set.add(review_key)
                        print(f"리뷰 {len(reviews)}: {review_text} ({review_date}) - 작성자: {author_name} ({review_count_text})")

                    if len(reviews) >= total_reviews:
                        break
                except NoSuchElementException:
                    continue
        else:
            retry_count += 1
            print("새로운 리뷰가 로드되지 않아 스크롤을 조정하고 재시도합니다...", f"(재시도 {retry_count}/{max_retries})")
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            for scroll_pos in range(500, 8000, 500):
                driver.execute_script(f"window.scrollTo(0, {scroll_pos});")
                time.sleep(1)

        # '더보기' 버튼 클릭
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            time.sleep(3)
        except (NoSuchElementException, TimeoutException):
            print("'더보기' 버튼이 더 이상 없습니다.")
            break

    print(f"총 {len(reviews)}개의 리뷰를 수집했습니다. (예상 총 리뷰 수: {total_reviews})")

except Exception as e:
    print(f"오류 발생: {e}")

# 수집한 리뷰를 CSV 파일로 저장
if reviews:
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 청담배짱이
업종 카테고리: 낙지요리
별점: 방문자 리뷰 30
방문자 리뷰: 25
블로그 리뷰: 없음
리뷰 탭 클릭 완료
총 리뷰 수: 23


로드된 리뷰 개수: 10, 수집된 리뷰 개수: 0
리뷰 1: 훈제 삼겹 쭈꾸미 를 주문했어요.
훈제 삼겹이 훈제라서 맛있어요.
쭈꾸미는 약간 매운 편인데 같이 주는 콩나물 국에 씻어 먹으면 싱싱하고 맛있어요.
쭈꾸미 공장이 있어서 바로 오는 거라 그런지 쭈꾸미가 싱싱하고 오동통통 해요.
쭈꾸미는 강남 청담배짱이 추천합니다. (2024년 11월 4일 월요일) - 작성자: momoa2100 (사진 1,724)
리뷰 2: 진짜 너무맛있어요!!!!
또 다녀갑니다 총총..

꽃게 된찌 낙지 항상 맛있어여 양재맛집입니다!!!!!

그리고 반찬 샐러드 왕맛있어요 ㅎ.ㅎ (2024년 10월 23일 수요일) - 작성자: 한행복이 (사진 34)
리뷰 3: 진짜 너무너무 맛있게 잘먹었습니다
감칠맛있게 매운 쭈꾸미삼겹살과
채수된장도 너무 고소하고 진한맛이 일품이에요!
한번 꼭 방문해보세요 ㅎㅎ (2024년 10월 28일 월요일) - 작성자: 서이밤 (사진 90)
리뷰 4: 강남 낙지 맛집, 청담배짱이에 다녀왔어요.
비가 와서 매콤한 음식이 당겼는데 통통하고, 큼직한 낙지가 엄청 많이 들어가 있어 배불리 먹었네요.
밥에 낙지 양념 비벼서 먹으면 이게 바로 밥도둑!

참! 낙지볶음에 어울리는 채수된장찌개도 주문해보세요. 조금 얼얼해진 맛을 중화시켜주는 건강한 맛이었습니다. 지하에 위치해 있어 모르고 지나칠 뻔 했는데 더 많은 사람들이 알아봐주면 좋겠네요. 강남 낙지 맛집, 청담배짱이 최곱니다. (설거지하고 왔어요) (2024년 9월 12일 목요일) - 작성자: 베드포드 (사진 1,581)
리뷰 5: 직화낙지볶음이랑 버섯야채채수된장 먹었는데 너무 맛있었습니다 !! 사장님 엄청 친절하시고 매장도 넓고 깔금하더라구요 ㅎㅎ 직장인분들 점심식사로도 좋지만 회식하기에도 좋아보였어요 (2024년 11월 5일 화요일) - 작성자: lucey (사